Data collection and preprocessing

1. We're using collection created on https://github.com/norpadon/navalny and add Oleg Navalny's texts to it

In [2]:
import pandas as pd

# Load the existing corpus
file_path = 'data/fbk_archive.csv'
existing_corpus = pd.read_csv(file_path)

# Display the first few rows
#existing_corpus.head()


FileNotFoundError: [Errno 2] No such file or directory: 'data/fbk_archive.csv'

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

# Now you can access your environment variables as usual
fb_c_user = os.environ.get('FB_C_USER')
fb_xs = os.environ.get('FB_XS')
fb_datr = os.environ.get('FB_DATR')

In [17]:
# set up the necessary environment for scraping both Facebook and Instagram posts
import os
import requests

from typing import List, Iterable

# import csv
# import time
# import json
from datetime import datetime
# from dateutil.parser import parse as parse_date

# import bs4
import requests
from attrs import frozen
# from lxml import etree
# import feedparser
# from snscrape.modules.telegram import TelegramChannelScraper
from facebook_scraper import get_posts as get_facebook_posts
from ensta import Host as InstaClient
from tqdm.notebook import tqdm

@frozen
class TextEntry:
    author: str
    text: str
    date: datetime
    source: str
    source_type: str

# Load Instagram credentials
# insta_username = os.getenv('INSTA_USERNAME')
# insta_password = os.getenv('INSTA_PASSWORD')

# # Now you can use these variables to initialize the InstaClient
# insta_client = InstaClient(insta_username, insta_password)

# Load Facebook cookies
fb_cookies = requests.cookies.RequestsCookieJar()
fb_cookies.set('c_user', fb_c_user)
fb_cookies.set('xs', fb_xs)
fb_cookies.set('datr', fb_datr)
# Add other cookies as needed


Cookie(version=0, name='datr', value='xADmZBxtSh8_X6r9bKW-BhjR', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [5]:
#  scraping Oleg Navalny's posts from Facebook
OLEG_FB_URL = 'https://www.facebook.com/profile.php?id=100050656621768'
OLEG_FB_NAME = '100050656621768'  # You may need to adjust this based on the actual Facebook username


In [6]:
from facebook_scraper import get_posts as get_facebook_posts

def parse_fb_feed(author: str, username: str, pages: int = 100) -> List[TextEntry]:
    posts = get_facebook_posts(username, cookies=fb_cookies, pages=pages, options={"posts_per_page": 200})
    result = []
    for post in tqdm(posts, desc=f'Parsing facebook {username}'):
        date = post['time']
        text = post['post_text']
        if not text:
            continue
        source = post['post_url']
        source_type = f'facebook.com/{username}'
        result.append(TextEntry(author=author, date=date, text=text, source=source, source_type=source_type))

    return result

In [7]:
def parse_oleg_fb_feed():
    posts = parse_fb_feed('Oleg Navalny', OLEG_FB_NAME)
    oleg_navalny_jailed = []
    oleg_navalny_free = []
    for post in posts:
        if post.date < datetime(2018, 7, 1):
            oleg_navalny_jailed.append(post)
        else:
            oleg_navalny_free.append(post)
    return oleg_navalny_jailed, oleg_navalny_free

oleg_navalny_jailed_fb, oleg_navalny_free_fb = parse_oleg_fb_feed()


C:\Users\User\AppData\Roaming\Python\Python310\site-packages\facebook_scraper\facebook_scraper.py:855: UserWarning: Facebook language detected as en_GB - for best results, set to en_US
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\facebook_scraper\facebook_scraper.py:912: UserWarning: Facebook says 'Unsupported Browser'
  warnings.warn(f"Facebook says 'Unsupported Browser'")


Parsing facebook 100050656621768: 0it [00:00, ?it/s]

TemporarilyBanned: You’re Temporarily Blocked

In [18]:
from tqdm import tqdm

def parse_oleg_fb_feed():
    posts = parse_fb_feed('Oleg Navalny', OLEG_FB_NAME)
    oleg_navalny_jailed = []
    oleg_navalny_free = []
    for post in tqdm(posts, desc=f'Parsing facebook {OLEG_FB_NAME}'):
        # Print the date and source URL of the post
        print(f"Date: {post.date}, Source URL: {post.source}")

        if post.date < datetime(2018, 7, 1):
            oleg_navalny_jailed.append(post)
        else:
            oleg_navalny_free.append(post)
    return oleg_navalny_jailed, oleg_navalny_free


In [19]:
oleg_navalny_jailed_fb, oleg_navalny_free_fb = parse_oleg_fb_feed()

Parsing facebook 100050656621768: 308it [07:56,  1.55s/it]
Parsing facebook 100050656621768: 100%|██████████| 220/220 [00:00<00:00, 111577.62it/s]

Date: 2021-01-21 12:53:16, Source URL: https://facebook.com/100050656621768/posts/2836912636629388
Date: 2020-08-24 18:02:00, Source URL: https://facebook.com/100050656621768/posts/2704043689916284
Date: 2020-08-21 15:19:35, Source URL: https://facebook.com/100050656621768/posts/2701096116877708
Date: 2020-04-04 02:19:23, Source URL: https://facebook.com/100050656621768/posts/2576943759292945
Date: 2019-09-03 00:41:50, Source URL: https://facebook.com/100050656621768/posts/2379745652346091
Date: 2019-07-18 15:54:04, Source URL: https://facebook.com/100050656621768/posts/2345559325764724
Date: 2019-07-14 04:05:33, Source URL: https://facebook.com/100050656621768/posts/2342298252757498
Date: 2019-07-04 19:41:37, Source URL: https://facebook.com/100050656621768/posts/2335622196758437
Date: 2019-06-25 21:31:30, Source URL: https://facebook.com/100050656621768/posts/2328806927439964
Date: 2019-05-15 11:22:42, Source URL: https://facebook.com/100050656621768/posts/2297170247270299
Date: 2019

In [30]:
# Sort the jailed posts by date in ascending order
oleg_navalny_jailed_fb = sorted(oleg_navalny_jailed_fb, key=lambda post: post.date)

# Sort the free posts by date in ascending order
oleg_navalny_free_fb = sorted(oleg_navalny_free_fb, key=lambda post: post.date)


In [31]:
print(f"Number of posts while jailed: {len(oleg_navalny_jailed_fb)}")
print(f"Number of posts while free: {len(oleg_navalny_free_fb)}")


Number of posts while jailed: 160
Number of posts while free: 60


In [26]:
# Print the first jailed post
#print(oleg_navalny_jailed_fb[0].text)

# Print the first free post
print(oleg_navalny_free_fb[0].text)


Сегодня моему сыну Остапу 7 лет.
Он родился, когда я находился под следствием по делу «Ив Роше».
Ему не исполнилось года, когда меня отправили в колонию.
Первые слова от него: «а ты мой папа?» я услышал вЕщё комнате свиданий Ик-5 п. Нарышкино Орловской области.
Эта фотография сделана за две недели до того, как его дядю пытались убить химическим оружием.
Я не хочу, чтобы вся его жизнь сопровождалась подобными событиями. Не хочу, чтобы он рос в полицейском государстве, где людей отправляют в тюрьмы за правду, брошенный стаканчик или разбитое стекло, а лжецов и убийц награждают орденами. Не хочу, чтобы он своим трудом оплачивал «комнаты для грязи» во дворцах обезумевших дедов.
И я этого не допущу.
Смотрите последнее расследование ФБК, а 23 января выходите на центральную площадь своего города, чтобы его обсудить.


In [32]:
print(f"Jailed posts date range: {min(post.date for post in oleg_navalny_jailed_fb)} to {max(post.date for post in oleg_navalny_jailed_fb)}")
print(f"Free posts date range: {min(post.date for post in oleg_navalny_free_fb)} to {max(post.date for post in oleg_navalny_free_fb)}")


Jailed posts date range: 2015-10-25 17:18:00 to 2018-06-29 07:00:00
Free posts date range: 2018-07-02 10:48:10 to 2021-01-21 12:53:16


In [33]:
# Print first 5 jailed posts
print("First 5 jailed posts:")
for post in oleg_navalny_jailed_fb[:5]:
    print(post.date, post.text[:50])  # Print date and first 50 characters of text


First 5 jailed posts:
2015-10-25 17:18:00 http://7x7-journal.ru/item/70118
2015-10-27 23:59:28 http://www.newtimes.ru/articles/detail/103385
2015-10-28 20:31:48 "Нарушений Уголовного законодательства и прав чело
2015-10-29 18:37:53 http://www.novayagazeta.ru/politics/70527.html
2015-10-30 16:19:43 Всем привет!
Я не в ШИЗО, я уже на СУСе (адвокатам


In [34]:
# Print first 5 free posts
print("\nFirst 5 free posts:")
for post in oleg_navalny_free_fb[:5]:
    print(post.date, post.text[:50])  # Print date and first 50 characters of text


First 5 free posts:
2018-07-02 10:48:10 Король Карандашиков покинул Орловскую область и ве
2018-07-06 16:18:04 Ну, чего, дорогие вы мои? Прошла неделя с моего ос
2018-07-11 15:25:02 После того как национальная сборная по футболу не 
2018-07-12 17:20:52 А почитайте мой монолог на медиазоне https://zona.
2018-07-13 13:47:36 Что в первую очередь делает политзэк после освобож


In [37]:
# Save Oleg Navalny's posts separately
import csv

def write_posts_to_csv(posts, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['author', 'text', 'date', 'url', 'source_type'])
        writer.writeheader()  # Write the header row
        for post in posts:
            writer.writerow(dict(
                author=post.author,
                text=post.text,
                date=post.date.strftime('%Y-%m-%d'),
                url=post.source,
                source_type=post.source_type
            ))

# Save posts to separate files
write_posts_to_csv(oleg_navalny_jailed_fb, '../data/oleg_navalny_jailed.csv')
write_posts_to_csv(oleg_navalny_free_fb, '../data/oleg_navalny_free.csv')


In [11]:
# Read all the text files to check whether they have the same structure

# Read existing FBK archive
fbk_archive = []
with open('../data/fbk_archive.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        fbk_archive.append(row)

# Read Oleg's jailed posts
oleg_navalny_jailed_fb = []
with open('../data/oleg_navalny_jailed.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        oleg_navalny_jailed_fb.append(row)

# Read Oleg's free posts
oleg_navalny_free_fb = []
with open('../data/oleg_navalny_free.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        oleg_navalny_free_fb.append(row)

# Print the first few rows of each dataset to verify
fbk_archive[:2], oleg_navalny_jailed_fb[:2], oleg_navalny_free_fb[:2]


([{'Navalny': 'Navalny',
   "I wanted to write about this for a long time. Well, let it be the first post after the new sentence. It's like a confession. I need to get over this loathing and fear, maybe you can help me with this.Loathing. People ask me a lot about it, and I started receiving letters again: do you hate the judge? Do you hate Putin even more? I have said many times before that hate is the main thing that must be overcome in prison. There are so many reasons for it, and your powerlessness is a strong catalyst for the process. So if you let it go, it will eat and end you up.I'll be honest, I have a hatred and I've got a lot of it. Old-users of the Internet will remember that meme: I fiercely, wildly hate it. Something like this. Most often I have it after «trials». The last one, by the way, where I got 19 years, was not one of those. There, on the contrary, we all competed in showering each other with pleasantries. During the whole process, no one raised their voice once. 

In [12]:
# It seems that the fbk_archive.csv file does not have a well-structured CSV format
# Instead of having a header row that defines the field names, the CSV file appears to be composed of rows that represent the post's data directly.

# We can still read and process this data by mapping each row to the expected field names manually.

# Function to read CSV file with the structure found in fbk_archive.csv
def read_fbk_archive(file_path):
    posts = []
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            post = {
                'author': row[0],
                'text': row[1],
                'date': row[2],
                'url': row[3],
                'source_type': ''  # You can replace this with the appropriate value if available
            }
            posts.append(post)
    return posts

# Read FBK archive using the custom function
fbk_archive_custom = read_fbk_archive('../data/fbk_archive.csv')


In [142]:
# Combine Oleg's texts with the rest of the FBK texts

# Combine the archives
combined_archive_custom = fbk_archive_custom + oleg_navalny_jailed_fb + oleg_navalny_free_fb

# Write to the new CSV file
with open('../data/fbk_oleg_archive.csv', 'w', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['author', 'text', 'date', 'url', 'source_type'])
    writer.writeheader()  # Write the header row
    for entry in combined_archive_custom:
        writer.writerow(entry)

In [143]:
# Check unique authors that the combined text archive contains

# Reading the combined CSV file
with open('../data/fbk_oleg_archive.csv', 'r', encoding='utf-8') as file:
    combined_reader = csv.DictReader(file)
    authors = [row['author'] for row in combined_reader]

# Finding the unique authors
unique_authors = set(authors)
unique_authors  

{'Alburov',
 'Navalny',
 'Oleg Navalny',
 'Pevchikh',
 'Sobol',
 'Volkov',
 'Yarmysh',
 'Zhdanov'}

In [43]:
# Check the text time range for each author

# Reading the combined archive
archive_path = "../data/fbk_oleg_archive.csv"
combined_archive = pd.read_csv(archive_path)

# Converting the 'date' column to a datetime object for easy manipulation
combined_archive['date'] = pd.to_datetime(combined_archive['date'])

# Finding the first and last dates for each author
author_first_last_dates = combined_archive.groupby('author')['date'].agg(['min', 'max'])
author_first_last_dates.reset_index(inplace=True)
author_first_last_dates.rename(columns={'min': 'first_date', 'max': 'last_date'}, inplace=True)
author_first_last_dates

,author,first_date,last_date
0,Alburov,2011-09-10,2023-08-15
1,Navalny,2009-01-12,2023-08-17
2,Oleg Navalny,2015-10-25,2021-01-21
3,Pevchikh,2021-08-09,2023-08-16
4,Sobol,2008-01-14,2023-08-14
5,Volkov,2012-11-07,2023-08-17
6,Yarmysh,2014-08-30,2023-08-04
7,Zhdanov,2012-02-16,2023-08-17


In [44]:
# Check what source domains each author has 

from urllib.parse import urlparse

# Dictionary to store unique domains for each author
author_domains = {}

# Iterate through the archive and extract unique domains for each author
for index, row in combined_archive.iterrows():
    author = row['author']
    url = row['url']
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    
    # Initialize the set of domains for the author if not already present
    if author not in author_domains:
        author_domains[author] = set()
    
    # Add the domain to the author's set
    author_domains[author].add(domain)

# Display the unique domains for each author
author_domains

{'Navalny': {'facebook.com', 'navalny.com', 't.me', 'www.instagram.com'},
 'Volkov': {'facebook.com',
  'leonwolf.livejournal.com',
  't.me',
  'www.instagram.com'},
 'Sobol': {'facebook.com',
  'jenya-khimles.livejournal.com',
  'sobollubov.livejournal.com',
  't.me',
  'www.instagram.com'},
 'Pevchikh': {'www.instagram.com'},
 'Alburov': {'facebook.com', 'www.instagram.com'},
 'Zhdanov': {'facebook.com', 't.me', 'www.instagram.com'},
 'Yarmysh': {'t.me', 'www.instagram.com'},
 'Oleg Navalny': {'facebook.com'}}

In [144]:
# Let's search our new text archive for some particular posts

def search_text_in_archive(search_phrase, file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if search_phrase in row['text']:
                print("Found the specified phrase in the text!")
                return row
        else:
            print("Specified phrase not found in the text.")
            return None

# Usage
searched_phrase = "Свободу Алексею Навальному"
file_path = '../data/fbk_oleg_archive.csv'
found_entry = search_text_in_archive(searched_phrase, file_path)

if found_entry:
    print(found_entry)  # This will print the entry (keys and values) containing the found text


Found the specified phrase in the text!
{'author': 'Navalny', 'text': 'Сегодня Алексею Навальному вынесут приговор. Прокурор запросил 20 лет лишения свободы в колонии особого режима. Чудовищный, сталинский срок для любого человека, но при этом мы понимаем, что Алексея судят как главного оппозиционного политика в России.Все, что происходит с Алексеем Навальным, — это преступление. Заказчик у этого преступления понятен. Он сидит в Кремле. Это преступление совершено большой группой, и в нее входят не только следователи, прокуроры, судьи, но и люди, которые непосредственно пытались убить Алексея. ФСБ. И сейчас мы вам это докажем.Но сначала давайте о самом деле — и о том, за что Навального, собственно, «судили». Этот процесс, который сегодня завершится приговором, можно назвать самым засекреченным. Суд снова проходил в колонии, внутри тюрьмы, в актовом зале. Туда не пускали ни зрителей, ни журналистов. Не пустили даже родителей Алексея. Адвокатам запретили вообще хоть что-то рассказывать о 

In [245]:
# Let's search our new text archive for some particular posts

import csv

def search_text_in_archive(search_phrase=None, file_path=None, author=None, date_range=None, domains=None, key_urls=None):
    found_entries = []
    total_entries = 0  # Counter for the total number of entries checked

    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            total_entries += 1 
            match_found = False
            
            # If search_phrase is provided, look for it in the text
            if search_phrase and search_phrase in row['text']:
                match_found = True
            
            # If key_urls are provided, look for them in the URL field
            if key_urls and row['url'] in key_urls:
                match_found = True

            # If a match is found, apply the additional filters
            if match_found:
                if author and row['author'] != author:
                    continue
                if date_range and (row['date'] < date_range[0] or row['date'] > date_range[1]):
                    continue
                if domains and not any(domain in row['url'] for domain in domains):
                    continue
                print(f"Found the specified phrase or URL: {row}")
                found_entries.append(row)

    if not found_entries:
        print("Specified phrase or URL not found.")
        return None, total_entries

    print(f"Total number of entries checked: {total_entries} \nTotal number of posts found: {len(found_entries)} ")
    return found_entries, total_entries


# Usage
searched_phrase = "Channel created"
file_path = '../data/fbk_oleg_archive.csv'
author = 'Navalny'
#date_range = ['2022-01-01', '2022-12-31']
domains = ['t.me']
#key_urls = ['https://t.me/navalny/2970']

found_entries = search_text_in_archive(searched_phrase, file_path, author, None, domains, None)


Found the specified phrase or URL: {'author': 'Navalny', 'text': 'Channel created', 'date': '2017-11-15', 'url': 'https://t.me/s/navalny/1', 'source_type': ''}
Total number of entries checked: 39471 
Total number of posts found: 1 


In [122]:
# Read the combined archive and find the last Telegram post from Navalny
last_telegram_post = None

with open('../data/fbk_oleg_archive.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if row['author'] == 'Navalny' and "t.me" in row['url']:
            last_telegram_post = row

# Display the last Telegram post
last_telegram_post

{'author': 'Navalny',
 'text': 'Channel created',
 'date': '2017-11-15',
 'url': 'https://t.me/s/navalny/1',
 'source_type': ''}

In [220]:
# We copied the archive to fbk_oleg_archive_clean.csv

# Copy the archive
import shutil

existing_archive_path = "../data/fbk_oleg_archive.csv"
cleaned_archive_path = "../data/fbk_oleg_archive_clean.csv"

# Copy the existing archive to create a cleaned version
shutil.copy(existing_archive_path, cleaned_archive_path)

# Confirming that the file has been created
cleaned_archive_path

'../data/fbk_oleg_archive_clean.csv'

Data collection and preprocessing

2. We're meticulously selecting (sometimes manually) Alexey Navalny's texts to ensure the corpus only contains his writings, excluding repetitions and texts clearly authored by his team, which could impact authorship analysis.

In [221]:
def remove_or_keep_entries(cleaned_archive_path, author=None, url_domains=None, key_wordings=None, key_urls=None, date_range=None, keep_instead_of_remove=False):
    # Read the existing archive
    with open(cleaned_archive_path, 'r', encoding='utf-8') as file:
        archive = list(csv.DictReader(file))

    def matches_criteria(row):
        if author and row['author'] != author:
            return False
        if url_domains and not any(domain in row['url'] for domain in url_domains):
            return False
        if date_range and (row['date'] < date_range[0] or row['date'] > date_range[1]):
            return False
        return True

    initial_count = sum(matches_criteria(row) for row in archive)
    print(f"Initial number of matching posts (Author, Domain, Dates): {initial_count}")

    def should_remove_or_keep(row):
        if not matches_criteria(row):
            return False if keep_instead_of_remove else True

        if key_wordings and not any(wording in row['text'] for wording in key_wordings):
            return True
        if key_urls and row['url'] not in key_urls:
            return True

        return False

    if keep_instead_of_remove:
        filtered_archive = [row for row in archive if should_remove_or_keep(row) == False]
    else:
        filtered_archive = [row for row in archive if should_remove_or_keep(row) == True]

    final_count = sum(matches_criteria(row) for row in filtered_archive)
    if keep_instead_of_remove:
        print(f"Number of posts to keep: {final_count}")
        print(f"Number of posts removed: {initial_count - final_count}")
        print(f"Number of posts of this Author, Domain, Dates left after removal: {final_count}")

    else:
        print(f"Number of posts removed: {initial_count - final_count}")
        print(f"Number of posts of this Author, Domain, Dates left after removal: {final_count}")

    # Write the filtered archive back to the file
    with open(cleaned_archive_path, 'w', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['author', 'text', 'date', 'url', 'source_type'])
        writer.writeheader()
        writer.writerows(filtered_archive)



In [246]:
# Posts that end up with “Свободу Алексею Навальному” or “Свободу Навальному” 
# are highly likely written by the ACF team, not by Alexey himself, so should be removed

key_wordings_team=['Свободу Алексею Навальному', 'Свободу Навальному']

remove_or_keep_entries(
    cleaned_archive_path='../data/fbk_oleg_archive_clean.csv',
    author='Navalny',
    url_domains=['navalny.com'],
    key_wordings= key_wordings_team,
    key_urls= None,
    date_range= None,
    keep_instead_of_remove=False
)

Initial number of matching posts (Author, Domain, Dates): 2978
Number of posts removed: 53
Number of posts of this Author, Domain, Dates left after removal: 2925


In [433]:
# Remove texts from after the imprisonment and during the poisoning coma
# that were posted on navalny.com not in the name of Navalny 

key_urls_n_site_jail_coma = (
    #after jail
    'https://navalny.com/p/6630/',
    'https://navalny.com/p/6625/',
    'https://navalny.com/p/6619/',
    'https://navalny.com/p/6615/',
    'https://navalny.com/p/6608/',
    'https://navalny.com/p/6607/',
    'https://navalny.com/p/6605/',
    'https://navalny.com/p/6603/',
    'https://navalny.com/p/6602/',
    'https://navalny.com/p/6565/',
    'https://navalny.com/p/6552/',
    'https://navalny.com/p/6547/',
    'https://navalny.com/p/6544/',
    'https://navalny.com/p/6533/',
    'https://navalny.com/p/6528/',
    'https://navalny.com/p/6524/',
    'https://navalny.com/p/6517/',
    'https://navalny.com/p/6518/',
    'https://navalny.com/p/6514/',
    'https://navalny.com/p/6509/',
    'https://navalny.com/p/6505/',
    'https://navalny.com/p/6498/',
    'https://navalny.com/p/6496/',
    'https://navalny.com/p/6497/',
    'https://navalny.com/p/6495/',
    'https://navalny.com/p/6494/',
    'https://navalny.com/p/6491/',
    'https://navalny.com/p/6492/',
    'https://navalny.com/p/6488/',
    'https://navalny.com/p/6486/',
    'https://navalny.com/p/6484/',
    'https://navalny.com/p/6483/',
    'https://navalny.com/p/6482/',
    'https://navalny.com/p/6481/',
    'https://navalny.com/p/6479/',
    'https://navalny.com/p/6478/',
    'https://navalny.com/p/6475/',
    'https://navalny.com/p/6474/',
    'https://navalny.com/p/6473/',
    'https://navalny.com/p/6472/',
    'https://navalny.com/p/6470/',
    'https://navalny.com/p/6468/',
    'https://navalny.com/p/6467/',
    'https://navalny.com/p/6466/',
    'https://navalny.com/p/6461/',
    'https://navalny.com/p/6458/',
    'https://navalny.com/p/6457/',
    'https://navalny.com/p/6455/',
    'https://navalny.com/p/6454/',
    #poisoning & coma
    'https://navalny.com/p/6430/',
    'https://navalny.com/p/6426/',
    'https://navalny.com/p/6425/',
    'https://navalny.com/p/6422/',
    'https://navalny.com/p/6421/',
)

key_urls_n_site_jail_2 = (
'https://navalny.com/p/6453/',
'https://navalny.com/p/6459/',
'https://navalny.com/p/6489/',
'https://navalny.com/p/6535/',
'https://navalny.com/p/6543/',
'https://navalny.com/p/6577/',
'https://navalny.com/p/6579/',
'https://navalny.com/p/6588/',
'https://navalny.com/p/6587/',
'https://navalny.com/p/6610/',
'https://navalny.com/p/6612/',
'https://navalny.com/p/6622/',
'https://navalny.com/p/6628/',
'https://navalny.com/p/6635/',
'https://navalny.com/p/6648/',
'https://navalny.com/p/6650/',
'https://navalny.com/p/6652/',
'https://navalny.com/p/6476/',
'https://navalny.com/p/6460/',
'https://navalny.com/p/6510/',
'https://navalny.com/p/6493/',
'https://navalny.com/p/6464/',
'https://navalny.com/p/6563/')

remove_or_keep_entries(
    cleaned_archive_path='../data/fbk_oleg_archive_clean.csv',
    author='Navalny',
    url_domains=['navalny.com'],
    key_wordings= None,
    key_urls= key_urls_n_site_jail_2,
    date_range= None,
    keep_instead_of_remove=False
)

Initial number of matching posts (Author, Domain, Dates): 2797
Number of posts removed: 1
Number of posts of this Author, Domain, Dates left after removal: 2796


In [251]:
# Remove texts from after the imprisonment and during the poisoning coma
# that are Youtube announces on navalny.com

key_urls_youtube = (
    'https://navalny.com/p/6601/',
    'https://navalny.com/p/6600/',
    'https://navalny.com/p/6596/',
    'https://navalny.com/p/6595/',
    'https://navalny.com/p/6594/',
    'https://navalny.com/p/6593/',
    'https://navalny.com/p/6592/',
    'https://navalny.com/p/6591/',
    'https://navalny.com/p/6590/',
    'https://navalny.com/p/6589/',
    'https://navalny.com/p/6585/',
    'https://navalny.com/p/6584/',
    'https://navalny.com/p/6583/',
    'https://navalny.com/p/6582/',
    'https://navalny.com/p/6581/',
    'https://navalny.com/p/6580/',
    'https://navalny.com/p/6578/',
    'https://navalny.com/p/6576/',
    'https://navalny.com/p/6575/',
    'https://navalny.com/p/6574/',
    'https://navalny.com/p/6572/',
    'https://navalny.com/p/6571/',
    'https://navalny.com/p/6569/',
    'https://navalny.com/p/6568/',
    'https://navalny.com/p/6567/',
    'https://navalny.com/p/6566/',
    'https://navalny.com/p/6562/',
    'https://navalny.com/p/6561/',
    'https://navalny.com/p/6560/',
    'https://navalny.com/p/6559/',
    'https://navalny.com/p/6558/',
    'https://navalny.com/p/6557/',
    'https://navalny.com/p/6556/',
    'https://navalny.com/p/6555/',
    'https://navalny.com/p/6554/',
    'https://navalny.com/p/6553/',
    'https://navalny.com/p/6551/',
    'https://navalny.com/p/6550/',
    'https://navalny.com/p/6549/',
    'https://navalny.com/p/6548/',
    'https://navalny.com/p/6545/',
    'https://navalny.com/p/6541/',
    'https://navalny.com/p/6540/',
    'https://navalny.com/p/6537/',
    'https://navalny.com/p/6536/',
    'https://navalny.com/p/6532/',
    'https://navalny.com/p/6527/',
    'https://navalny.com/p/6523/',
    'https://navalny.com/p/6522/',
    'https://navalny.com/p/6520/',
    'https://navalny.com/p/6519/',
    'https://navalny.com/p/6516/',
    'https://navalny.com/p/6515/',
    'https://navalny.com/p/6512/',
    'https://navalny.com/p/6511/',
    'https://navalny.com/p/6508/',
    'https://navalny.com/p/6507/',
    'https://navalny.com/p/6506/',
    'https://navalny.com/p/6504/',
    'https://navalny.com/p/6503/',
    'https://navalny.com/p/6501/',
    'https://navalny.com/p/6500/',
    'https://navalny.com/p/6499/',
)

remove_or_keep_entries(
    cleaned_archive_path='../data/fbk_oleg_archive_clean.csv',
    author='Navalny',
    url_domains=['navalny.com'],
    key_wordings= None,
    key_urls= key_urls_youtube,
    date_range= None,
    keep_instead_of_remove=False
)


Initial number of matching posts (Author, Domain, Dates): 2875
Number of posts removed: 59
Number of posts of this Author, Domain, Dates left after removal: 2816


In [434]:
# Remove telegram posts during the poisoning coma that were not written by Navalny himself 

key_urls_n_telegram_coma = ('https://t.me/s/navalny/2620',
 'https://t.me/s/navalny/2621',
 'https://t.me/s/navalny/2622',
 'https://t.me/s/navalny/2623',
 'https://t.me/s/navalny/2624',
 'https://t.me/s/navalny/2625',
 'https://t.me/s/navalny/2626',
 'https://t.me/s/navalny/2627',
 'https://t.me/s/navalny/2628',
 'https://t.me/s/navalny/2630',
 'https://t.me/s/navalny/2633',
 'https://t.me/s/navalny/2634',
 'https://t.me/s/navalny/2635',
 'https://t.me/s/navalny/2637',
 'https://t.me/s/navalny/2638',
 'https://t.me/s/navalny/2639',
 'https://t.me/s/navalny/2640',
 'https://t.me/s/navalny/2641',
 'https://t.me/s/navalny/2642',
 'https://t.me/s/navalny/2643',
 'https://t.me/s/navalny/2644',
 'https://t.me/s/navalny/2645',
 'https://t.me/s/navalny/2646',
 'https://t.me/s/navalny/2647',
 'https://t.me/s/navalny/2648',
 'https://t.me/s/navalny/2651',
 'https://t.me/s/navalny/2656',
 'https://t.me/s/navalny/2657',
 'https://t.me/s/navalny/2658',
 'https://t.me/s/navalny/2659',
 'https://t.me/s/navalny/2660',
 'https://t.me/s/navalny/2663',
 'https://t.me/s/navalny/2669',
 'https://t.me/s/navalny/2671',
 'https://t.me/s/navalny/2675',
 'https://t.me/s/navalny/2676',
 'https://t.me/s/navalny/2680',
 'https://t.me/s/navalny/2684',
 'https://t.me/s/navalny/2688',
 'https://t.me/s/navalny/2694',
 'https://t.me/s/navalny/2698',
 'https://t.me/s/navalny/2703',
 'https://t.me/s/navalny/2704',
 'https://t.me/s/navalny/2705',
 'https://t.me/s/navalny/2712',
 'https://t.me/s/navalny/2715',
 'https://t.me/s/navalny/2723',
 'https://t.me/s/navalny/2734',
 'https://t.me/s/navalny/2735',
 'https://t.me/s/navalny/2736',
 'https://t.me/s/navalny/2741',
 'https://t.me/s/navalny/2744',
 'https://t.me/s/navalny/2747',
 'https://t.me/s/navalny/2749',
 'https://t.me/s/navalny/2751',
 'https://t.me/s/navalny/2752',
 'https://t.me/s/navalny/2754',
 'https://t.me/s/navalny/2760',
 'https://t.me/s/navalny/2764',
 'https://t.me/s/navalny/2765',
 'https://t.me/s/navalny/2767',
 'https://t.me/s/navalny/2768',
 'https://t.me/s/navalny/2770',
 'https://t.me/s/navalny/2772',
 'https://t.me/s/navalny/2773',
 'https://t.me/s/navalny/2775',
 'https://t.me/s/navalny/2778',
 'https://t.me/s/navalny/2779',
 'https://t.me/s/navalny/2781',
 'https://t.me/s/navalny/2785',
 'https://t.me/s/navalny/2786',
 'https://t.me/s/navalny/2788',
 'https://t.me/s/navalny/2790',
 'https://t.me/s/navalny/2792',
 'https://t.me/s/navalny/2795',
 'https://t.me/s/navalny/2798',
 'https://t.me/s/navalny/2802',
 'https://t.me/s/navalny/2804',
 'https://t.me/s/navalny/2810',
 'https://t.me/s/navalny/2811',
 'https://t.me/s/navalny/2812',
 'https://t.me/s/navalny/2815',
 'https://t.me/s/navalny/2818',
 'https://t.me/s/navalny/2819',
 'https://t.me/s/navalny/2820',
 'https://t.me/s/navalny/2821',
 'https://t.me/s/navalny/2827',
 'https://t.me/s/navalny/2829',
 'https://t.me/s/navalny/2830',
 'https://t.me/s/navalny/2834',
 'https://t.me/s/navalny/2837',
 'https://t.me/s/navalny/2845',
 'https://t.me/s/navalny/2859',
 'https://t.me/s/navalny/2860',
 'https://t.me/s/navalny/2866',
 'https://t.me/s/navalny/2868',
 'https://t.me/s/navalny/2870',
 'https://t.me/s/navalny/2871',
 'https://t.me/s/navalny/2874',
 'https://t.me/s/navalny/2878',
 'https://t.me/s/navalny/2882',
 'https://t.me/s/navalny/2883',
 'https://t.me/s/navalny/2884',
 'https://t.me/s/navalny/2886',
 'https://t.me/s/navalny/2887',
 'https://t.me/s/navalny/2889',
 'https://t.me/s/navalny/2890',
 'https://t.me/s/navalny/2892',
 'https://t.me/s/navalny/2893',
 'https://t.me/s/navalny/2894',
 'https://t.me/s/navalny/2897',
 'https://t.me/s/navalny/2898',
 'https://t.me/s/navalny/2902',
 'https://t.me/s/navalny/2904',
 'https://t.me/s/navalny/2905',
 'https://t.me/s/navalny/2906',
 'https://t.me/s/navalny/2908',
 'https://t.me/s/navalny/2913',
 'https://t.me/s/navalny/2914',
 'https://t.me/s/navalny/2915',
 'https://t.me/s/navalny/2920',
 'https://t.me/s/navalny/2925')

# Duplicate posts to remove
key_urls_n_telegram_jail2 = (
'https://t.me/s/navalny/3242',
'https://t.me/s/navalny/3368',
'https://t.me/s/navalny/3413',
'https://t.me/s/navalny/3477',
'https://t.me/s/navalny/3482',
'https://t.me/s/navalny/3057',
'https://t.me/s/navalny/3225',
'https://t.me/s/navalny/3005')

remove_or_keep_entries(
    cleaned_archive_path='../data/fbk_oleg_archive_clean.csv',
    author='Navalny',
    url_domains=['t.me'],
    key_wordings= None,
    key_urls= key_urls_n_telegram_jail2,
    date_range= None,
    keep_instead_of_remove=False
)

Initial number of matching posts (Author, Domain, Dates): 2564
Number of posts removed: 1
Number of posts of this Author, Domain, Dates left after removal: 2563


In [258]:
# Keep from the Navalny Telegram channel's posts after his detainment (2021.01.18)
# only the posts written from the first person and remove everything else

key_urls_n_telegram_jail = ('https://t.me/s/navalny/2970',
 'https://t.me/s/navalny/2973',
 'https://t.me/s/navalny/2975',
 'https://t.me/s/navalny/2986',
 'https://t.me/s/navalny/2997',
 'https://t.me/s/navalny/3003',
 'https://t.me/s/navalny/3004',
 'https://t.me/s/navalny/3005',
 'https://t.me/s/navalny/3006',
 'https://t.me/s/navalny/3013',
 'https://t.me/s/navalny/3017',
 'https://t.me/s/navalny/3051',
 'https://t.me/s/navalny/3057',
 'https://t.me/s/navalny/3059',
 'https://t.me/s/navalny/3072',
 'https://t.me/s/navalny/3081',
 'https://t.me/s/navalny/3086',
 'https://t.me/s/navalny/3093',
 'https://t.me/s/navalny/3096',
 'https://t.me/s/navalny/3097',
 'https://t.me/s/navalny/3098',
 'https://t.me/s/navalny/3101',
 'https://t.me/s/navalny/3104',
 'https://t.me/s/navalny/3106',
 'https://t.me/s/navalny/3108',
 'https://t.me/s/navalny/3109',
 'https://t.me/s/navalny/3112',
 'https://t.me/s/navalny/3116',
 'https://t.me/s/navalny/3120',
 'https://t.me/s/navalny/3122',
 'https://t.me/s/navalny/3125',
 'https://t.me/s/navalny/3128',
 'https://t.me/s/navalny/3129',
 'https://t.me/s/navalny/3130',
 'https://t.me/s/navalny/3131',
 'https://t.me/s/navalny/3132',
 'https://t.me/s/navalny/3133',
 'https://t.me/s/navalny/3134',
 'https://t.me/s/navalny/3136',
 'https://t.me/s/navalny/3137',
 'https://t.me/s/navalny/3143',
 'https://t.me/s/navalny/3146',
 'https://t.me/s/navalny/3149',
 'https://t.me/s/navalny/3157',
 'https://t.me/s/navalny/3160',
 'https://t.me/s/navalny/3170',
 'https://t.me/s/navalny/3174',
 'https://t.me/s/navalny/3177',
 'https://t.me/s/navalny/3182',
 'https://t.me/s/navalny/3186',
 'https://t.me/s/navalny/3189',
 'https://t.me/s/navalny/3193',
 'https://t.me/s/navalny/3196',
 'https://t.me/s/navalny/3202',
 'https://t.me/s/navalny/3203',
 'https://t.me/s/navalny/3204',
 'https://t.me/s/navalny/3207',
 'https://t.me/s/navalny/3210',
 'https://t.me/s/navalny/3212',
 'https://t.me/s/navalny/3213',
 'https://t.me/s/navalny/3216',
 'https://t.me/s/navalny/3222',
 'https://t.me/s/navalny/3225',
 'https://t.me/s/navalny/3227',
 'https://t.me/s/navalny/3229',
 'https://t.me/s/navalny/3230',
 'https://t.me/s/navalny/3234',
 'https://t.me/s/navalny/3239',
 'https://t.me/s/navalny/3242',
 'https://t.me/s/navalny/3243',
 'https://t.me/s/navalny/3244',
 'https://t.me/s/navalny/3246',
 'https://t.me/s/navalny/3252',
 'https://t.me/s/navalny/3253',
 'https://t.me/s/navalny/3254',
 'https://t.me/s/navalny/3259',
 'https://t.me/s/navalny/3262',
 'https://t.me/s/navalny/3264',
 'https://t.me/s/navalny/3265',
 'https://t.me/s/navalny/3270',
 'https://t.me/s/navalny/3271',
 'https://t.me/s/navalny/3272',
 'https://t.me/s/navalny/3273',
 'https://t.me/s/navalny/3274',
 'https://t.me/s/navalny/3276',
 'https://t.me/s/navalny/3278',
 'https://t.me/s/navalny/3279',
 'https://t.me/s/navalny/3282',
 'https://t.me/s/navalny/3283',
 'https://t.me/s/navalny/3284',
 'https://t.me/s/navalny/3285',
 'https://t.me/s/navalny/3286',
 'https://t.me/s/navalny/3289',
 'https://t.me/s/navalny/3292',
 'https://t.me/s/navalny/3294',
 'https://t.me/s/navalny/3295',
 'https://t.me/s/navalny/3298',
 'https://t.me/s/navalny/3302',
 'https://t.me/s/navalny/3305',
 'https://t.me/s/navalny/3306',
 'https://t.me/s/navalny/3307',
 'https://t.me/s/navalny/3309',
 'https://t.me/s/navalny/3310',
 'https://t.me/s/navalny/3312',
 'https://t.me/s/navalny/3313',
 'https://t.me/s/navalny/3316',
 'https://t.me/s/navalny/3317',
 'https://t.me/s/navalny/3318',
 'https://t.me/s/navalny/3319',
 'https://t.me/s/navalny/3320',
 'https://t.me/s/navalny/3323',
 'https://t.me/s/navalny/3324',
 'https://t.me/s/navalny/3325',
 'https://t.me/s/navalny/3326',
 'https://t.me/s/navalny/3327',
 'https://t.me/s/navalny/3328',
 'https://t.me/s/navalny/3330',
 'https://t.me/s/navalny/3331',
 'https://t.me/s/navalny/3332',
 'https://t.me/s/navalny/3333',
 'https://t.me/s/navalny/3334',
 'https://t.me/s/navalny/3336',
 'https://t.me/s/navalny/3337',
 'https://t.me/s/navalny/3339',
 'https://t.me/s/navalny/3343',
 'https://t.me/s/navalny/3355',
 'https://t.me/s/navalny/3357',
 'https://t.me/s/navalny/3358',
 'https://t.me/s/navalny/3359',
 'https://t.me/s/navalny/3361',
 'https://t.me/s/navalny/3362',
 'https://t.me/s/navalny/3364',
 'https://t.me/s/navalny/3365',
 'https://t.me/s/navalny/3368',
 'https://t.me/s/navalny/3383',
 'https://t.me/s/navalny/3388',
 'https://t.me/s/navalny/3396',
 'https://t.me/s/navalny/3397',
 'https://t.me/s/navalny/3398',
 'https://t.me/s/navalny/3400',
 'https://t.me/s/navalny/3403',
 'https://t.me/s/navalny/3407',
 'https://t.me/s/navalny/3409',
 'https://t.me/s/navalny/3410',
 'https://t.me/s/navalny/3413',
 'https://t.me/s/navalny/3414',
 'https://t.me/s/navalny/3415',
 'https://t.me/s/navalny/3416',
 'https://t.me/s/navalny/3424',
 'https://t.me/s/navalny/3428',
 'https://t.me/s/navalny/3438',
 'https://t.me/s/navalny/3441',
 'https://t.me/s/navalny/3442',
 'https://t.me/s/navalny/3443',
 'https://t.me/s/navalny/3444',
 'https://t.me/s/navalny/3456',
 'https://t.me/s/navalny/3457',
 'https://t.me/s/navalny/3459',
 'https://t.me/s/navalny/3460',
 'https://t.me/s/navalny/3461',
 'https://t.me/s/navalny/3462',
 'https://t.me/s/navalny/3463',
 'https://t.me/s/navalny/3464',
 'https://t.me/s/navalny/3465',
 'https://t.me/s/navalny/3466',
 'https://t.me/s/navalny/3468',
 'https://t.me/s/navalny/3470',
 'https://t.me/s/navalny/3471',
 'https://t.me/s/navalny/3472',
 'https://t.me/s/navalny/3474',
 'https://t.me/s/navalny/3475',
 'https://t.me/s/navalny/3476',
 'https://t.me/s/navalny/3477',
 'https://t.me/s/navalny/3480',
 'https://t.me/s/navalny/3482',
 'https://t.me/s/navalny/3484',
 'https://t.me/s/navalny/3487')

remove_or_keep_entries(
    cleaned_archive_path='../data/fbk_oleg_archive_clean.csv',
    author='Navalny',
    url_domains=['t.me'],
    key_wordings=None,
    key_urls=key_urls_n_telegram_jail,
    date_range=('2021-01-18', '9999-12-31'),  # Starting from 18th Jan 2021 to far future
    keep_instead_of_remove=True
)


Initial number of matching posts (Author, Domain, Dates): 500
Number of posts to keep: 175
Number of posts removed: 325
Number of posts of this Author, Domain, Dates left after removal: 175


In [291]:
# From Navalny's Facebook page keep only those after-jail posts (>2021.01.18) that
# do not duplicate posts from his Telegram channel

key_wordings_n_fb_jail = ('Поздравляю с юбилеем Ольгу Михайлову',
'Очень надеюсь, что этот мой пост вам не нужен',
'Бывают же классные совпадения. Некоторое время',
'Я хочу выразить свою поддержку каналу',
'Вести об уголовном деле Собчак дошли',
'Ваша честь, вы видите каждый день, что происходит',
'Алексея Навального попросили написать приветственное',
'Я крайне признателен и благодарен Международному',
'Дорогие все. Я хочу сказать, что вы очень грубо',
'А иногда мне ясно, что мое заключение - это никакое',
'Путин - царь лжи. Он и в обычное время чемпион',
'Пост для таких, как я сам. Тех, для кого один из',
'Правда и свободная информация бьют по безумному',
'Все больше подтверждений тому, что я действительно',
'Илья Яшин - честный человек и смелый политик',
'Встречая Дмитрия Борисовича Зимина',
'Легитимность — очень непростая штука',
'Лилия Чанышева работала аудитором',
'От всей души поздравляю Дмитрия Муратова')

remove_or_keep_entries(
    cleaned_archive_path='../data/fbk_oleg_archive_clean.csv',
    author='Navalny',
    url_domains=['facebook.com'],
    key_wordings=key_wordings_n_fb_jail,
    key_urls=None,
    date_range=('2021-01-18', '9999-12-31'),  # Starting from 18th Jan 2021 to far future
    keep_instead_of_remove=True
)

Initial number of matching posts (Author, Domain, Dates): 385
Number of posts to keep: 11
Number of posts removed: 374
Number of posts of this Author, Domain, Dates left after removal: 11


In [343]:
# Remove all the after-jail Instagram posts that duplicate posts from Telegram channel, navalny.com and facebook

key_urls_n_insta_jail = (
'https://www.instagram.com/p/CwDCsClNeiN',
'https://www.instagram.com/p/CvzEGR8vM7L',
'https://www.instagram.com/p/Cvh6kPUtTln',
'https://www.instagram.com/p/Cveztf1rTqW',
'https://www.instagram.com/p/CvW-98JNZGm',
'https://www.instagram.com/p/CvKNlpItqre',
'https://www.instagram.com/p/CvExnC_tgvQ',
'https://www.instagram.com/p/Cu7ArN6M-lI',
'https://www.instagram.com/p/Cu1jzBAtqmB',
'https://www.instagram.com/p/CuhcchAtKIJ',
'https://www.instagram.com/p/CuUWcjZtVBq',
'https://www.instagram.com/p/CuPAsquOXTT',
'https://www.instagram.com/p/CuEU4daNNzX',
'https://www.instagram.com/p/CuABw2oNlyD',
'https://www.instagram.com/p/Ctq8KqqttkI', 
'https://www.instagram.com/p/CtrjfdpNG7k',
'https://www.instagram.com/p/CtjQaUWNsXg',
'https://www.instagram.com/p/CtglkHNukS-',
'https://www.instagram.com/p/CtHTW-DNJIA',
'https://www.instagram.com/p/CtD58yDtogJ',
'https://www.instagram.com/p/Cs-tZI1tZN5',
'https://www.instagram.com/p/Cs3LAqZtp5P',
'https://www.instagram.com/p/CstNU4KNjOd',
'https://www.instagram.com/p/Crxh3TqttFO',
'https://www.instagram.com/p/CrlECRutHj5',
'https://www.instagram.com/p/CrfcLG1tZ-h',
'https://www.instagram.com/p/CrdJ1batPlS',
'https://www.instagram.com/p/CrLcPgTNOXt',
'https://www.instagram.com/p/CrFoDKuNAmS',
'https://www.instagram.com/p/Cq-tKiRtSn8',
'https://www.instagram.com/p/Cq5yV2mNppV',
'https://www.instagram.com/p/CqztSv1tE4-',
'https://www.instagram.com/p/CqPon1RN_v5',
'https://www.instagram.com/p/CqKyQACt-YB',
'https://www.instagram.com/p/CqGKYwZIWGI',
'https://www.instagram.com/p/Cp0JwhcM5m2',
'https://www.instagram.com/p/Cphap0ztxe5',
'https://www.instagram.com/p/Co4fFg9NNQP',
'https://www.instagram.com/p/CooxPTTtGFG',
'https://www.instagram.com/p/CoXD40wtK1Y',
'https://www.instagram.com/p/CoHnG4zN5ex',
'https://www.instagram.com/p/CoFWym5NZQr',
'https://www.instagram.com/p/Cn4FR8DtjSR',
'https://www.instagram.com/p/Cn1S81JN4aC',
'https://www.instagram.com/p/CngkEWutzdT',
'https://www.instagram.com/p/CnUnaFbNF_Y',
'https://www.instagram.com/p/CnMRKWqNwc6',
'https://www.instagram.com/p/Cm1KVGBtkGy',
'https://www.instagram.com/p/CmoBV16oyOx',
'https://www.instagram.com/p/CmlhXyONxAX',
'https://www.instagram.com/p/CmgfzjYNHvP',
'https://www.instagram.com/p/CmeIFSPNkZl',
'https://www.instagram.com/p/CmbapwKt6iI',
'https://www.instagram.com/p/CmYm-C3I4JF',
'https://www.instagram.com/p/CmZQiYoI7SG',
'https://www.instagram.com/p/CmJVtcJNCW3',
'https://www.instagram.com/p/CmD-RVKNErn',
'https://www.instagram.com/p/Cl8mwfRt9PM',
'https://www.instagram.com/p/ClWTgmvNad7', 
'https://www.instagram.com/p/ClTKAPttURa',
'https://www.instagram.com/p/ClOWRiwNEvO',
'https://www.instagram.com/p/ClD2mCPNHHI',
'https://www.instagram.com/p/Ck8S7pPtwU5',
'https://www.instagram.com/p/CkyAMKlp71h',
'https://www.instagram.com/p/CkgBqeftraV',
'https://www.instagram.com/p/CkdM-hCNSB9',
'https://www.instagram.com/p/Cj7xKdMtYnL',
'https://www.instagram.com/p/Cj8A0NBNaNl',
'https://www.instagram.com/p/Cj29rnNvpcg',
'https://www.instagram.com/p/Cjmy2IiJMuW',
'https://www.instagram.com/p/CjkpTxKtCgj',
'https://www.instagram.com/p/CjYMP7ds3qu',
'https://www.instagram.com/p/Ci18k4eNaaX',
'https://www.instagram.com/p/CiuJl-otl-V',
'https://www.instagram.com/p/CiuguvnNSp-',
'https://www.instagram.com/p/CiPz1lUtkm3',
'https://www.instagram.com/p/CiM_zlgtNQd',
'https://www.instagram.com/p/CiKT3J7tMEC',
'https://www.instagram.com/p/Ch9VcP2JkzF',
'https://www.instagram.com/p/Ch4bwvuIM6P',
'https://www.instagram.com/p/ChuEgywIbc8',
'https://www.instagram.com/p/ChpVBZYoxz1',
'https://www.instagram.com/p/Chebj1box0I',
'https://www.instagram.com/p/ChSBd9JtFWn',
'https://www.instagram.com/p/ChHX2hMN84h',
'https://www.instagram.com/p/Cgtnp3Osguv',
'https://www.instagram.com/p/CgY5iCqNqfE',
'https://www.instagram.com/p/CgB9bQUNEol',
'https://www.instagram.com/p/Cf38RkqNQ1R',
'https://www.instagram.com/p/CfqnwqRtxYE',
'https://www.instagram.com/p/CfmF2LTtShC',
'https://www.instagram.com/p/CfdlIaTt2Gl',
'https://www.instagram.com/p/CfI_-BvNK7I',
'https://www.instagram.com/p/CfEcxsjNKaA',
'https://www.instagram.com/p/Ce6QGdyJDUj',
'https://www.instagram.com/p/Ce1EzKUtZvq',
'https://www.instagram.com/p/CeYE9NSND5D', 
'https://www.instagram.com/p/CeOam9gtn5P',
'https://www.instagram.com/p/CeBmGDRMTqd',
'https://www.instagram.com/p/CdIX-d9t7cT',
'https://www.instagram.com/p/Cczzrwct-uv',
'https://www.instagram.com/p/CcumA3VIgV1',
'https://www.instagram.com/p/CciXgkFttWk',
'https://www.instagram.com/p/CcS8tK5ttuQ',
'https://www.instagram.com/p/CcQCN1lPRwz',
'https://www.instagram.com/p/CbkDFtbNROw',
'https://www.instagram.com/p/CbaCdFzNdsE',
'https://www.instagram.com/p/CbIf-xcN4x6',
'https://www.instagram.com/p/CbFoJh6N-HD',
'https://www.instagram.com/p/Ca_xoRMNlf8',
'https://www.instagram.com/p/Ca9MzTetcIT',
'https://www.instagram.com/p/CauBrl3NK10',
'https://www.instagram.com/p/CamISi5N_zQ',
'https://www.instagram.com/p/CaXbyU7gMdD',
'https://www.instagram.com/p/CaR4GuwtOqE',
'https://www.instagram.com/p/CZ9gm7aNBDb', 
'https://www.instagram.com/p/CZ10PVsNo1Y',
'https://www.instagram.com/p/CZwwI56tbV9',
'https://www.instagram.com/p/CZTqCJMNa6q', 
'https://www.instagram.com/p/CZMpwBSN2f5',
'https://www.instagram.com/p/CY9ML9LIDLU',
'https://www.instagram.com/p/CY6XS1IIMpZ',
'https://www.instagram.com/p/CY01DoLoJec',
'https://www.instagram.com/p/CYrc1w7o2vC',
'https://www.instagram.com/p/CYjdiM1IkEe',
'https://www.instagram.com/p/CYJQ7alo7pL',
'https://www.instagram.com/p/CX-wFYWo_aA',
'https://www.instagram.com/p/CX0Vrjso1WI',
'https://www.instagram.com/p/CXywoeQouqw',
'https://www.instagram.com/p/CXtYBCIoSNR',
'https://www.instagram.com/p/CXdRvzloO2l',
'https://www.instagram.com/p/CXLRSx8oPt6', 
'https://www.instagram.com/p/CXLRSx8oPt6',
'https://www.instagram.com/p/CW-wAVmo623',
'https://www.instagram.com/p/CWaUaCGIB5v',
'https://www.instagram.com/p/CWBO653on3-',
'https://www.instagram.com/p/CVnedCdI6e_',
'https://www.instagram.com/p/CVS8n10oVlt',
'https://www.instagram.com/p/CVNHEeZozCb',
'https://www.instagram.com/p/CU4ffm2ow0K',
'https://www.instagram.com/p/CUc2J77IZpE',
'https://www.instagram.com/p/CUKxgF5oq32',
'https://www.instagram.com/p/CUFWtmZIMwP',
'https://www.instagram.com/p/CUCsCdeI0BN',
'https://www.instagram.com/p/CT_npujIxWP',
'https://www.instagram.com/p/CT6lV7HjOuA',
'https://www.instagram.com/p/CT4sfpEIAEc',
'https://www.instagram.com/p/CT1f5YOodBi',
'https://www.instagram.com/p/CT1vNyyoeYl',
'https://www.instagram.com/p/CTwr-MtIPCm',
'https://www.instagram.com/p/CTxLu-eIZI4',
'https://www.instagram.com/p/CTmqyyNoCou',
'https://www.instagram.com/p/CThrPYoocVa',
'https://www.instagram.com/p/CTWo7Aso_L8',
'https://www.instagram.com/p/CSye0lqoUwq',
'https://www.instagram.com/p/CSuAMWCIPTn',
'https://www.instagram.com/p/CSgueZ8ImG8',
'https://www.instagram.com/p/CSeaj9_onAq',
'https://www.instagram.com/p/CR9BZAjHevz',
'https://www.instagram.com/p/CQ0tkqVNJ3x',
'https://www.instagram.com/p/CQd3tK2taXj',
'https://www.instagram.com/p/CP6YLkztEF-',
'https://www.instagram.com/p/CPsUdnaNsb_',
'https://www.instagram.com/p/CPnXkD_NNFE',
'https://www.instagram.com/p/CPSkeFZNTag',
'https://www.instagram.com/p/CNzf5FhntYE',
'https://www.instagram.com/p/CNw2fwSlWUC',
'https://www.instagram.com/p/CNucykqFS_s',
'https://www.instagram.com/p/CNXy3DGFZx1',
'https://www.instagram.com/p/CNFlxVIF7Wg',
'https://www.instagram.com/p/CM_6xtrF3s_',
'https://www.instagram.com/p/CM4cmFEFjOd',
'https://www.instagram.com/p/CMwlbdsnEGn',
'https://www.instagram.com/p/CK4DuqLIGZf',
'https://www.instagram.com/p/CKOmJc8nBOB',
'https://www.instagram.com/p/CKLqe3JDyda',
'https://www.instagram.com/p/Cb-Yad7NCwE')

remove_or_keep_entries(
    cleaned_archive_path='../data/fbk_oleg_archive_clean.csv',
    author='Navalny',
    url_domains=['instagram.com'],
    key_wordings= None,
    key_urls= key_urls_n_insta_jail,
    date_range= None,
    keep_instead_of_remove=False
)

Initial number of matching posts (Author, Domain, Dates): 1848
Number of posts removed: 1
Number of posts of this Author, Domain, Dates left after removal: 1847


In [411]:
# Seems like all the Navalny texts from Telegram and navalny.com, when later imported, don't have line breaks.
# This might affect our future text analysis.
# Since the majority of after-jail Telegram posts duplicate Instagram posts, which a imported with line breaks, 
# we can replace the former with the latter.

# The function below search for Instagram duplicates in our initial archive fbk_oleg_archive.csv
# and replace Telegram posts in the fbk_oleg_archive_clean.csv with those duplicates

import csv

def replace_entries_from_source_to_target(source_archive_path, target_archive_path, author, source_domain, target_domain, date_range):
    # Read the source archive
    with open(source_archive_path, 'r', encoding='utf-8') as file:
        source_archive = list(csv.DictReader(file))
        
    # Read the target archive
    with open(target_archive_path, 'r', encoding='utf-8') as file:
        target_archive = list(csv.DictReader(file))

    # Initialize counter for replaced entries
    replaced_entries = 0

    # Loop through target archive
    for target_row in target_archive:
        if target_row['author'] == author and target_domain in target_row['url'] and date_range[0] <= target_row['date'] <= date_range[1]:
            snippet = target_row['text'][20:40]  # Take the first 30 characters of the text

            # Look for a matching entry in the source archive
            for source_row in source_archive:
                if source_row['author'] == author and source_domain in source_row['url'] and snippet in source_row['text']:
                    print(f"Replacing target entry (date: {target_row['date']}, snippet: {snippet}) with source entry (date: {source_row['date']}, snippet: {source_row['text'][:30]})")
                    
                    # Replace entry in target archive
                    target_row.update(source_row)
                    replaced_entries += 1
                    break  # Exit the inner loop once the replacement is done

    # Write the modified target archive back to the file
    with open(target_archive_path, 'w', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['author', 'text', 'date', 'url', 'source_type'])
        writer.writeheader()
        writer.writerows(target_archive)

    print(f"Replaced {replaced_entries} entries.")

# Example usage
source_archive_path = '../data/fbk_oleg_archive.csv'
target_archive_path = '../data/fbk_oleg_archive_clean.csv'
author = 'Navalny'
source_domain = 'instagram.com'
target_domain = 't.me'
date_range = ['2021-01-18', '9999-12-31']

replace_entries_from_source_to_target(source_archive_path, target_archive_path, author, source_domain, target_domain, date_range)


Replacing target entry (date: 2023-06-05, snippet: наю, что я не один. ) with source entry (date: 2023-06-05, snippet: Спасибо, родные! 

Я знаю, что)
Replacing target entry (date: 2023-03-26, snippet:  этой фотографии, он) with source entry (date: 2023-03-26, snippet: Как ни хороша Юля @yulia_naval)
Replacing target entry (date: 2023-01-26, snippet:  время съемок фильма) with source entry (date: 2023-01-26, snippet: Выполняю обещание.

Во время с)
Replacing target entry (date: 2022-12-26, snippet: очная разгадка в кон) with source entry (date: 2022-12-26, snippet: Отгадайте, что это?

Точная ра)
Replacing target entry (date: 2022-12-09, snippet: го времени, когда ем) with source entry (date: 2022-12-09, snippet: Знаю Илью Яшина @ilya_yashin с)
Replacing target entry (date: 2022-08-30, snippet: еяться, но я снова в) with source entry (date: 2022-08-30, snippet: Вы будете смеяться, но я снова)
Replacing target entry (date: 2022-06-04, snippet:  у меня все отлично.) with source entry (da

In [425]:
# For the rest of Navalny's after-jail Telegram posts (35) and navalny.com posts (21) 
# let's manually replace texts without line breaks to texts with line breaks

import csv

def replace_texts_by_urls(file_path, url_text_pairs):
    # Read the existing archive
    with open(file_path, 'r', encoding='utf-8') as file:
        archive = list(csv.DictReader(file))
    
    # Flag to check if any entries were modified
    modified_count = 0
    
    # Loop over each URL-text pair to find and replace text
    for target_url, new_text in url_text_pairs:
        for row in archive:
            if row['url'] == target_url:
                row['text'] = new_text
                modified_count += 1
                print(f"Replaced text for the entry with URL: {target_url}")
                break

    # Write the modified archive back to the file
    if modified_count > 0:
        with open(file_path, 'w', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=['author', 'text', 'date', 'url', 'source_type'])
            writer.writeheader()
            writer.writerows(archive)
        print(f"Total modified entries: {modified_count}")
    else:
        print("No entries found with the given URLs.")

# Example usage
file_path = '../data/fbk_oleg_archive_clean.csv'  # Replace with your actual file path
url_text_pairs_tg = [('https://t.me/s/navalny/3005', 'Обыски, задержания и аресты — это так Путин стучит ножками, когда видит свой рейтинг. А смотрит он на него часто, ведь скоро выборы. И сейчас очень важно помочь независимым кандидатам в Государственную думу, чтобы расстроить его посильнее.\nЕсть такой кандидат — Олег Степанов. Он был главой московского штаба: провёл кучу расследований против московских жуликов, решил много локальных проблем, организовал важные проекты — Центр сбора подписей, подготовку наблюдателей и другие.\nИ ваша помощь его штабу нужна прямо сейчас.'),
('https://t.me/s/navalny/3072', 'Как потратить 10 тысяч евро?\n\nДа уж, хорошо бы почаще мучиться подобными вопросами.\n\nНо в моем распоряжении не простые 10 тысяч евро, а прилагающиеся к премии имени Бориса Немцова, присужденной мне. Это большая честь, и я чувствую, что как-то странно и невозможно просто взять и потратить деньги на себя и свою семью.\n\nПоэтому я попросил Жанну Немцову поделить эти деньги на равные 4 части по 2,5 тысяч и передать их семьям тех, кому сейчас хуже, чем мне. А главное, использовать это как повод, чтобы рассказать вам три очень короткие истории. О нашей стране.\n\n1. Павел Зеленский работал видеооператором ФБК. Когда в Нижнем Новгороде журналистка Славина совершила самосожжение, он принял это близко к сердцу. У Славиной дети, у Павла тоже - четверо. Он написал два твита о том, что проклинает власти и считает их виновными в гибели Славиной.\n\nЗа эти два твита Павел Зеленский получил 2 года тюрьмы.\n\n2. Андрей Боровиков любит группу Rammstein (я тоже). 7 лет назад он повесил их клип на свою стену ВК. Официальный клип, он размещен миллион раз в миллионе мест.\n\nОднако Андрей был обвинен за это в распространении порнографии и получил 2,5 года тюрьмы.\n\nНа самом деле его «преступление» заключалось в том, что он боролся против мусорного полигона в Шиесе и позже стал координатором нашего штаба в Архангельске.\n\n3. Несколько лет назад в Ростове-на-Дону была эпидемия поджогов. Все в городе знали - бандиты, нанятые властями, жгут частные дома, чтобы освободить место для застройщиков.\n\n5.11.17 Владислав Мордасов и Ян Сидоров вышли на пикет к зданию правительства Ростовской области. У них было два плаката: «Верните землю ростовским погорельцам» и «Правительство в отставку». За это они получили 6,5 лет тюрьмы.\n\nИстории совершенно реальные, и если вы не слышали о них раньше, то вы наверняка ужаснулись: как же так можно?!\n\nА я хочу вам сказать: ужаснитесь еще сильнее. Ведь это не только история о поломанных судьбах хороших невиновных людей, но и истории о вас. О вашем настоящем, которое хуже, чем могло быть.\n\nИ о будущем, которое станет еще хуже. Ведь пока у власти сидит эта банда злобных воров-извращенцев, не будет в России ни экономического роста, ни высоких зарплат, не перспектив, ни справедливости.\n\nЭто вам и любой экономист, и любой историк подтвердит.\n\nДа чего уж там. Зайдите в магазин, посмотрите на цены, а потом подумайте, когда вам последний раз повышали зарплату так, что вы действительно могли купить больше.\n\n19 сентября выборы, и «Умное голосование» - отличный способ отвесить затрещину ворам-извращенцам из «ЕР». Участвуйте.\n\nНу а я буду очень рад, если к моим 2500 евро семьям этих людей вы добавите письмецо им самим.\n\nПавлу Зеленскому и Яну Сидорову можно написать ФСИН-письмо, а Владу Мордасову - обычной почтой. Вот их адреса:\n\nЗеленский Павел Андресович, 1981 г.р., Тульская область, ИК-2 Тула\n\nСидоров Ян Владимирович, 1999 г.р., Ульяновская область, ИК-10 Дмитровград\n\nМордасов Владислав Евгеньевич, 1996 г.р., 346519, Ростовская область, г. Шахты, ул. Открытая, д. 10, ФКУ ИК-9 ГУФСИН России по Ростовской области\n\nКуда писать Андрею Боровикову, пока неизвестно. Его на днях этапировали из СИЗО Кирова, но куда его отправили и где он находится сейчас, не знает никто, даже его семья.'),
('https://t.me/s/navalny/3086', 'БЕСИТ.\n\nНет, правда, прям вот бесит.\n\nИ не думайте, что я продолжу про какую-то тюремную фигню. Здесь бесит почти все - специально так устроено. С этим я борюсь, развивая тюремный дзен - спокойное и равнодушное отношение к идиотским правилам, меняющимся буквально каждый день. Получается с переменным успехом, но я стараюсь.\n\nА вот к чему нельзя научиться относиться спокойно, так это к некоторым вашим комментам. Я прошу распечатывать их выборочно, чтобы понимать, что вы думаете про Умное голосование. Большинство нормальные, конечно, но вот сегмент нытиков реально бесит и сводит с ума.\n\n- Ой, зачем я пойду голосовать, от меня ничего не зависит.\n- Да там все кандидаты одинаковые.\n- Нет смысла менять одних на других, я лучше чем-нибудь другим займусь.\n\nТакая беспомощность и покорность, что можно подумать, это вы в тюрьме, а не я. И сделать ничего не можете. И подписчиков мало, чтобы агитировать. И занятий так много, что времени проголосовать нет.\n\nСлушайте, а я вот на обед хожу строем и интернетом пользовался вечность назад. Но совсем не считаю, что ничего не могу изменить. И не оплакиваю свои утерянные возможности обращаться к миллионам через ютюб-канал, вести прямые эфиры и тд.\n\nЗнаете почему? Потому что я сравниваю себя не с Алексеем Навальным на свободе. У которого возможности, офис и штабы. А с Алексеем Навальным, который ничего не делает. Бездельничает, горюя о своей мнимой беспомощности. Я хочу быть лучше него, и это несложно. Надо просто сделать элементарные вещи, внести хоть какой-то вклад.\n\nИ вы тоже так можете. Вы, получивший имя кандидата через «Умное голосование» и сходивший на выборы, на 1000% более влиятельны и могущественны, чем та ваша версия, что ноет и бездельничает. Приведете в УмГ родственника - станете круче на 3000%.\n\nПовесите у себя призыв к Умному голосованию - вы вообще политический терминатор по сравнению с собой бездельником.\n\nПослушайте. С 2003 года, то есть ПОЧТИ 20 ЛЕТ на выборах в ГД России по округам не побеждали независимые кандидаты.\n\nЧтобы сломать эту ситуацию, мы придумали Умное голосование и успешно испытали его.\n\nНеужели вам самим не интересно попробовать? И заодно стать лучше версией себя?'),
('https://t.me/s/navalny/3116', 'В тюрьме все время ждешь.\n\nЛюбой сидевший вам расскажет, что срок раскладывается на большие и маленькие периоды ожидания. В бесконечном дне сурка есть какие-то регулярные события (хорошо, если приятные), вот и живешь ожиданием от одного к другому.\n\nПонятно, что еще и общий срок считают. Большинство зэков может моментально ответить: “Мне осталось 583 дня”. Но у меня перспективы окончания срока, скажем мягко, неясны, поэтому и считать смысла не имеет.\n\nТечение времени заметно по интервалам. Самый короткий период: от яйца до яйца. По понедельника и пятницам на завтрак дают вареное яйцо. Это не только важное (и приятное) кулинарное событие, но и календарное тоже.\n\nПолучил на завтрак яйцо - значит, рабочая неделя заканчивается. Снова получил яйцо - рабочая неделя началась.\n\nПериод побольше - ларек. Туда водят раз в две недели. Здесь уже не просто ждешь, а прям планируешь и гадаешь - получится ли купить молока или выйдет облом, его разберут к твоему приходу. Можно даже позволить себе смелые фантазии и допустить, что останется творог или капуста. Но с такими мыслями надо поосторожнее, разочарование может быть горьким.\n\nПередача - это уже большой и четкий период времени. Шесть раз в год. Один в два месяца. Каждый знает дату, когда у него “открывается передача”. Получил, скажем, 15 сентября, значит, 15 ноября получаешь право на еще 20 кг еды и необходимых вещей от родственников.\n\nНу и самое важное - длительное свидание. Четыре раза в год для зэка на общем режиме. Раз в три месяца к тебе могут приехать родственники, и общаешься с ними не через стекло и телефон, а - какой восторг - живьем.\n\nЗа три дня до свидания я поймал себя на том, что сижу на кухне (a.k.a. “комната приема пищи”) в бараке с кружкой чая, смотрю на пустую табуретку рядом с собой и представляю, что на ней сидит Юля, и я веду с ней мысленный разговор. Типа, скажу ей это, а она мне ответит то, и тогда я пошучу это, а она засмеется.\n\nЭто такой легкий и приятный формат съезжания крыши. Чем ближе к дате свидания, тем больше впадаешь в ажитацию. Пишешь на листочке, что хочешь обсудить и какие вопросы задать - чтоб не забыть важного.\n\nВ день свидания время просто останавливается и тянется невыносимо медленно - высшие силы дразнят тебя и посмеиваются. А потом - ура, ура, наконец-то “осужденный Навальный, собирайтесь на свидание”.\n\nВот сейчас пишу это, сидя на настоящей кухне (с плитой!) комнат свиданий. Юля еще спит в нашей комнате, а я пошел выпить кофе и пожарить нам яичницу с грудинкой. Кайф же? Кайф. Через пять часов снова “осужденный Навальный, собирайтесь”, и я начну новый отсчет.'),
('https://t.me/s/navalny/3174', 'Восемь дней назад нашу страну стали считать абсолютным злом. Восемь дней на украинские города падают бомбы и ракеты. Гибнут тысячи людей, и никто так и не понял, ради чего.\n\nИ всё это из-за одного человека. Путин позорит российский флаг, русский язык и само слово Россия. Безумный диктатор за считанные дни спустил на ветер все достижения российских ученых, спортсменов и артистов. На много десятилетий нас обрекают на чувство вины. Из-за Путина сейчас для многих Россия значит Война. Но это не так.\n\nНе Россия напала на Украину, а Путин. Он много лет запугивал, арестовывал, убивал россиян. Он крал выборы и миллиарды. Это Путин захотел бомбить мирные города, убивать детей и разрушать жизни. Россиян не спросили, хотят ли они войны. В России, которую мы любим и которой гордимся, миллионы людей против этого безумия. И сейчас их помощь нужна как никогда. Не только Украине, и даже не России, помощь нужна всему миру. Помощь в борьбе с Путиным и его войной.\n\nВам может быть страшно, но поддаться этому страху — значит встать на сторону фашистов и убийц. Путин уже отнял у России экономику, отношения с миром и надежду на будущее. Это намного страшнее, чем штраф за митинг, который вам, к тому же, помогут оплатить. А в бомбоубежищах и на линии фронта куда ужаснее, чем на сутках в спецприемнике. Не поддавайтесь страху, выходите на улицы, пусть весь мир увидит, что Путин — это не Россия. Только так мы сможем вернуть нашей Родине честь. Только так каждый из нас сможет сохранить свое собственное достоинство. Сможет сказать — в эти самые страшные дни я оставался человеком и не молчал.\n\nЗнаете, часто перед митингами спрашивают: «ну и что этот митинг изменит?». Этот митинг изменит нас. Этот митинг изменит то, что вы скажете своим детям и внукам, когда они спросят вас про это страшное время. Мы должны выйти ради самих себя.\n\nА даже если вы уже уехали из России, потому что не могли больше мириться с наступлением военной диктатуры. Выходите в тех странах, где вы живёте. Покажите всему миру, что россияне не хотят войны. Выходите на площади Берлина, Нью-Йорка, Амстердама или Мельбурна, где бы вы ни были. Сейчас мы все в ответе за будущее России. За то, какой Россия будет в глазах всего мира.\n\nКаждый будний день в 19:00, но самое главное — в это воскресенье, шестого марта в 14:00, на центральных площадях всех городов России и мира. Москва — Манежная площадь, Петербург — Гостиный двор. За достоинство. Против Путина. За Россию. Против войны.'),
('https://t.me/s/navalny/3182', 'Вопрос о том, поддерживают ли россияне преступную войну, которую Путин на ровном месте развязал против Украины, имеет огромное политическое значение. Он во многом будет определять место России в истории XXI века. Одно дело — если Путин убивал мирных граждан Украины и разрушал инфраструктуру при полной поддержке и одобрении российских граждан — в этом нас пытаются убедить и провластные социологические центры, и совсем другое — если его кровавая авантюра не основана на поддержке общества. Ответ на этот вопрос важен и с практической точки зрения: стоит ли пытаться организовать протест против войны? Можно ли надеяться, что давление на Путина внутри России сможет заставить его свернуть агрессию?\n\nНо как это измерить? Стандартный инструмент социологической службы ФБК — классические телефонные опросы по репрезентативной выборке. Это достаточно надежный инструмент исследования общественного мнения (нас он никогда не подводил, скажем, во время избирательных кампаний), но он требует времени. Опрос длится неделю — в военное время за неделю меняется слишком многое. И, кроме того, готовность людей говорить по телефону, тем более говорить искренне, постоянно снижается в России, а уж во время войны, когда власть пугает уголовными сроками за «неправильное» мнение — и подавно.\n\nПоэтому мы решились на новый для нас подход: мы провели серию из четырех очень коротких и быстрых интернет-опросов. В каждом из этих опросов приняло участие по 700 интернет-пользователей из Москвы; выборка была квотирована по полу и возрасту. Полученные данные, как нам кажется, представляют огромный интерес. Посмотрите сами:'),
('https://t.me/s/navalny/3202', '9 лет строгого режима.\n\nМой космический полет немного затягивается - корабль попал во временную петлю.\n\nПришло в голову, что я в этой саге как тот парень из “Интерстеллара”.\n\nПомните, главные герои заходят на станцию, поднявшись с планеты, время на которой течет в несколько раз быстрее. И их встречает чувак с бородой в халате: “Я ждал вас 23 года 4 месяца 8 дней”.\n\nПравда, мысль эту я сразу прогнал. Во-первых, у меня нет ни бороды, ни халата. А во-вторых: ерунда это все. Цифры значения не имеют. Это табличка над нарами, вот и все. И просто “ждать” ни я, ни мои товарищи не будем.\n\nКак я сказал в “последнем слове”, мы не просто продолжаем работу Фонда борьбы с коррупцией, мы выведем его на новый уровень. Фонд станет глобальной международной организацией. И вы очень нужны нам в ней. Заходите: https://acf.international/\n\nДенежная часть премии имени Сахарова, врученная мне Европейским парламентом, станет первым взносом в этот фонд. Граждане ЕС в лице своих представителей дали мне эту премию за борьбу с коррупцией. Я благодарен и направляю их деньги на продолжение этой борьбы.\n\nКстати, обратите внимание, что мое “последнее слово” глушили, прерывая трансляцию.\n\nНу конечно, слова имеют силу. Путин боится правды, я всегда это говорил. Борьба с цензурой, донесение правды до жителей России остались нашим приоритетом.\n\nКремль громит медиа, а мы в ответ создаем новые. Я вам 5 марта анонсировал “Популярную политику” - сейчас у нее уже почти миллион подписчиков. Присоединяйтесь - мы вместе сделаем лучшее СМИ России: https://www.youtube.com/channel/UC7Elc-kLydl-NAV4g204pDQ\n\nЯ очень благодарен всем за поддержку. И, ребята, хочу сказать: лучшая поддержка и мне, и другим политзэкам - это не сочувствие и теплые слова, а действия. Любая активность против лживого и воровского путинского режима. Любое противодействие этим военным преступникам.\n\nВ 2013, услышав свой первый приговор, я это написал и сейчас повторю: не бездельничайте. Эта жаба, сидящая на нефтяной трубе, сама себя не скинет.\n\nВсех обнимаю и люблю! ❤️'),
('https://t.me/s/navalny/3203', 'Оксана Баулина работала на высокой должности в “глянцевой журналистике”, но в 2013-м пришла к нам в штаб, сильно потеряв в зарплате. “Не могу заниматься модой, когда власть в моей стране захватили воры и бандиты. Хочу бороться”.\n\nИ она по-настоящему боролась. Много лет работала в ФБК, потом была волонтером в каждом проекте. Она помогла мне запустить мои первые стримы в ютюбе.\n\nПомню, как мы смеялись с ней, когда она купила мне в магазине театрального реквизита пиратскую повязку для того, чтобы я вышел в ней в эфир - накануне мне плеснули в лицо токсичной жидкостью и врачи сказали, что студийный свет вреден поврежденному глазу.\n\nПоследнее время работала в оставшихся независимых СМИ России. Оксана погибла, попав под обстрел в Киеве. Воры и бандиты из Кремля, с которыми она боролась, убили ее, как убили многих в этой преступной войне.\n\nМы - русские, украинцы, весь мир, обязаны сделать все, чтобы победить Путина - чудовище, разбрасывающее смерть и хаос. Убивающее десятки тысяч и разрушающее города в Европе ради личной власти и личного обогащения.\n\nМои соболезнования родным и близким Оксаны Баулиной.'),
('https://t.me/s/navalny/3213', 'Я приветствую решение Канады внести наконец-то в санкционные списки главу Центрального банка России Эльвиру Набиуллину. Надеюсь, остальные страны последуют примеру.\n\nПочему это важно? Набиуллина - пример одного из самых отвратительных видов путинских слуг. Она работает на Кремль, работает на уничтожение и порабощение моей страны. Но при этом страшно боится санкций и поэтому через своих знакомых экономистов организует утечки о том, как она рада была бы уйти, но ее “не отпускают”.\n\nО том, как она безмерно страдает в своем огромное кабинете, дорогом лимузине, получая огромный доход и помогая тем, кто убивает ни в чем не повинных мирных людей.\n\nНикакого сочувствия. Если эта лицемерка помогает превратить Россию в Северную Корею, то пусть и сама живет в этой Северной Корее, а не надеется уехать жить в Европу, когда здесь все окончательно рухнет.\n\nЕсли не хочешь работать на Путина, то можешь сделать хотя бы как трусливое ничтожество Чубайс - с госслужбы ушел, из страны уехал, но ни слова против войны не сказал. Он не против войны, он деньги спасает. Но хоть так.\n\nА Набиуллина и такие, как она, надеются “пройти между струйками”. Помогать Путину убивать людей, но еще и претендовать на то, что они остались приличными людьми. Ведь хоть и помогали, но медленно и печально.\n\nМы эту тошнотворную ложь не покупаем. С пистолетом у виска никто не стоит.\n\nНе хочешь быть пособницей убийцы - не будь. А если остаешься, то видишь в этом выгоды, перспективы и разделяешь такой взгляд на мир.\n\nРад, что Канада и Джастин Трюдо отвергают такое лицемерие.'),
('https://t.me/s/navalny/3222', 'Почему я пишу о санкциях, введенных 🇬🇧 против Кабаевой и “кошельков” Путина только сегодня? Да потому что об этом не говорят по телевизору!\n\nИ это отдельно указывает на то, как верно действует сейчас Британия, Борис Джонсон и Лиз Трасс, проявляя замечательное лидерство по вопросу войны Путина с Украиной в целом, и по санкциям в частности.\n\nИндивидуальные санкции, заморозка и конфискация активов, запрет виз, аресты яхт и счетов чиновников очень популярны среди граждан РФ. Можно сказать, что это тот вопрос, где даже самый дремучий российский избиратель солидарен с Западом и радикальнее его.\n\nИменно поэтому пропагандисты, 24/7 проклиная Запад в целом и каждого политика там в частности, хранят полное молчание о санкциях против чиновников и олигархов.\n\nИ уж тем более о Кабаевой, которую 🇬🇧 прямо, честно и без лицемерия называют матерью детей Путина и владелицей части его огромного состояния.\n\nВедь единственное, что скажет на это зритель: так и надо, этих не жалко, все правильно делают. А потом задумается о странности устройства России, где любовница президента, ее бабушка и мама (она еще не под санкциями, но, надеемся, что это техническая ошибка) оказались миллиардерами и хранят свои деньги почему-то на территории опаснейшего НАТО.\n\nТак что действия Британии можно только приветствовать, а США и Евросоюз я призываю не только как можно скорее догнать 🇬🇧, но и перегнать их, отбросить полумеры и принять санкционный список разжигателей войны, подготовленный ФБК https://acf.international/ru/bribetakers-list\n\nЭто будет важный и значимый удар по путинской “партии войны”, который поддержит большинство граждан России.'),
('https://t.me/s/navalny/3225', 'Путин решил устроить концерты по всей стране в поддержку войны. Назвал их марафон “Zа Россию”. Денег выделили столько, что по отчетам видно: гонорары артистов вдвое больше обычного.\n\nИтог: участники концертов жалуются, что им заплатили в несколько раз меньше, чем на бумаге. Если ты путинский чиновник, то ты не можешь не украсть. Это в твоей ДНК.\n\nОднако точку в это истории должны поставить США, Евросоюз, Британия, Канада, Австралия, Новая Зеландия.\n\nАлчные лицемерные мерзавцы, желавшие подзаработать на рекламе войны и смерти, должны получить награду от запада: санкции и пожизненный запрет на въезд в цивилизованные страны.\n\nСигнал должен быть прямой и ясный: вам не удастся сидеть на двух стульях. У вас не получится сегодня петь в Новосибирске за войну, а завтра зарабатывать, развлекая гостей на дне рождения олигарха на Французской Ривьере или в Аспене.'),
('https://t.me/s/navalny/3227', 'Возможно, истинная миссия Владимира Путина в том, чтобы преподавать уроки. Всем — от мировых лидеров и мировых же экспертов до обычных людей. В 2022-м он был особенно хорош в этом.\n\nОн вновь напомнил нам о том, что путь, начинающийся с «немного подделать выборы» всегда заканчивается диктатурой. О том, что диктатура всегда ведет к войне. Ох, как не стоило об этом забывать, верно?\n\nМировые лидеры годами лицемерно говорили о «прагматичной позиции» и выгодах в международной торговле. Цена войны: военная и экономическая помощь Украине, беженцы, санкции, — все это будет стоить в сотни раз дороже, чем приносили нефтяные и газовые контракты, на подписании которых было так принято пить шампанское.\n\nПутин напомнил нам о «законе утки». Помните? Ходит, как утка, выглядит и крякает, как утка, — значит, это утка.\n\nТак и здесь: уничтожил СМИ, организует политические убийства и пребывает в имперских иллюзиях — значит, это безумец, способный устроить кровавую бойню в центре Европы в ХХI веке. И не надо с ним обниматься на международных форумах.\n\nПрямо сейчас Путине ведет урок на тему: как обнулить экономический рост своей страны за 20 лет.\n\nОднако ответ на главный вопрос: как же остановить злобного сумасшедшего с армией, ядерной бомбой и членством в Совбезе ООН, нам всем предстоит найти самим'),
('https://t.me/s/navalny/3254', 'Я беспокоюсь за Илью Яшина.\n\nСейчас он в камере под арестом на 15 суток. Но выглядит этот арест как классическая ситуация, когда сажают на небольшой срок, чтоб за это время сделать документы и посадить надолго.\n\nВласти — и московские, и кремлевские, и, не сомневаюсь, сам Путин, ненавидят Яшина и мечтают от него избавиться. Для этого у них есть веская причина: Илья Яшин своим существованием и деятельностью опровергает их постоянную и однообразную ложь об оппозиции.\n\nМол, конструктива нет, программы нет, на выборы не идут. Только по митингам бегают. Если придут к власти — будет хуже.\n\nА Яшин, хоть и по митингам бегает, но «Единую Россию» разгромил, сформировав большинство в московском районе (это как приличного размера город в России). Программа четкая, дела ясные. Привел оппозицию к власти и стал работать на народ, целыми днями занимался городскими проблемами, встречаясь с людьми. Служебную машину отдал под бесплатное такси для инвалидов. То есть реально «при Яшине» в районе жить стало лучше.\n\nПри этом ни в какие «малые дела» его муниципальное собрание не ушло. Они избранники народа и от его имени делали политические заявления.\n\nПротив войны, в частности (за это заявление совершенно беззаконно арестованы глава Красносельского муниципального округа Елена Котеночкина и муниципальный депутат Алексей Горинов).\n\nНу, то есть люди смотрели на Яшина и его коллег и говорили: «Такая власть лучше, чем единороссовская». В глазах Кремля это преступление.\n\nПоэтому Илье Яшину лучи поддержки, пожелания скорейшей свободы. Всех призываю следить за его судьбой и поддерживать его.\n\nА еще посмотрите его отличное интервью Юрию Дудю. Я, правда, читал, не смотрел. Но смотреть же еще лучше!\n\nТам Яшин много меня хвалил, но рекомендую это интервью я не поэтому! Оно правда хорошее, а места с хвалением меня можно перематывать 😉'),
('https://t.me/s/navalny/3265', 'У меня для вас есть «Сирена». Она уже хороша, а будет еще лучше.\n\nЯ изучаю довольно много соцопросов и вижу, что все они показывают: телеграм-каналы стали одним из главных источников новостей, причем совсем не только для молодежи. Так что вот вам еще один канал.\n\n«Сирена» честно, без вранья и цензуры рассказывает важные для российской аудитории новости – о происходящем в нашей стране, в Украине и в мире. Самые сложные темы корреспонденты разбирают в карточках, инфографиках и больших материалах – будет нетрудно разобраться.\n\nДайте нам шанс – подпишитесь @news_sirena. Мне кажется, вам должно понравиться.'),
('https://t.me/s/navalny/3272', 'Война Путина с Украиной идет уже полгода. С первого дня лидеры Запада твердо заявили, что путинских олигархов и коррупционеров ждут неминуемые санкции и на этот раз они точно не отвертятся. Отвертелись. Это пост ярости о том, что почти ничего не сделано.\n\nВ сериале “Симпсоны” есть персонаж - коррумпированный мэр Квимби. Его фирменная фраза: “Мне кажется, или я слышу звук открывающегося кейса?”\n\nЯ вспоминаю его каждый раз, когда думаю, как путинские олигархи и их подручные - лоббисты, юристы и банкиры - успешно блокируют санкции даже во время войны.\n\nСмотрите, какая ситуация. С одной стороны, на уровне Европарламента, Конгресса и Сената США, национальных парламентов есть полная поддержка массовых санкций против коррупционеров и разжигателей войны.\n\nА с другой, как только дело доходит до конкретной работы, в недрах исполнительной власти все рассыпается в прах. Просто ничего не происходит. Громкие заявления лидеров есть, а решения где-то застревают. Давайте я приведу вам несколько примеров.'),
('https://t.me/s/navalny/3273', 'Вышел из ШИЗО (или как тут говорят, “поднялся с кичи”) - дают уведомление: олигарх Дерипаска подал на вас в суд.\n\nДействительно, этот путинский жулик судится со мной, Марией Певчих, Георгием Албуровым, а заодно с изданием The Insider и другими СМИ, требуя удалить фотографию, на которой он запечатлен c Лавровым в Японии.\n\nЕще Дерипаска требует удалить и опровергнуть наше расследование о том, что он дает взятки главе МИД РФ Лаврову и содержит его вторую семью: https://www.youtube.com/watch?v=xNa5XknuXkQ. А также уничтожить любые упоминания о своей связи с Полом Манафортом.\n\nСудить нас будут в Зеленоградском суде. В способностях путинского олигарха убедить путинский суд я не сомневаюсь.\n\nИ еще раз призываю США, ЕС и Великобританию вводить против этих воров и пособников войны действительно эффективные санкции, не позволяющие уходить от них элементарными уловками. А также внимательно следить за попытками этой публики вести пиаровские и юридические кампании на Западе, отмывая свой имидж и подкупая политиков и журналистов.'),
('https://t.me/s/navalny/3282', 'Тюремное радио объявило о смерти Михаила Горбачева. Именно при нем в СССР были освобождены последние политзаключенные. И то, что сегодня такие, как я, узнают о его смерти через камерные “радиоточки”, прекрасно характеризует начатый этим выдающимся человеком кульбит, совершенный моей страной.\n\nМое отношение к Горбачеву эволюционировало от дикого раздражения - он стоял на дороге обожаемых мною “радикальных демократов”, до печального уважения. Когда выяснилось, что “радикальные демократы” оказались преимущественно сборищем воров и лицемеров, Горбачев остался одним из очень немногих, кто не использовал власть и возможности для личной выгоды и обогащения. Он ушел от власти мирно и добровольно, уважая волю избирателей. Одно это - великий подвиг по стандартам бывшего СССР.\n\nУверен, что его жизнь и история, имевшие ключевое значение для происходившего в конце ХХ века, будут оценены потомками куда более благожелательно, чем современниками.\n\nМои глубокие соболезнования родным и близким Михаила Сергеевича.'),
('https://t.me/s/navalny/3292', 'На сегодняшней встрече с адвокатом мы уже не возмущались, глядя на новое изобретение кремлевско-тюремных властей, а просто смеялись. Минут 5 не могли остановиться.\n\nТакого точно никто никогда не видел. Если раньше адвокатам запретили передавать какие-либо документы, отгородив их от меня сплошной стеной из металлической решетки и пластика, то сейчас этот пластик сделали непрозрачным. Заклеили пленкой, и теперь адвокаты не могут даже ПОКАЗАТЬ мне какие-то документы.\n\nИ теперь свидание с адвокатом максимально похоже на исповедь в католической церкви. Я сижу на стуле в комнатке метр на метр. Передо мной пластиковая стена, за которой смутно виден силуэт человека. О том, что это адвокат, можно догадаться только по голосу. И теперь мне постоянно хочется сказать: «Простите, святой отец. Я согрешил».'),
('https://t.me/s/navalny/3289', 'Мир Дому нашему Общему. Процветания Ходу Арестантскому.\n\nХорошие новости, зэки и их родственники! 20 КГ НА ЧЕЛОВЕКА. Важность моей новости будет мало понятна вольным людям, но для тех, кто за решеткой, она ох как важна.\n\nИтак, недавно вступили в силу “Правила внутреннего распорядка”. В них много всяких глупостей, направленных на ограничение прав осужденных и один пункт вызывал особое возмущение. Это касается длительного свидания с родственниками, самого дорогого и желанного события для каждого зэка. Мало того, что установлено ограничение на количество посещающих, так еще включен иезуитский пункт о том, сколько килограммов вещей и продуктов родственники могут пронести с собой на свидание. Это принципиально важно. Раньше ограничений не было, а сейчас пункт прописан так, что администрации колоний по всей стране начали трактовать его как “20 кг на всех”. И мне именно так объявили - сначала начальник отряда, а затем и.о. начальника колонии - 20 кг на всех.\n\nИ это безумие. “Стандартный” комплект посетителей зэка выглядит так: мать, жена, ребенок. Или бабушка, мать, брат. Эти люди должны привезти с собой еды-питья, одежды и гигиенических принадлежностей на 3 суток. Почти всегда еще и постельное белье и полотенца с собой приносят. Плюс “вольную одежду” для зэков. А представьте себе, если ребенок маленький, то еще и коляска нужна. Вот и получается в итоге, что если общий лимит 20 кг, то трем взрослым людям и ребенку банально не хватит еды на 3-е суток. Ну или придется брать сухари и “бомж-пакеты”.\n\nЕстественно, новость про этот 20-кг лимит всех взбесила. А я не просто взбесился, а сел внимательно читать ПВР. И в моем прочтении выходило, что 20 кг - не на всех пришедших на свидание, а на каждого из них. То есть пришло трое - они могут принести 60 кг. Но на мою трактовку администрация сказала: “Ничего не знаем, 20 кг и все”. Я точно знаю, что так и стало применяться по всей стране. А где-то по старой русской традиции зэкам, близким к администрации, говорили: “20 кг на каждого”, а не близким: “20 кг на всех”.\n\nВ добрые намерения ФСИН и Минюста я не верю, но тем не менее посчитал, что даже для них это будет слишком. Если бы хотели установить такое драконовское ограничение, то написали бы это однозначно. Поэтому написал запрос с просьбой разъяснить ситуацию.\n\nИ вот получил ответ: однозначно каждый пришедший на свидание может принести с собой 20 кг. Прилагаю этот ответ целиком, как и сам запрос.\n\nАдминистрации колонии у нас грозные, а родственники всегда пугливые - боятся, что их просто не пустят на свидание, если они будут качать права. Поэтому, если есть риск конфликтной ситуации относительно количества вещей, то можно просто распечатать этот ответ и взять его с собой.\n\nНа этом ограничусь, пожелав вам Здоровья, Братства и Благополучия. Ходу Арестантскому быть, свободы не миновать.'),
('https://t.me/s/navalny/3302', 'Образ будущего для России — это не «сильная власть» и «твердая рука», а согласие, договор и учет интересов всего общества. России нужна парламентская республика, и только она остановит бесконечный цикл самовозрождающегося имперского авторитаризма.\n\nБезусловно, и парламентская республика не дает стопроцентных гарантий. И само понятие «парламентской республики» слишком обширно.\n\nОднако в этом лекарстве я предлагаю видеть главное: радикальное сокращение власти в руках одного человека, формирование правительства парламентским большинством, независимую судебную систему, значительное увеличение полномочий местных властей. Этого в России не было никогда за всю историю, и мы отчаянно в этом нуждаемся'),
('https://t.me/s/navalny/3307', 'Куда нас втянули? У нас сидят какие-то люди, которые говорят таким образом: „Мы начнем агрессивную войну, захотим оттяпать кусок территории соседнего государства. Но если мы ее будем проигрывать, мы еще устроим ядерный Армагеддон и просто уничтожим все вокруг ради нашего права воровать и делать то, что мы хотим”. Они знают, что есть люди, которые будут против этого выступать, и они этих людей пытаются стереть в пыли и в порошок. У них это не получится'),
('https://t.me/s/navalny/3312', 'Отсидел в ШИЗО 14 суток. Отдохнул 6 дней в отряде СУОН. Ну и всё, хватит. А то расслабиться можно. Снова дают 11 суток ШИЗО, т.к «плохо убирал прогулочный двор и оскорбил оперуполномоченного лейтенанта Неймовича, обратившись к нему «лейтенант Неймович», а не по имени-отчеству».'),
('https://t.me/s/navalny/3325', 'Один день Алексея Анатольевича.\n\nРапорт: «Осужденный Навальный допустил нарушение порядка отбывания наказания. Находясь в помещении умывальной комнаты отряда №1 (СУОН) безадресно использовал в своей речи нецензурную брань (произнес слово «пиздец»)».\n\nОбъяснение осужденного Навального:\n«При посещении туалетной комнаты и санузла отр.№1 СУОН я действительно произнес слово «пиздец», так как никаким другим термином, кроме «пиздец», нельзя описать санитарное и гигиеническое состояние туалетной комнаты и санузла отр.№1 СУОН»\n\nРешение комиссии: «Наложить на осужденного Навального взыскание в виде выговора».'),
('https://t.me/s/navalny/3327', 'Очень беспокоюсь за Алексея Горинова. Этот честный человек, коллега Ильи Яшина по муниципальному совету Красносельского округа, после начала войны с Украиной предложил отменить проведение местных праздников, за что получил 7 лет колонии по статье о «фейках».\n\nСейчас он находится в ИК-2 города Покров, где в свое время сидел я. Это «красная» зона и территория беззакония. Я читаю о том, что Горинов, немолодой уже человек, заболел, оказался в местной санчасти и жалуется на то, что таким образом он долго не протянет.\n\nЯ по своему опыту знаю, что там действительно не лечат. В медсанчасти работает одна несчастная девушка-фельдшер с полным отсутствием оборудования и лекарств.\n\nНапомню, что я сам добился получения элементарного лечения и обезболивающих только после 24 дней голодовки и международной кампании поддержки, поэтому я точно знаю, что любое серьезное заболевание в российской тюрьме и тем более в беззаконных «красных» зонах Владимирской области равносильно смерти.\n\nПризываю всех относиться к ситуации с Гориновым серьезно и оказывать ему посильную поддержку — информационную в первую очередь. Несмотря на всё происходящее, публичное массовое недовольство даже в соцсетях всё ещё работает.\n\nТот, о ком не пишут, тихо умрёт на своей койке, а тому, кого не забывают, может, всё-таки принесут лекарства. Солидарность — наше всё.'),
('https://t.me/s/navalny/3358', 'Дорогие члены ОНК Алексей Мельников и Георгий Волков, ну, спасибо, пацаны, удружили. Очень, блин, помогли.\n\nВы приехали ко мне и рассказали, как у меня все классно, здорово и законно. Но это ладно. Я понимаю новые условия работы ОНК. Но ведь вы в конце своего отчета добавили фразу: «Только с освещением там проблема». Поэтому по результатам вашего визита не изменилось абсолютно ничего (и даже ваши лекарства мне так и не передали), за исключением того, что в мою камеру ШИЗО 2х3 метра зафигачили огромную лампу, в которой сияют аж три большие лампы дневного освещения. Поэтому естественного освещения у меня как не было, так и нет, зато искусственного столько, что позавидует любая операционная. И сидеть при таком свете по 16 часов в сутки - это же просто одуреть можно, даже читать тяжело и хочется прикрыть глаза рукой.\n\nЯ отчаянно требую от администрации убрать хотя бы одну из трех ламп, но они ни в какую. Видимо, это их месть за то, что вы их заставили переделать освещение. То есть заставили вы, а слепым через три месяца стану я.\n\nНе очень-то справедливо.\n\nПриезжайте, блин, снова и переделывайте все как было.'),
('https://t.me/s/navalny/3359', 'Может, они правда какой-то эксперимент проводят? Вы, если следите за моими тюремными приключениями, не поверите. Они сделали это в пятый раз!\n\nПомните моего тракториста, которого тюремщики закидывают ко мне в камеру и выпускают только для того, чтобы продержать сутки в стационаре, забитом больными гриппом и ковидом? Не, ну можно раз так сделать, ну два, ну три, если вы уж совсем демонстративно беспределите. Но, понимаете, они в пятый раз взяли человека, сказали: «Он такой больной, такой больной» и госпитализировали его, а через сутки сказали: «Ой, совсем здоровый» и выписали его обратно ко мне в камеру.\n\nНюанс еще в том, что тракторист, получается, просидел со мной в ШИЗО безвылазно два месяца. Все это время ему запрещено пользоваться тюремным ларьком и покупать в нем элементарные средства гигиены, а у него ни зубной щетки, ни пасты, ни мыла нет.\n\nНаверное, во ФСИНе сидит какой-то биолог-любитель. Знаете, как раньше были юные натуралисты: «А полетит ли этот жук, если я оторву ему лапки?» Так и здесь биолог-любитель из ФСИНа проверяет, сколько раз можно заразить человека гриппом, если переносчиком инфекции будет деревенский алкотракторист.\n\nА демонстративность и наглость, конечно, впечатляют, да.'),
('https://t.me/s/navalny/3400', 'Тюремщики мои как дрессированные тюлени. Их научили хлопать ластами и угукать, вот они хлопают и угукают. Ничего другого придумать не могут.\n\nПосадили меня снова в ШИЗО. На следующий день под каким-то глупым предлогом вывели из камеры на 10 минут, возвращаюсь — и кто, вы думаете, там меня ждет, глядя на меня голодными глазами, полными ужаса? Правильно, мой старый друг-тракторист.\n\nПонимаете, в очередной раз случилось удивительное совпадение. Меня сажают в ШИЗО, и он сразу совершает удачный проступок. Два месяца он курил в одном и том же месте, и никто не обращал внимания. Но как только потребовалось невозможно душную камеру сделать еще более душной и невыносимой, ему за это курение тоже дали 15 суток.\n\nОсобенная прелесть в том , что последние два месяца ему не давали даже элементарных гигиенических принадлежностей — ни зубной пасты, ни туалетной бумаги. Ну и вы уже сами поняли — перевели его ко мне, конечно же, помариновав в санчасти. Используют человека как ватку с бактериями.'),
('https://t.me/s/navalny/3416', 'Какой-то пошел странный и, скажу прямо, мошеннический нарратив о том, что мои тексты — не мои тексты, а за меня, мол, пишет кто-то другой.\n\nМошенничество это потому, что я всегда писал много, делал это годами и никогда не было ничего такого, что давало бы основания заподозрить, что я — это не я. И что мы будем заниматься такими глупостями, как размещение текстов одних людей от лица других.\n\nМеня действительно очень сильно ограничили в доступе к бумаге и ручке. Написать письмо жене и детям теперь может занять пять дней. Я не в состоянии прочитать многие из пришедших писем, уж не говоря о том, что значительное количество входящей и исходящей почты пропадает в неизвестном направлении, а тюремная цензура, как мы узнали на одном из заседаний судов, периодически сжигает пришедшие письма на «большом металлическом блюде».\n\nТем не менее мне же все-таки не романы писать, я работаю в жанре куда более лаконичном. Если очень надо что-то сказать — говорю.\n\nТак что заявляю вам совершенно ответственно: всех, кто утверждает/намекает, что мои тексты и посты пишет кто-то другой, можете записать на отдельный листочек. И озаглавить этот список «жулики».'),
('https://t.me/s/navalny/3428', 'Вчера в 8 вечера выпустили из ШИЗО. Сегодня в 9:30 утра я снова в ШИЗО. «Осужденный Навальный не поддается воспитательной работе и должных выводов для себя не делает». Таким образом, мой счетчик штрафного изолятора перескочил на отметку 165 суток.'),
('https://t.me/s/navalny/3459', 'Привет, это Навальный.\n\nСегодня начинается очередной суд, который сильно добавит мне срок. Но я хочу использовать этот день не для того, чтобы вызвать сочувствие к себе и другим политзекам. Я хочу призвать всех к действию и использовать этот день для объявления о нашем новом, очень важном проекте. Большой агитационной машине. Машине правды. Мы не просто хотим, а обязательно сделаем её, чтобы объединить усилия в борьбе с путинской ложью и кремлёвским лицемерием. Вы очень нужны нам. Присоединяйтесь.\n\nПочему сегодня — хороший день для этого объявления? А потому что сам суд доказывает правильность и необходимость такого проекта. Что главное в этом суде? Не беззаконие, не телефонное право, не покорность бессовестных судей и прокуроров. Главное — его форма: это суд внутри тюрьмы. Путин не стесняется сажать невиновных и не боится, что меня могут отбить восставшие толпы в ходе заседания в Москве. Но он боится того, что я скажу. Даже если это будут очевидные, известные всем слова. Он боится слова. Конечно, не только моего, поэтому и Кара-Мурзу, и многих других судили закрытым судом.\n\nПутин боится любого слова правды, ненавидит реплики, превращающиеся в интернет-мемы, бесится от «последних слов», получающих миллионные аудитории. По сути, задача укрепления и продления путинской власти решается через затыкание ртов тем, кто осмеливается говорить правду. Решению этой задачи подчинено почти все, что делается в российской политике в последние годы. Ну а с начала войны власти вообще ни о чём другом не думают. Сроки за посты, за дискредитацию, за «фейкньюс», аресты, блокировки, иноагенты и нежелательные организации. Казалось бы — ну пусть болтают? Когда власть борется с митингами, которые кажутся ей опасными, в этом есть логика, но трёп в интернете или даже по телефону — с ним-то какая проблема?\n\nКажется, что даже наоборот, недовольство уходит в пустоту. Диванные войска ставят лайки вместо того, чтобы строить баррикады. Но на самом деле политика работает не так.\n\nПутин показал себя дураком в военной сфере, бездарным военачальником, но в политике он не дурак. Он знает, что основа любого политического действия — идея и слово. Агитация и убеждение. Это очень хорошо видно во время выборов, особенно там, где они высококонкурентны. Каковы бы ни были особенности государства, политические традиции и вопросы повестки дня, все равно на выборах всё сводится к тому, что агитаторы ходят по домам и квартирам, звонят по телефону, убеждают людей в соцсетях и мессенджерах. И на каждых выборах президента США, со всеми их высокими технологиями и огромными бюджетами, кандидаты сами будут волонтёрить в колл-центрах, чтобы побудить своих сторонников тоже приходить и звонить, разъяснять и убеждать. Потому что по силе убеждения ещё ничто не перебило и не перебьёт самый базовый вид агитационного воздействия — просто поговорить, просто привести примеры и аргументы.\n\nЛюди любят заявлять, что на них не влияет предвыборная агитация. Что они и так твёрдо знают, чего хотят, и что их не переубедишь. Но это не так. Большая часть избирателей определяется с выбором на избирательном участке, поэтому хороший агитатор и сомнение зародит, и сагитирует, и переубедит. Это давно доказано. И мы сами проводили эксперименты в этой области.\n\nТак за что же агитировать, когда выборов нет? Есть за что, и ставки очень высоки. Мы будем вести избирательную кампанию против войны. И против Путина. Именно так. Долгую, упорную, изматывающую, но принципиально важную кампанию, где будем настраивать людей против войны.'),
('https://t.me/s/navalny/3462', 'Начавшийся мой уголовный суд «задвинул» все остальные мои процессы, кроме одного. В этот четверг, 22 июня, в 9:30 будет заседание Верховного суда по моему иску. И оно открытое!\n\nА кроме того, что открытое, оно еще и очень важное, касающееся не только и не столько меня, но и огромного количества других зеков.\n\nЕсли вы откроете любой документ, описывающий права осужденных, то первым там будет указано право писать жалобы, заявления, обращения и т. д. Оно вроде как железное. Понятно почему: осужденные находятся в подчиненном и почти бесправном положении, тюремные администрации имеют естественное желание превратить их вообще в бесправных и бессловесных скотов.\n\nПоэтому общество в лице законодателя говорит им: если что не так, жалуйтесь. Сразу пишите жалобу, это ваше главное право.\n\nВсе красиво, только для того, чтобы жаловаться, ручка нужна и листочек.\n\nИ тут происходит восхитительный в своей наглости и беззаконии трюк. Уголовно-исполнительный кодекс никаких ограничений на пользование письменными принадлежностями не содержит. А вот правила внутреннего распорядка зон и тюрем написаны так, что основные жалобщики — те, кого по этой причине держат в ШИЗО, ПКТ и т. д., то есть «под крышей», — легко лишаются письменных принадлежностей, если этого хочет начальник зоны. Их «выдают по распорядку». А распорядок утверждает начальник. Поэтому кому-то бумагу и ручку дают на час. В каких-то местах — на 15 минут, и, чтобы написать одно письмо, зеку нужна неделя.\n\nУ меня из распорядка время письменных принадлежностей вообще убрали. Как? Да вот так — начальник так решил. Мы пошли в суд, который постановил: начальник все решает.\n\nНу и ясное дело, что если начальник не хочет, чтоб на него писали жалобы, то жалобщику просто не дадут ни ручку, ни бумагу. Типа мы не отрицаем ваше право жаловаться, но ручку давать не обязаны. Ходите по камере и жалуйтесь сами себе.\n\nВот поэтому сейчас мы (с моими адвокатами и юристами) обжалуем в Верховном суде положения ПВР, которые делают такое беззаконие возможным.\n\nВерховный суд в качестве первой инстанции — штука не очень частая. Я в таком процессе впервые. Посмотрим, что из этого выйдет. Текст нашей жалобы, фундаментальной и классной, можно посмотреть здесь. Юристам будет интересно.'),
('https://t.me/s/navalny/3461', 'Привет всем, особенно тем, кто сидит под крышей.\n\nЯ полгода добивался, чтобы мне покрасили стены в ШИЗО, и добился. Хочу поделиться опытом, уверен, многим пригодится.\n\nИтак, вы в ШИЗО. Администрация зоны пытается сделать вашу жизнь невыносимой. Для этого есть много методов, но один из постоянно применяющихся - стены.\n\nВ ШИЗО человек постоянно стоит или сидит на ужасной табуретке/лавке. В обычной жизни мы всегда сидим, опираясь спиной на что-то. Поэтому здесь через некоторое время спина деревенеет и начнет болеть даже у здорового. Хочется опереться, сидя или стоя, хотя бы на стену.\n\nИ вот на этот случай негодяи из ФСИН покрывают стены “шубой” - неровной бугристой поверхностью, либо - как у меня - толстым слоем очень маркой побелки. Прислонился - и ты покрыт слоем мела. Плюс этот мел на полу, в воздухе, на постельном белье, одеяле (шконка-то к стене пристегнута). Короче, проблема.\n\nГлавное, что вам надо знать - это абсолютно незаконно. Проблема известная, поэтому свод правил, регулирующих тюремные заведения (СП 308.1325800.2017), говорит очень четко: “Внутренние поверхности стен камер следует штукатурить гладко и окрашивать эмульсионными красками светлых, преимущественно теплых тонов”.\n\nПоэтому пишите жалобу и дословно переписывайте этот пункт. Требую, чтобы было так.\n\nВ ответ на это администрация либо все отштукатурит и перекрасит. Либо, как это было у меня, продемонстрирует 85-й уровень наглости и беззакония и скажет: “А у нас так и есть. Все так и покрашено”.\n\nИ хоть бейся головой о стену (с которой летит штукатурка). В жизни одно, на бумажке другое, но бумажка все врет, а ты доказать ничего не можешь. Всякие ОНК и уполномоченные по правам тоже не помогут и будут на стороне администрации.\n\nЧто я сделал. Я подал в суд. Администрация, ни капли не стесняясь - суд у них в кармане, и в суд эту справку приносит: “У нас все покрашено по норме”.\nТогда я начал делать вот что. На проверках, когда все с видеорегистраторами, я говорил, что у меня есть жалобы на состояние стен и сразу проводил ладонью по стене, а потом тыкал ею в регистратор, показывая, что рука покрыта густым слоем мела.\n\nИ - самое важное - просил суд истребовать эти видео и посмотреть на заседании.\n\nАдминистрация идет в отказ: стены покрашены эмульсионной краской, а почему ладонь белая, не знаем. Может, она всегда у него такая, от рождения покрыта мелом.\n\nТогда я на очередной проверке устроил полномасштабный эксперимент. Под запись показал, что стена красит сухую руку, мокрую руку, сухую тряпку, мокрую тряпку. Одежду. Тут было смешно: прислоняюсь одним плечом - оно белое. Присутствующий замначальника по БиОр (“кум” то есть): “А у вас так и было”. Прислоняюсь другим плечом - теперь и оно белое. “”Кум” орет: “Прекратите цирк!” Я ору на него. Короче, скандал, но все на видео, и менты понимают, что это видео будет в суде. Суд, конечно, все равно будет на их стороне, но они судью-то выставят идиотом, а ему это явно не очень приятно.\n\nВ итоге через пару дней заканчивается ШИЗО, меня переводят в другую камеру, и когда через три дня возвращают, стены покрашены.\n\nГоворю ментам: “Покрасили же, охота было вам полгода скандалить и носить подложные документы в суд?”\nОни: “Ничего не знаем. Никто ничего тут не красил, всегда так было”.\n\nНу и хрен с ними. Зато я теперь могу взять книжку, сесть на пол и читать ее, уперевшись спиной в стену. Такой кайф.\n\nРезюме.\n\nСтены в камере (ШИЗО, ПКТ, ЕПКТ, любой) должны быть гладко оштукатурены и покрашены эмульсионной краской. Требуйте, и точно добьетесь. Используйте проверки для получения видеодоказательств (запись хранится месяц) и указывайте дату записи в иске в суд, жалобах в прокуратуру, Минюст и ФСИН.\n\nВезде ссылайтесь на пункт 17.15 свода правил СП 308.1325800.2017, утвержденных 20 октября 2017 года.\n\nВот. Надеюсь, совет был полезным. Пользуйтесь сами и посоветуйте остальным. Всем мир.'),
('https://t.me/s/navalny/3464', 'Интересно складывается жизнь.\n\nЕсли бы мне 22 года назад сказали: “Знаешь, к 2023 году Путин вас всех пересажает и вы будете друг друга поздравлять с днем рождения из тюрьмы”, - я бы удивился. В шоке бы не был, но удивился.\n\nНо если бы меня спросили: “Как думаешь, из тех, с кем ты работаешь, кого посадят за то, что он не испугался, не продался и не замолчал?” - я бы сказал: “Яшина”. Он всегда был такой.\n\nИлья, ты очень хороший и честный человек, но награда твоя за это пока выглядит сомнительно.\n\nНо мы знаем, что история любит неожиданные трюки, поэтому будут и другие награды.\n\nПоздравляю тебя с днем рождения. Будь здоров и оставайся таким, как есть.\n\nОбнимаю тебя, чувак.'),
('https://t.me/s/navalny/3474', 'Игорь Стрелков задержан незаконно. До тех пор, пока он остаётся под стражей по этому нелепому и очевидно политически мотивированному обвинению, он является политическим заключённым.\n\nРанее Стрелков был осуждён судом Нидерландов в процессе, который, с моей точки зрения, соответствует принципу независимости суда. В любом случае факт убийства 298 пассажиров «Боинга» должен быть расследован Россией, а все виновные — понести справедливое наказание. Сам Стрелков не отрицает своего участия в боевых действиях в Украине в неопределённом статусе. Все это тоже должно быть расследовано и получить справедливую оценку суда. Но сейчас Стрелков арестован не по обвинению в совершении настоящего преступления, а по фиктивному и сфабрикованному делу об экстремизме. Говоря просто — за критику власти. А конкретно Шойгу и Путина.\n\nБезусловно, критика Стрелкова — это критика справа. Он ругал людоедов за их недостаточный аппетит. И давал советы, как половчее есть людей. И как правильнее вести незаконную, агрессивную и глупую войну. Однако вне зависимости от взглядов Стрелкова мы должны стоять на позициях закона и справедливости. Человека можно судить только за то преступление, которое он совершил. Злорадствовать о том, что Стрелков пал жертвой своей собственной власти, означает публично поддерживать беззаконие.'),
('https://t.me/s/navalny/3488', 'С маршем на Вашингтон за рабочие места и свободу, прошедшим в этот день в 1963 году, меня разделяют десятилетия и тысячи километров. Но последние полгода это историческое мероприятие стало мне так близко, что иногда мне кажется, что я там был.\n\nДело в том, что, сидя в ШИЗО без ручки и бумаги, занимаешь себя тем, что учишь что-то наизусть. В какой-то момент я выучил легендарную речь Мартина Лютера Кинга I have a dream и произнёс её, гуляя по камере, кажется, раз двести. Каждый раз восхищаюсь ею, как в первый, и мне не надоедает это.\n\nРечь обращена к американцам и посвящена американской политике. Но она, как и сам Кинг, безусловно, достояние всего человечества. Поздравляю всех с годовщиной того, что они у нас есть.')
]

url_text_pairs_site = [('https://navalny.com/p/6653/', 'Многие удивятся, но выборы все ещё существуют. 10 сентября они пройдут в 40 регионах, в том числе в Екатеринбурге, в Омской области, в Хабаровске, в Красноярске, в Ярославской и Архангельской областях, в Хакасии и Якутии. Таким образом, нам надо определиться с отношением к этим выборам, и я позволю себе внести предложения.\n\n\nНачиная с 2011-го именно предложенные нами тактики голосования признавались наиболее эффективными. Хотя каждый раз их приходилось отстаивать в ожесточённых дискуссиях. В авторитарной стране, которой является Россия, сам по себе факт получения большего количества голосов совсем не гарантирует ни успеха, ни победы. Что мы, в общем-то, и наблюдаем, учитывая, где я пишу этот пост. Поэтому мы расцениваем участие в выборах как важнейший, но далеко не единственный элемент политической борьбы. Фразу «власть в России сменится не в результате выборов» повторили все много раз, но впервые произнёс её я в 2011 году. Однако это совсем не отменяет важности правильной предвыборной тактики.\n\nГлавное, что надо понять и запомнить (это лежит в основе каждой нашей выборной стратегии): все выборы различаются. Надо принимать решения, исходя из ситуации. Универсальных правил нет. Мы должны адаптироваться, отвечая на новые трюки и изобретения власти. Ведь посмотрите, с первого года президентства Путина правила меняются в каждом выборном цикле, чтобы наша ранее эффективная стратегия не сработала. И мы будем идиотами, если продолжим делать одно и то же, а не менять стратегию в зависимости от новых правил.\n\nВ 2011 году наша кампания «голосуй за любую партию, кроме „Единой России“» нанесла поражение партии власти. Они получили менее 50% голосов и для того, чтобы не потерять большинство в Государственной думе, пошли на совершенно открытые фальсификации в Москве. Что и привело к широкому протестному движению впервые с перестроечных времён и к формированию оппозиции нового типа.\n\n\nВ 2013 году мы впервые в истории России провели избирательную кампанию на выборах мэра Москвы, опираясь на волонтёров и активистов, и добились высокого результата без олигархических денег и административного ресурса.\n\n\nИ вновь от второго тура и невероятного поражения единоросса Собянина спасла только фальсификация.\n\nВ 2015 году мы обратили всеобщее внимание на региональные выборы, которые до этого столичной оппозицией игнорировались. Однако тут уже власть, поняв, что мы и в регионах её потреплем, не пустила на выборы людей из наших списков, несмотря на протесты и голодовки.\n\nСамой успешной стратегией стало Умное голосование, впервые использованное в 2019 году. До этого в России существовало железное политическое правило: оппозиционный кандидат не может выиграть в одномандатном округе. Для этого надо получить не менее 35–40% голосов, а это невозможно без помощи губернатора или мэра. Однако в 2019 году мы просто с первой попытки разгромили в Москве и Собянина, и Путина, и «Единую Россию». Несмотря на то, что почти никого из независимых команд кандидатов на выборы не пустили, Умное голосование, поддержавшее сильнейших кандидатов от системной оппозиции, выиграло тогда вместе с ними 24 округа из 45, лишив «Единую Россию» большинства в столичном парламенте. Как обычно, власть спасли фальсификации. Мэрия Москвы украла четыре мандата и сохранила свое большинство. Но всё же в столичный парламент прошло рекордное количество реально оппозиционных кандидатов.\n\nПоняв, что выиграть у нас с вами в крупных городах они уже не смогут, в ответ на Умное голосование власти изобрели «Дистанционное электронное голосование» (ДЭГ). Это система, которая просто рисует нужным кандидатам несуществующие голоса, и проверить это никак нельзя: наблюдают за системой сами организаторы фальсификаций. Несмотря на это, Умное голосование отлично себя показало и на региональных выборах 2020 года. Меня тогда и отравили, пытаясь убить в Томске, куда я приехал, чтобы поддержать независимых кандидатов в Городскую думу.\n\n\nОни, кстати, в итоге стали депутатами, о чём я узнал, уже лежа в больнице. По одномандатным округам кандидаты Умного голосования победили в Томске в 19 округах из 27.\n\nИ даже на последних, уже военного времени, выборах в сентябре 2022 года — на выборах депутатов муниципальных советов Москвы, — в соответствии с рекомендациями Умного голосования, которое поддерживало только антивоенных кандидатов, проголосовало более 400 тысяч избирателей (из 700 тысяч, пришедших физически на избирательные участки). Опять пришлось воровать мандаты с помощью ДЭГа — якобы в электронном виде голоса отдали 1,7 млн избирателей, — и всё равно 63 кандидата, поддержанных нами, получили депутатские мандаты.\n\nИ вот перед нами вопрос о том, как вести себя на выборах 2023-го.\n\nС 2011 года мы постоянно боремся с двумя типами дуриков.\n\nПервые кричат, что все выборы — это плохо, это работа на Путина, выборы ничего не решают и всё надо бойкотировать. Вторые — такие же дурики, но в противоположной крайности. Эти мчатся участвовать во всём, что власть назвала голосованием, даже если оно таковым совсем не является. Этим хоть на пчелиный улей, пенёк или на деревянный туалет повесь табличку «выборы», они сразу выстроятся в очередь.\n\n\nА каждые выборы или «выборы» надо анализировать, чтобы понять: чего мы можем добиться? Мы ослабим власть или усилим её, участвуя в выборах? Есть ли тут возможности хоть для какого-то контроля? Есть ли возможность для агитации? Чего от нас ждут? В чём подвох (а он всегда есть)? Исходя из этого, надо где-то голосовать и агитировать изо всех сил. Где-то просто наблюдать. Где-то игнорировать. Где-то вести кампанию забастовки и активного бойкота.\n\nВ чем особенность выборов-2023?\n1. Наша лучшая стратегия, Умное голосование, на них неприменима. По крайней мере, в прежнем виде. Объясняю.\n\nСитуация 1. Есть отличный кандидат, Умное голосование поддерживает его. Его моментально не просто снимают с выборов, а сажают.\nСитуация 2. Есть просто кандидат-неединоросс, который может победить с помощью Умного голосования. Мы поддерживаем его. Кандидата моментально везут на фронт, где он фотографируется, обвешанный автоматами, гранатами и отрезанными головами. После этого мы с вами занимаемся не выборами и агитацией — мы отбиваем в твиттере истерики о том, что Умное голосование поддерживает людоедов. Сама власть истерики и инициирует. Через «твёрдых собянинцев» в том числе.\n2. Уровень фальсификаций, в том числе через электронное голосование, так высок, а наблюдение так ограничено, что в этих выборах почти не осталось выборов. Сразу оговорюсь: не везде. Где-то можно и нужно участвовать. Где-то мы и Умное голосование применим втихаря, насколько это возможно. Но в целом с возможностью защитить результат всё не очень.\n\n3. Снова: все выборы разные. Например, выборы в Хакасии, где даже дебаты проходят между губернатором-коммунистом и соперничающим с ним единороссом, присланным из Москвы, интересны и конкурентны. Или выборы в Гордуму Екатеринбурга, где воюют между собой мэр города и губернатор области. А выборы мэра Москвы-2023 устроены так, что как ни проголосуй — проголосуешь за Путина. Либо за Собянина, либо за «путинское крыло КПРФ», представленное Зюгановым-внуком. Если пытаться устраивать протестное голосование в Москве, то вот заранее подготовлен канал для такого протеста: Зюганов-внук. Кампанию не ведёт. Власть хвалит. Делает что велят. Протестное голосование может немного добавить ему голосов, но это полная калька кремлёвской тактики с Собчак на выборах президента-2018: «Ах, у нас есть оппозиция, ну так вот тебе, оппозиция, кандидат. Давай, агитируй за него, а мы потом посмеёмся над результатами». Ну и интерес к московским выборам соответствующий. Это третья по важности избирательная кампания в России. А я уверен, что большинство из вас не знает кандидатов, кроме Собянина и Зюганова.\n\nТак можно ли чего-то добиться на выборах-2023? Я считаю, что можно. Просто наши цели возвращаются к самым базовым:\n\n1. Мы, оппозиционные избиратели, должны сохранить привычку и правило ходить на выборы. Рано или поздно в России они пройдут относительно свободно. Мы должны на них победить. Этого не получится, если мы убедим себя, что выборы не имеют смысла и значения, и привыкнем на них не ходить.\n\n2. Надо поддерживать желание людей баллотироваться. Немало хороших кандидатов в регионах. Есть прямо отличные. Просто лично я нахожусь в ситуации, когда мне лучше их фамилии не называть. Но в любом случае им нужны наши голоса и поддержка. И точно не нужно нытьё о том, что всё бесполезно.\n\n3. Крайне важно сохранить остатки инфраструктуры наблюдения. Как я сказал, рано или поздно в России пройдут относительно свободные выборы, но и на этих выборах у нас вновь попытаются украсть голоса. Нам нужны будут опытные наблюдатели.\n\nНаконец-то перехожу к тому, что делать.\n\nЯ считаю, что в нынешней ситуации нам надо сделать шаг назад и вернуться к стратегии голосования «за любого кандидата против „Единой России“». Идите на выборы с целью нанести максимально возможный урон партии власти и её кандидатам.\n\nРезюме:\n1. 10 сентября надо идти на участки и голосовать.\n\n2. Выберите в бюллетене кандидата, который не является членом «Единой России», и голосуйте за него. Хорошие кандидаты есть, и много.\n\n3. Те, кто участвует в выборах как независимые кандидаты, — молодцы, и их стоит поддерживать.\n\n4. Умное голосование мы делать будем, но точечно и непублично.\n\n5. Обязательно стоит участвовать в наблюдении за выборами, для этого использовать любые возможности. Наблюдательские организации существуют до сих пор, и многие толково работают.\n\n6. Выборы мэра Москвы 2023 значения не имеют. Они фиктивны. Можно проголосовать, ради любопытства сходив на участок, можно игнорировать. Они не стоят того, чтобы их вообще обсуждать.\n\n7. Тактика по президентским выборам в марте 2024 года может быть определена только после того, как будет ясен общий расклад с правилами, кандидатами и наблюдением.\n\nСтратегически мы считаем Умное голосование и его участников главным достоянием и лучшей возможностью оппозиции. Но ключевое слово здесь — «умное». Оценив новые обстоятельства, войну и политическую ситуацию, мы должны поступить умно и сделать шаг назад, голосуя за любого кандидата и за любую партию против «Единой России».'),
('https://navalny.com/p/6655/', 'В связи с терактом, который устроил Путин, чтобы убить своего солдата Пригожина, хочется сказать о порядочности, хитроумии и стратегии.\n\n\nПорядочность. Буквально пару дней назад писал одному человеку, объясняя, в чем секрет выживаемости Пригожина. Скорее всего, всё просто. Личная договорённость и путинскоe пацанское слово. Пригожин обещал Путину, что в лепёшку расшибется для него и его войны. Будет его самым надёжным командиром, готовым пойти на всё ради победы. Путин, глядя на, мягко говоря, отсутствие энтузиазма у Министерства обороны, это оценил. Он дал Пригожину небывалого уровня поддержку (шутка ли, тысячи зеков за один день из тюрем вынимали). Ну и был — предполагал я — некий классический диалог:\n\n— Я за вас на всё пойду, Владимир Владимирович. Победа будет за нами любой ценой. Но и вы уж меня не подведите и на съедение не отдавайте.\n\n— Женя, вот тебе моя рука и моё слово. Выполни задачу, а я всегда за твоей спиной. Поддержка гарантирована.\n\nВо время «мятежа» Пригожин психанул «по делу», и сути отношений это не изменило. Путинское слово продолжает действовать.\n\nКакой глупостью было написать это. Какая наивность — предположить, что Путину может быть свойственна порядочность любого вида: офицерская, пацанская, бандитская.\n\nТам же нет ничего, кроме постоянного вранья по абсолютно любому поводу. Ври, воруй, отнимай, убивай, убегай.\n\nХитроумность. В этом пункте снова стоит посмеяться над теми, кто верит в ювелирную работу спецслужб, многоходовые планы и «захотят убить — там умеют убивать так, что комар носа не подточит».\n\nВот вам тонкая работа путинских убийц. Сбить/взорвать над Россией пассажирский самолёт, то есть устроить настоящий теракт, убив в нём ещё и абсолютно не причастных пилота, стюардессу и т. д.\n\n\nСтратегия. По стране ходят 50–60 тысяч бывших зеков и просто бывших участников войны, облепленных нашивками «Вагнер».\n\nЕщё несколько сотен тысяч поклонников этого культа. Его официально воспевали, и он довольно популярен: «Вагнер», «оркестр», «музыканты», «наши в Африке», продвигают российские интересы и так далее.\n\nПригожина и Уткина награждают звёздами героев. Два президента, России и Беларуси, дают им гарантии безопасности. А теперь — что произошло глазами поклонников «Вагнеров»: русские офицеры-герои не щадили себя на войне в Украине. Это супермегаспецназовцы-профи. Их обманули предатели-генералы. Их сослали в Африку — они и там заявили, что будут укреплять мощь России, начинать экспансию и вновь заставят считаться с русским оружием.\n\nНо предатель и трус Путин в своём Кремле завидует популярности этих русских героев в народе и войсках и ненавидит их за это. Поэтому отдал подлый приказ убить их.\n\nТаким образом, Пригожин и Уткин в глазах многих из тех чувачков, что любят к месту и не к месту наряжаться в камуфляж, превратились в мучеников.\n\n\nВ «легенды» этой войны.\n\nВ глазах историков, кстати, тоже легенды и самые интересные персонажи. Фильмы, книги и конспирологические теории о том, что они выжили и воюют в джунглях Конго.\n\nНо дело даже не в том, что для части самых агрессивных сторонников войны с Украиной история этой войны — это теперь сага о том, как Путин предал и убил своих верных офицеров.\n\nА в том, что именно из таких ингредиентов создается блюдо «гражданская война».\n\nСоздали банду. Вооружили банду. Распустили банду. Убили главарей.\n\nМетафора «пауки в банке» часто применяется к тем, кто сидит в Кремле. Но больше подходит другая — «обезьяна с гранатой».'),
('https://navalny.com/p/6654/', 'Начали приходить письма с отзывами о моем посте про страх и ненависть.\n\nЧитаю с огромным интересом. Большое спасибо всем, кто обратил внимание на эту дискуссию. Повторюсь, она не теоретическая, не историческая, не политологическая. Это самая актуальная дискуссия о том, как победить «путинский режим». Ведь этот режим — это не только Путин персонально, но и часть элиты (в широком смысле), которая считает текущий способ управления нашей страной подходящим. «Есть, конечно, перегибы, но в целом приемлемо. Иначе нельзя. Такой народ» (c).\n\n\nА у нас — оппозиции — другая точка зрения… «Эээ…а какая?» — уточняют у нас избиратели. А мы, выясняется, до сих пор не определились с вопросом, нужно ли обсуждать подделку выборов и коррупцию. Победа над «путинским режимом» в конечном итоге означает победу над его сторонниками на выборах. А у нас на знамёнах Чубайс, Собчак, ельцинская Конституция и союз с ЧВК Вагнера имени Ходорковского. Класс.\n\nГарантированные 1,5% на несвободных выборах и 0,5% на свободных. Обо всём этом я постараюсь подробно написать в своей реакции на реакции, когда все прочитаю и обдумаю.\n\nА пока два очень важных замечания по ходу дискуссии.\n\nПервое. Эту часть я переписывал три раза, каждый раз снижая уровень оскорблений и персональных нападок, пока не дошел до доброжелательного и конструктивного тона (кажется).\n\nЯ веду блог лет 20 и написал тысячи постов. Я сижу уже 2,5 года и тоже сумел что-то написать.\n\nНо вот во второй раз в своей жизни (первый после поста «Мне стыдно за Михаила Ходорковского») я сталкиваюсь с реакцией в виде откровенной и наглой лжи о том, что тексты якобы пишу не я. Ложь эта варьируется от прямой («это пишет не Навальный»):\n\n\nдо ещё более тошнотворного виляния и намёков в стиле «многие говорят, что писал не он, но опровержений нет, поэтому, наверное, он, но вообще кто его знает».\n\n\nОбратите внимание, что это не просто враньё в соцсетях, а кто-то даже оплатил и разместил эту заказную статью.\n\nТут надо понять, что во лжи обвиняют меня. Я ж, выходит, вам вру. Кто-то пишет за меня, а я присваиваю себе его тексты. И не технические какие-то, а программные. Не надо придуриваться и предполагать, что я могу этого не знать. Уровень моей изоляции весьма высок. Но я, как и любой зек, имею право на переписку и на любое своё высказывание.\n\nЯ уже высказывался о «сомнениях в авторстве» тех слов ясно и повторяю последний раз доброжелательно и конструктивно, но твёрдо.\n\nЧасть из тех, кто повторяет эту ложь, просто глупенькие. Им простим.\n\n\nЧасть — лицемеры. Знают, что врут, но придуриваются. Это тоже ладно.\n\nНо организаторы и постоянные настойчивые промоутеры этой идеи не только отвратительные негодяи и бессовестные люди, но и публичные доносчики (стукачи).\n\nОбъясню.\n\nМне за что только что 19 лет дали? За слова. В буквальном смысле. За посты, видеоролики, которые путинский суд признал экстремистскими. Значительная часть из них размещена ПОСЛЕ того, как я сел.\n\nА в идущем сейчас следствии по делу о терроризме суть обвинения та же — публикации в интернете. Но все они размещены уже после моей посадки.\n\nВы не представляете, до какой степени совпадают интересы и аргументация тех лжецов и жуликов, что обсуждают моё авторство в соцсетях, и путинских следователей.\n\n\n— А вы это пишете? Подтвердите авторство. А как вы это пишете? Нет, ну давайте обсудим. Как вот технически это делается? Телефон у вас есть или передаёте через кого-то? Через адвокатов или через письма? Нам всё-всё надо узнать. Объяснитесь перед нами.\n\nПотому то, что делают эти мерзавцы, и есть публичный донос. Они отлично всё знают, всё понимают, но продолжают своё: «А чем ты докажешь, что это ты? А ну-ка раскрой нам все детали. А то мы так и будем продолжать каждый раз наш блядский цирк с тем, что пишешь не ты».\n\n«А вы их, товарищ Бастрыкин и товарищ Бортников, проверьте. Вы всё проверьте и обратите внимание. Как это он сидит и продолжает нехорошие вещи и про товарища Путина писать».\n\n«Мы его сейчас вынудим всё рассказать, а вы уж примите меры. Всю цепочку накройте. Безобразия эти прекращайте».\n\nЭто всё именно так и есть. Не выглядит публичным доносом, а с учётом настойчивости им является. И мне самому, положим, нечего терять, но ведь я-то не один.\n\nПоэтому у меня такое предложение ко всем нормальным людям:\n\nа) если видите какие-то очередные рассуждения об авторстве текстов политзеков (кроме очевидных случаев взлома соцсетей, но это сразу ясно), тем более с «а докажи нам, что это ты» и обсуждением механизма, — знайте: перед вами лжец, негодяй и доносчик. Никогда больше не доверяйте такому человеку.\n\nб) Видите такое — заходите в комментарии и возите негодяя лицом об стол. Вопрос этот касается далеко не меня одного. Политзеков много и будет больше. Все эти тупые обсуждения осложнят жизнь и им, и их родственникам, и адвокатам.\n\nВажное замечание номер два. Чуть ли не самая популярная реакция от тех, кто посчитал мой пост критикой в свой адрес, выглядит так: «Всё Навальный неправильно написал, но я не могу критиковать его, пока он сидит в тюрьме».\n\n\nПризываю абсолютно всех: давайте перестанем так рассуждать, это глупости. В тюрьмах сидит полмиллиона человек. Их всех критиковать нельзя? И не все политзеки обязательно являются какими-то хорошими и святыми людьми. Само по себе сидение в тюрьме не делает никого лучше или хуже. Судить человека надо по его делам, а не по месту нахождения.\n\nС сидящими не нужно спорить только по всяческим тюремным темам, потому что вы в них ничего не понимаете, если сами не сидели. И даже если прочитали кучу книг об этом и родственник ваш сидел — всё равно не понимаете. В этом я на собственном примере убедился. Да и не надо вам в этом ничего понимать, оно того не стоит.\n\nИ самое важное. Вы же меня так самого главного лишаете. И самого дорогого. Критики.\n\nКак мне иначе-то думать, работать и улучшать себя?\n\nКритика и конкуренция — основа прогресса, без них мне (и ФБК) никак.\n\nНапример, я ругаю Алексея Венедиктова и продолжу это делать. Но, объективно говоря, именно он оказал мне (и ФБК) самую ценную услугу за последний год.\n\nОн опубликовал известное письмо Леонида Волкова. Спровоцировал огромную волну критики. Была важнейшая дискуссия и вообще, и внутри ФБК. Мы были вынуждены справляться с кризисом. Лёня ушел в отставку с поста главы международного ФБК. Мы изменили некоторые наши внутренние процедуры. Мы думали, как исправить проблему, и исправили её. Леонид очень подробно и публично объяснил свои действия (раз, два, три). Я об этом тоже сделал заявление.\n\nФБК получил критический опыт, исправил ошибку и стал лучше. Волков отреагировал как ответственный политик, исправил ошибку и стал лучше. Сейчас он очень успешно работает над нашим важнейшим проектом. Я тоже тогда получил свою долю критики и навсегда учту этот опыт в своей работе.\n\nВы, в свою очередь, тоже получили полезные знания: увидели, как ФБК справляется с кризисной ситуацией. Исправляет ли ошибку. Действует ли ответственно. Теперь вы имеете большее представление о том, чего от нас ожидать и как мы будем действовать, в том числе оказавшись во власти. Вы можете сравнить наше поведение в кризисной ситуации с поведением других политиков и политических организаций.\n\nКритика и конкуренция оказались полезны всем.\n\nВ ситуации с моим постом (программным) то же самое.\n\nЯ нуждаюсь в вашем мнении. Если вы согласны — это важно, и я это отмечу. Если не согласны — я либо опровергну ваши аргументы, либо соглашусь с ними.\n\nЗнаете, для меня давно фраза «оппозиция снова ругается» — маркер идиота.\n\nЭто политические дебаты, даже если они в твиттере или комментах фейсбука. Без них нет жизни и нет прогресса. Если вам хочется, чтобы люди с разными точками зрения не спорили между собой, то вам в «Единую Россию» или в КПСС.\n\nТак что критикуйте меня, пожалуйста, если не согласны, я без этого никуда.'),
('https://navalny.com/p/6651/', 'Давно собирался написать об этом. Ну, пусть будет первым постом после нового приговора. Исповедь практически. Мне надо побороть эти ненависть и страх, может быть, вы мне поможете в этом.\n\n\nНенависть. Об этом всегда спрашивают много, и снова письма пошли: ну как, ненавидишь судью? А Путина ещё больше ненавидишь? А я и раньше много раз говорил, что ненависть — это главное, что нужно побороть в тюрьме. Тут так много причин для неё, а твоё бессилие — мощнейший катализатор процесса. Так что, если дать ненависти волю, она доконает и сожрёт тебя.\n\nНо, признаюсь честно, есть она у меня. И ещё какая. Старожилы интернета помнят мем: люто, бешено ненавижу. Вот примерно так. Причём действительно чаще всего после «судов». Последний, кстати, где мне дали 19 лет, не из таких. Там, наоборот, мы все соревновались в осыпании друг друга любезностями. За весь процесс никто ни разу голоса не повысил. Это самый опасный вид судьи: он тебе и 19 лет впаяет, и сделает так, что ты проникнешься к нему симпатией.\n\nБешусь я страшно после заседаний местного райсуда. Там простейшие дела, пространства для юридических ухищрений нет, и судьи просто и откровенно говорят на чёрное: «А это белое, смотрите, в справке написано — белое» и выносят демонстративно незаконные решения.\n\nНо хоть я иногда, не выдерживая, ору на какого-нибудь «судью» Самойлова, ненавижу я своей огромной ненавистью совсем не его. И не ментов-беспредельщиков-воров из колонии. И не ФСБшников, что ими командуют. И даже, вы удивитесь, не Путина. Ненавижу я в такие моменты тех, кого любил. За кого был горой, из-за кого спорил до хрипоты. И себя ненавижу за то, что их когда-то любил.\n\nСмотрите. Сижу я в своем ШИЗО и читаю книгу Натана Щаранского «Не убоюсь зла» (рекомендую). Щаранский отсидел в СССР девять лет, в 1986 году его обменяли. Он уехал в Израиль, создал партию, добился огромных успехов. В общем, крутой он. Провёл, кстати, в карцерах и ШИЗО 400 дней. И я реально не представляю, как он выжил.\n\nТак вот, Щаранский описывает арест и следствие. 1977 год. Мне тогда был один год. Книга вышла в СССР в 1991-м. Мне тогда было 15 лет. А сейчас мне 47, и, читая его книгу, я иногда трясу головой, чтобы избавиться от ощущения, что я читаю свое личное дело. Например: здание ШИЗО/ПКТ — отдельный барак за колючей проволокой. Максимальный срок в ШИЗО — 15 суток. И я не удивился, когда после нескольких «пятнашек» подряд меня как злостного нарушителя перевели в ПКТ на полгода. Один в один.\n\nВ предисловии (напоминаю, это 91-й год) Щаранский пишет, что именно в тюрьмах сохранился вирус свободомыслия и он надеется, что КГБ не найдёт «противоядие против этого вируса». Щаранский ошибся. Противоядие нашлось. Да такое, что сейчас, в 2023-м, политзеков в России, кажется, больше, чем в брежневско-андроповские времена. Только при чём тут КГБ? Не было в нашей стране ни ползучего, ни явного переворота во главе с людьми из спецслужб. Не приходили они к власти, оттеснив от неё демократов-реформаторов. Сами. Сами позвали. Сами пригласили их. Сами научили, как выборы подделывать. Как собственность целыми отраслями красть. Как врать в СМИ. Как менять законы под себя. Как силой подавлять оппозицию. И даже как устраивать идиотские бездарные войны.\n\nПоэтому ничего я не могу с собой поделать и люто, бешено ненавижу тех, кто продал, пропил, растратил впустую тот исторический шанс, что был у нашей страны в начале девяностых. Ненавижу Ельцина с «Таней и Валей», Чубайса и всю остальную продажную семейку, которые поставили Путина у власти. Ненавижу аферистов, которых мы почему-то называли реформаторами. Сейчас-то ясно как день, что ничем, кроме интриг и собственного благосостояния, они не занимались. В какой ещё стране столько министров «правительства реформ» стали миллионерами и миллиардерами? Ненавижу авторов глупейшей авторитарной конституции, которую нам, идиотам, втюхали как демократическую, уже тогда дав президенту полномочия полноправного монарха.\n\nОсобенно ненавижу всех за то, что не было даже серьёзной попытки убрать основы беззакония — провести судебную реформу, без которой все остальные реформы обречены на провал. Я много изучаю это сейчас. В 1991-м ещё в РСФСР приняли хорошую концепцию судебной реформы, но уже с 93-го года начались контрреформы, направленные на строительство судебной вертикали. Тогда все политические силы хотели честных судов. Полнейший консенсус в обществе. Если бы независимая судебная власть была создана, то новая узурпация было бы невозможна или сильно затруднена. Так что не заблуждайтесь: штуку, которая сейчас лихо выдает невиновным сроки по 8-15-20 лет, начали строить задолго до Путина. И сейчас ясно: в Кремле и правительстве девяностых никто и не хотел независимого суда. Потому что такой суд стал бы помехой на пути коррупции, подделки выборов, превращения губернаторов и мэров в несменяемых князьков.\n\nНенавижу «независимые СМИ» и «демократическую общественность», обеспечившие полную поддержку одному из самых драматичных поворотных событий нашей новой истории — подделке президентских выборов 96-го года. Повторюсь, я был тогда активным сторонником этого всего. Не фальсификации выборов, конечно, это бы мне и тогда не понравилось, — но я сделал всё, чтобы этого не замечать, а общая несправедливость выборов меня ни капли не смущала. Сейчас мы платим за то, что в 96-м посчитали, что подделка результатов выборов — это не всегда плохо. Цель оправдывала средства.\n\nНенавижу олигарха Гусинского (хоть он уже давно не олигарх) за то, что он демонстративно взял на работу замглавы КГБ Бобкова, отвечавшего за преследования диссидентов. Им тогда казалось, что это шуточки: ха-ха, он сажал невиновных, а теперь работает у меня. Типа как медведь в ливрее. То есть не просто не было люстрации — было поощрение негодяев. И сейчас буквально те, кто работал у Бобкова молодыми сотрудниками, сажают и Яшина, и Кара-Мурзу, и меня.\n\nЧасто приходится слышать, что ельцинское правительство ничего не могло сделать, потому что в парламенте им противостояли коммунисты. Но тем не менее залоговым аукционам 96-го года это не помешало. А вот судебной реформе и реформе спецслужб почему-то помешало.\n\nНенавижу все руководство России, имевшее в 91-м (после путча) и в 93-м (после расстрела парламента) абсолютную полноту власти и даже не попытавшееся делать очевидные демократические реформы. Того, что было сделано в Чехии (где сейчас демократия и средняя зарплата 181 тысяча рублей), Польше (демократия и 179 тысяч рублей), Эстонии (демократия и 192 тысячи рублей), Литве (демократия и 208 тысяч рублей) и других странах Восточной Европы. Конечно, разные люди тогда были у власти. И очень хорошие, честные, искренние тоже. Но это крохотное меньшинство, отчаянная и безуспешная борьба которого только ещё лучше показывает нам коррупцию и бессовестность тогдашней властной элиты.\n\nЭто не с Путиным в 2011-м, а с Ельциным, Чубайсом, олигархами и всей комсомольско-партийной бандой, называвшей себя «демократами», мы в 1994-м пошли не в Европу, а в Среднюю Азию. Обменяли наше европейское будущее на виллы «Тани и Вали» на «острове миллионеров» Сен-Барт. И когда пресловутые путинские КГБшники/ФСБшники получили свободный доступ к политическим постам, им и делать ничего не пришлось. Просто оглядеться и с удивлением воскликнуть: а что, так можно было? И если правила игры такие, что можно красть, врать, фальсифицировать, цензурировать, а все суды под нашим контролем, то мы тут, считай, неплохо развернёмся.\n\nМы пустили козла на капустный склад, а потом удивляемся, что он сожрал всю капусту. Он козёл, его миссия и цель — сожрать капусту, ничего другого ему в голову не приходит в принципе. Бесполезно его агитировать. Так и путинскому чиновнику из ФСБ в принципе не приходит и не может прийти в голову ничего, кроме того, чтобы построить огроменнейший дом и сажать тех, кто ему не нравится. Я хоть терпеть не могу козла, но люто и бешено ненавижу тех, кто пустил его на капустный склад.\n\nХотя, конечно, я понимаю, что лучше вообще никого не ненавидеть, а думать о том, как это не повторить. И тут я перехожу к своему большому страху. Я не просто верю, а знаю, что у России будет ещё шанс. Это исторический процесс. Мы опять будем на развилке.\n\nВ ужасе и холодном поту я подскакиваю ночью на своих нарах, когда мне чудится, что у нас снова был шанс, но мы опять пошли той же дорогой, что в девяностые. По указателю «цель оправдывает средства». Там, где ещё маленькими буковками приписано: «подделывать выборы — это не всегда плохо», «да ты посмотри на этот народ, ну какие из них присяжные», «неважно, что он вор, зато технократ и за велодорожки», «этим людям дай волю, они такого навыбирают», «правительство все еще единственный европеец в России» и другие мудрости просвещённого авторитаризма.\n\nТо, что я написал про девяностые, — это не исторические упражнения, не рефлексия и не бессмысленное нытьё. Это важнейший и самый актуальный вопрос политической стратегии всех сторонников европейского пути и демократического развития. На меня, знаете ли, огромное впечатление произвела большая подборка разных мнений о нашем расследовании про Алексея Венедиктова и Ксению Собчак. Они получали десятки и сотни миллионов рублей из бюджетного фонда, служившего единороссам общаком. Причём Венедиктов получил 550 миллионов прямо в то время, когда он руководил штабом наблюдения и непосредственно организовал кражу голосов избирателей. Был лицом, агитатором и контролёром электронного голосования, цель которого в том, чтобы взять ваш голос и переложить его в стопочку единоросса. Фальсификации ДЭГ тщательно доказаны и не подлежат никакому сомнению. Так вот, я поразился тому, что нашлось-таки существенное количество людей, для которых ни элементы схемы «деньги из общака и подделка выборов», ни их объединение «деньги из общака во время подделки выборов» не являются ни порочащими, ни значимыми. Да ну, фигня какая-то. Да, что-то мутили там, нет же доказательства, что ему платили за фальсификации, — просто платили и просто фальсификации. Все это было во времена мамонтов. Началось аж в 2019 году. Уже никто не помнит. Это всё неважно, главное, что он сейчас «против войны». Как чётко было написано в одном из твитов на тему: «А чё такого?» — как национальная идея.\n\n\nЭто частный пример, но он, как и ситуация с Мурзагуловым, как и призывы Ходорковского брать оружие и вставать в отряды Пригожина, прекрасно показывает, что и сейчас, в 23-м году, на фоне репрессий, посадок и войны, у нас по-прежнему лояльность принципам подвергается сомнениям и рассматривается многими как наивная, романтическая и вообще как «белое пальто». Личная лояльность, принадлежность к корпорации, старая дружба расцениваются многими как более важные.\n\nЯ не предлагаю ни в коем случае расстрелять Алексея Венедиктова, или повесить, или аккуратно подстричь. Не надо никаких зверств. Но ведь можно же НЕ ОДОБРЯТЬ того, что он сделал (и делает, продолжая рассказывать, что ДЭГ не фальсифицировалось), и не считать его политическим союзником. Потому что, простите, если для нас политический союзник — тот, кто продаёт наши голоса «Единой России», то кто мы вообще, для чего мы нужны?\n\nПошли тогда просто вступим в «Единую Россию». Создадим там фракцию твёрдых собянинцев (я так их про себя называю), основа уже есть. Фигурантов примерно каждого расследования ФБК сразу мчится оправдывать дрим-тим: Ксения Собчак (раз, два), Алексей Венедиктов, Максим Кац (раз, два, три) и бывший «нашист», а сейчас почему-то главред «Новой газеты» Кирилл Мартынов. Все будет отлично. Денег будет много. Мы, твёрдые собянинцы, требуем: немедленно уберите от нас плохого Путина и дайте нам хороших Собянина и Мишустина, Шувалова и Ликсутова.\n\nТак что не сомневайтесь. Завтра будет новый шанс — то самое окно возможностей, прямо завтра нам придётся иметь дело с теми, кто считает, что кое-где выборы надо отменить или подделать («а то выберут экстремистов»), подкупать журналистов — это нормально («мы никому не платим, просто попросили знакомого олигарха купить этот телеканал»), суды лучше подержать на крючке («а то подкупят судей и присяжных»), кадровую основу власти менять нельзя («они же профессионалы, не с улицы же нам людей набирать»), и так далее, и так далее. Вплоть до того, что подряд на строительство вон того моста надо отдать не на конкурс, а «надёжному подрядчику», с которым мы давно работаем. И люди с такими идеями будут вовсе не путинисты и не коммунисты — они вновь станут называть себя демократами и либералами.\n\nРеальная жизнь сложна, тяжела и полна компромиссов с неприятными людьми. Но мы хотя бы сами не должны в инициативном порядке становиться неприятными людьми и приветствовать коррупцию с циничными махинациями ещё до того, как обстоятельства требуют компромиссов.\n\nЯ очень боюсь, что битва за принципы снова может быть проиграна под лозунгами «реальной политики». Посоветуйте мне, пожалуйста, как избавиться от этих ненависти и страха. Мне бы очень было интересно почитать какие-то ваши мысли об этом. Я попрошу, и мне пришлют по почте отзывы, если они будут. Пока, мне кажется, ничего тут не придумаешь лучше, чем оставаться верным себе и неустанно разъяснять людям на многочисленных примерах (кстати, на русском вышла книга Гуриева и Трейсмана «Диктаторы обмана», там как раз много об этом, очень советую), что демократические принципы — прагматичность, независимый суд, честные выборы и равенство всех перед законом — лучшие механизмы суровой реальной жизни на пути к процветанию. А тайные фонды-общаки и мечты о хорошем диктаторе — это химера и наивная чушь.\n\nТолько когда подавляющее большинство в российской оппозиции будет состоять из тех, кто ни при каких условиях не принимает поддельные выборы, неправильный суд и коррупцию, тогда мы сможем правильно распорядиться шансом, который обязательно выпадет вновь. Так, чтобы никто году этак в 2055-м не читал в ШИЗО книгу Щаранского, думая: надо же, всё как у меня.'),
('https://navalny.com/p/6647/', 'Через сутки зачитают приговор, и я хотел бы сказать пару вещей до того, как он прозвучит. Чтобы, так сказать, «поставить цифры в контекст».\n\nСрок будет большой. То, что называется «сталинский». Формула его расчёта простая: то, что попросил прокурор, минус 10–15%. Попросили 20, дадут 18 или вроде того. Это не имеет большого значения, ведь следом уже на всех парах идёт дело о терроризме. Там ещё лет 10 накинуть можно.\n\nТак вот, у меня просьба номер один. Когда прозвучит цифра, вы, пожалуйста, не проявляйте солидарность со мной причитаниями и восклицаниями: «Как при Сталине». Лучше проявите солидарность со мной и другими политзаключёнными тем, что минутку подумайте. Подумайте, зачем нужен такой показательно огромный срок. Его главный смысл — запугать. Вас, а не меня. Я даже так скажу: персонально вас, читающего эти строки.\n\nВы обратили внимание — пропаганда об этом процессе молчит. По телевизору о нём не рассказывают. Потому что на «простого россиянина» история произведёт эффект неоднозначный. Восемнадцать лет за какой-то «экстремизм» без жертв и последствий покажутся ему очевидно несправедливыми, и он ещё тайно симпатизировать начнёт. А вы и так всё знаете. Вас и надо ошеломить, запугать. Суровостью приговора выбить из головы мысли о противодействии.\n\nПожалуйста, подумайте и поймите, что, сажая сотни, Путин пытается запугать миллионы. Мы живём в стране, где десятки миллионов человек прямо сейчас настроены против коррупции, войны и беззакония. Прямо сейчас десятки миллионов за честные выборы, демократию, сменяемость власти и хотят, чтобы Путин ушёл. Мы точно знаем, что если бы каждый десятый из тех, кто возмущён коррупцией Путина и его чиновников, вышел на улицу, то власть пала бы завтра. Точно знаем, что недовольные войной, выйдя на улицу, прекратили бы её немедленно.\n\nНо это всё пустые мечтания. Так не работает. Кто-то должен быть первым, а им быть страшно. Россия тут не исключение. Просто недовольные не выходят на улицы стран с диктаторскими режимами почти до самого конца этих режимов: ни у нас, ни в СССР, ни в Иране, ни на Кубе, ни в ГДР. Всех перемен добиваются 10% граждан — самые активные. То есть вы. Репрессировать (посадить, наказать, оштрафовать) даже 10% из них — это полтора миллиона человек — сейчас нельзя ни политически, ни организационно. Значит, надо ошеломить и запугать, лишить охоты делать хоть что-то.\n\nТакая путинская (да и любого, в общем, диктатора) стратегия работает. Один пример. Главной неуязвимостью нашей организации всегда было то, что нам нельзя было перекрыть деньги: нас финансируют десятки, сотни тысяч людей небольшими платежами. Не перекроешь. Но, раздувая дела о финансировании экстремизма (одно из обвинений, по которому мне завтра вынесут приговор), власти добились того, что изнутри России, а это 95% наших доноров, поддерживать стало «стремновато». А финансы — основа деятельности, без них никуда. Запугивание сработало отлично. Напоминаю, кстати, что вот инструкция, как поддерживать нас анонимно и безопасно через криптовалюту.\n\nЧестно говоря, мы ещё часто сами помогаем путинской стратегии запугивания, устраивая истерики с хватанием за сердце из-за каждого ареста, дополнительно пугая и себя, и окружающих. Обо всех надо говорить, ни о ком нельзя забывать, но при этом твёрдо осознавать: власть в России узурпирована, незаконно захвачена. Те, кто у власти, не могут удержать её без арестов невиновных. Они сажают сотни, чтобы запугать миллионы.\n\nК этому надо относиться хладнокровно. Путин не должен достичь своей цели. И я очень хочу, чтобы в части приговора мне он её тоже не достиг. Поэтому моя просьба номер два: когда огласят приговор, думайте, пожалуйста, только одну, действительно важную мысль — что я лично ещё могу сделать, чтобы сопротивляться? Чтобы помешать негодяям и ворам, засевшим в Кремле, поедать мою страну и моё будущее? Что я могу сделать, взвесив все риски и учтя все обстоятельства?\n\nТретья просьба самая важная. Отвечая себе на этот вопрос, не смейте, пожалуйста, говорить: «Ничего». Вы можете. Что-то может сделать каждый. С соседями поговорить, листовку повесить. Наше расследование распространить. Отправлять 500 рублей в месяц нам или другим оппозиционным организациям и СМИ. Вести блог. Участвовать в ДМП-2, писать в соцсети. Поддерживать политзаключённых. Нарисовать граффити. Выйти на митинг.\n\nНет ничего стыдного в том, чтобы выбрать наиболее безопасный способ противодействия. Стыдно не делать ничего. Стыдно дать себя запугать. Какой бы они там ни запланировали приговор, он не достигнет своей цели, если вы поняли, для чего это всё, и ответили: «Я не боюсь» — ежедневным хладнокровным, пусть и небольшим, но вкладом в борьбу за свободу в России.\n\nБлагодарности:\n\n1. Вам всем, конечно, спасибо. Всё, что со мной происходит, переносить гораздо легче, потому что я чувствую вашу поддержку каждый день и каждую минуту.\n\n2. Огромное спасибо стороне здравого смысла в юридическом безумии под названием «суд в тюрьме», что завершится завтра. Моим адвокатам Ольге Михайловой, Вадиму Кобзеву, Александру Федулову и адвокату Дани Холодного Светлане Давыдовой. Они бьются как львы.\n\n3. Свидетелям защиты: Ступину, Кара-Мурзе, Ройзману, Чанышевой, Останину, Горинову, Яшину, Муратову, Демчук, Голикову, Николаенко, Попову, Морозу.\n\n4. Отдельное спасибо. Главный личный мой вдохновитель в этом процессе — мой подельник Даня Холодный, случайно попавший в эту мясорубку 25-летний парень.\n\n\nДаня Холодный\nТехник «Навальный LIVE», из которого слепили «организатора» экстремистского сообщества — наравне со мной, Волковым и Ждановым. Я, к сожалению, его почти не знал. Ну, включает парень провода на эфирах.\n\nДаня уже отсидел полтора года в СИЗО. В отличие от меня, на процесс его водили в наручниках. А в обед уводили и приковывали к какой-то стене. На следствии много раз предлагали свободу в обмен на показания на всех. К этим предложениям Холодный остался холоден (не удержался). Холодный бодр, весел, присутствия духа не теряет. А главное — понимает, зачем придуман этот процесс, но не даёт себя запугать и сломить свою волю. Будьте такими же.'),
('https://navalny.com/p/6644/', 'Каждому в России известно, что совершенно беззащитен тот, кто ищет справедливости в суде. Дело такого человека безнадёжно. Ведь если уж дело дошло до суда, то силы за этим человеком нет. Потому что в стране, управляемой преступником, спорные вопросы решаются торгом, властью, подкупом, обманом, предательством и другими механизмами из реальной жизни, а не каким-то там законом.\n\nЧто и было блестяще на днях продемонстрировано, когда те, кого объявили изменниками Родины и предателями, с утра убили на глазах у всей изумлённой России несколько офицеров российской армии, а к обеду договорились о чём-то с кем-то и разошлись по домам, разделить между собой чемоданы с деньгами. Причём чемоданы не метафорические, а самые настоящие. Их даже показали по российскому телевидению.\n\nТем самым закону и справедливости в России в очередной раз указали на их место. И оно совсем не престижное. В суде их точно не найдёшь.\n\nВ общем-то, суд давно превратился просто в площадку, где гражданин только и может, что произнести речь без (и эта фраза повторяется в моем обвинении сотни раз) «согласования с органами государственной власти». Правда, для особо хитрых, злоупотребляющих возможностями судебных прений и последнего слова, придумали сначала закрытый суд, а потом и закрытый суд на территории тюрьмы.\n\nТем не менее нужно использовать любую возможность высказаться, и, выступая сейчас перед аудиторией в восемнадцать человек, семь из которых надели на головы чёрные маски, закрывающие их лица, я хочу не просто объяснить, почему я продолжаю бороться с тем бессовестным злом, которое называет себя «государственная власть РФ», но и призвать вас делать это вместе со мной.\n\nПочему бы и нет? Может, вы и надели эти маски, потому что опасаетесь чего-то человеческого, того, что у вас есть и что может отразиться на вашем не прикрытом балаклавой лице? Например, тюремный надзиратель, который стоит сейчас за мной, по должности должен знать, какие мне предстоят суды. И вот я ему объясняю про ещё одно уголовное дело и предстоящий процесс, про срок новый, который мне грозит. Каждый раз он кивает головой, закрывает глаза и говорит: «Не понимаю я вас и никогда не пойму». Должен же я попробовать ему объяснить.\n\nВопрос, как поступать, — главный вопрос человечества. Ведь всё вокруг так сложно и так непонятно. Люди сбились с ног в поисках формулы правильного поступка. В поисках того, на что можно опереться, принимая решение.\n\nМне очень нравится формулировка нашего соотечественника, доктора филологических наук, профессора Лотмана. Выступая перед студентами, он однажды сказал: «Человек всегда находится в непредвиденной ситуации. И тут у него есть две ноги: совесть и интеллект».\n\nЭто очень мудрая мысль, как мне кажется. И опираться человек должен на обе этих ноги.\n\nОпираться только на совесть — интуитивно правильно. Но абстрактная мораль, не учитывающая человеческую натуру и реальный мир, выродится или в глупость, или в злодеяние, как было уже не раз.\n\nА вот опора на интеллект без совести — именно она сейчас в основе российского государства. Изначально эта идея казалась элитам логичной. Используя нефть, газ и другие ресурсы, мы построим бессовестное, но хитрое, современное, рациональное, безжалостное государство. Мы станем богаче, чем цари прежних лет. И нефти у нас так много, что и населению что-то достанется. Используя мир противоречий и уязвимость демократии, мы станем лидерами, и нас будут уважать. А если нет, то тогда бояться.\n\nНо происходит то же, что и везде. Интеллект, не ограниченный совестью, шепчет: отними, укради. Если ты сильнее, то твои интересы всегда важнее, чем права других.\n\nНе желая опираться на ногу совести, моя Россия сделала несколько больших прыжков, расталкивая всех вокруг, но потом поскользнулась и с грохотом, разрушая всё вокруг, рухнула. И барахтается сейчас в луже то ли грязи, то ли крови, с переломанными костями, с нищим, ограбленным населением, а вокруг лежат десятки тысяч погибших в самой глупой и бессмысленной войне 21 века.\n\nНо рано или поздно, конечно, снова поднимется. И от нас зависит, на что она будет опираться в будущем.\n\nЯ поступаю, как мне кажется, последовательно. Без всякой драмы.\n\nЯ люблю Россию. Мой интеллект говорит мне, что жить в свободной и процветающей стране лучше, чем в коррупционной и нищей. А когда я стою здесь и смотрю на этот суд, моя совесть говорит, что справедливости в таком суде не будет ни для меня, ни для кого-то другого. Страна, где нет справедливого суда, никогда не станет процветающей. Значит — теперь снова говорит интеллект — будет разумно и правильно с моей стороны бороться за независимый суд, честные выборы, быть против коррупции, так как тогда я достигну своей цели и смогу жить в моей свободной, процветающей России.\n\nВам, может, сейчас кажется, что я сумасшедший, а вы все нормальные, — ведь нельзя плыть против течения. А мне кажется, это вы сошли с ума. У вас единственная, Богом данная жизнь, и на что вы решили её потратить? На то, чтобы надеть себе на плечи мантии, а на голову эти чёрные маски и защищать тех, кто вас ещё и грабит? Чтобы помочь тому, у кого десять дворцов, построить одиннадцатый?\n\nДля того, чтобы в мир пришёл новый человек, двое людей должны заранее согласиться с тем, что пойдут на какие-то жертвы. Нового человека придётся рожать в муках, а потом проводить с ним бессонные ночи, а потом заводить с ним собаку. Потом гулять с этой собакой.\n\nИ точно так же для того, чтобы родилась новая, свободная, богатая страна, у неё должны быть родители. Те, кто её хочет. Кто её ждет и кто готов идти на какие-то жертвы ради её рождения. Понимая, что она того стоит. Вовсе не обязательно всем идти в тюрьму. Это скорее лотерея, и я вытянул такой билет. Но принести какую-то жертву, какое-то усилие должен сделать каждый.\n\nЯ обвиняюсь в том, что разжигаю ненависть к представителям власти и спецслужб, судьям и членам партии «Единая Россия». Но нет, я не разжигаю ненависть. Я просто помню, что у человека две ноги: совесть и интеллект. И когда вам надоест поскальзываться с этой властью, расшибая себе лоб и будущее, когда вы наконец поймёте, что отказ от совести в итоге приведёт к исчезновению интеллекта, тогда, может быть, вы встанете на те две ноги, на которых должен стоять человек, и мы вместе сможем приблизить Прекрасную Россию Будущего.'),
('https://navalny.com/p/6642/', 'Привет, это Навальный.\n\nСегодня начинается очередной суд, который сильно добавит мне срок. Но я хочу использовать этот день не для того, чтобы вызвать сочувствие к себе и другим политзекам. Я хочу призвать всех к действию и использовать этот день для объявления о нашем новом, очень важном проекте. Большой агитационной машине. Машине правды. Мы не просто хотим, а обязательно сделаем её, чтобы объединить усилия в борьбе с путинской ложью и кремлёвским лицемерием. Вы очень нужны нам. Присоединяйтесь.\n\n\nПочему сегодня — хороший день для этого объявления? А потому что сам суд доказывает правильность и необходимость такого проекта. Что главное в этом суде? Не беззаконие, не телефонное право, не покорность бессовестных судей и прокуроров. Главное — его форма: это суд внутри тюрьмы. Путин не стесняется сажать невиновных и не боится, что меня могут отбить восставшие толпы в ходе заседания в Москве. Но он боится того, что я скажу. Даже если это будут очевидные, известные всем слова. Он боится слова. Конечно, не только моего, поэтому и Кара-Мурзу, и многих других судили закрытым судом.\n\nПутин боится любого слова правды, ненавидит реплики, превращающиеся в интернет-мемы, бесится от «последних слов», получающих миллионные аудитории. По сути, задача укрепления и продления путинской власти решается через затыкание ртов тем, кто осмеливается говорить правду. Решению этой задачи подчинено почти все, что делается в российской политике в последние годы. Ну а с начала войны власти вообще ни о чём другом не думают. Сроки за посты, за дискредитацию, за «фейкньюс», аресты, блокировки, иноагенты и нежелательные организации. Казалось бы — ну пусть болтают? Когда власть борется с митингами, которые кажутся ей опасными, в этом есть логика, но трёп в интернете или даже по телефону — с ним-то какая проблема?\n\nКажется, что даже наоборот, недовольство уходит в пустоту. Диванные войска ставят лайки вместо того, чтобы строить баррикады. Но на самом деле политика работает не так.\n\nПутин показал себя дураком в военной сфере, бездарным военачальником, но в политике он не дурак. Он знает, что основа любого политического действия — идея и слово. Агитация и убеждение. Это очень хорошо видно во время выборов, особенно там, где они высококонкурентны. Каковы бы ни были особенности государства, политические традиции и вопросы повестки дня, все равно на выборах всё сводится к тому, что агитаторы ходят по домам и квартирам, звонят по телефону, убеждают людей в соцсетях и мессенджерах. И на каждых выборах президента США, со всеми их высокими технологиями и огромными бюджетами, кандидаты сами будут волонтёрить в колл-центрах, чтобы побудить своих сторонников тоже приходить и звонить, разъяснять и убеждать.\n\n\n\nПотому что по силе убеждения ещё ничто не перебило и не перебьёт самый базовый вид агитационного воздействия — просто поговорить, просто привести примеры и аргументы.\n\nЛюди любят заявлять, что на них не влияет предвыборная агитация. Что они и так твёрдо знают, чего хотят, и что их не переубедишь. Но это не так. Большая часть избирателей определяется с выбором на избирательном участке, поэтому хороший агитатор и сомнение зародит, и сагитирует, и переубедит. Это давно доказано. И мы сами проводили эксперименты в этой области.\n\nТак за что же агитировать, когда выборов нет? Есть за что, и ставки очень высоки. Мы будем вести избирательную кампанию против войны. И против Путина. Именно так. Долгую, упорную, изматывающую, но принципиально важную кампанию, где будем настраивать людей против войны.\n\nПротив войны и против всего того, что с ней связано. Против тупика, в который Путин так безумно и тупо свернул 24 февраля 2022 года: смерти, потери, мобилизация, военные преступления, изоляция, санкции, десятки тысяч погибших и миллионы уехавших. Деградация экономики и падение уровня жизни, воюющие уголовники и нищие мобилизованные, двухсотые и трёхсотые.\n\nЭто очень конкретная задача, и наша работа будет успешной, я не сомневаюсь. Вот она —главная таблица и главная цифра одного из наших опросов:\n\n\nУ каждого пятого есть родные или знакомые, погибшие на этой войне. Эти цифры, увы, будут и дальше только расти, меняя общественное сознание. Десятки тысяч раненых и инвалидов. Сотни тысяч мобилизованных, своими глазами видевших путинскую войну как она есть: от бездарных воров-генералов до отсутствия всего — от носков до снарядов. Они возвращаются домой, их истории слушают и пересказывают. Это совсем не означает, что эти люди автоматически становятся антивоенными активистами. Но это точно означает, что с нашей помощью они могут стать такими. У нас есть повод обсудить с ними важные вопросы, и многие не откажутся поговорить.\n\nМногих мы переубедим. Почти во всех зародим сомнения. Это — избирательная кампания против кандидатов «Война» и Путин. И делать её мы с вами будем по законам и технологиям хороших избирательных кампаний. Опрашивая всех, таргетируя сотни разных групп, подбирая подход к каждой, выявляя колеблющихся и переубеждая их.\n\nЯ сильно сомневаюсь в тех огромных цифрах «поддержки войны», которые дают кремлёвские социологи. В первую очередь потому, что неизвестно, что это означает — «поддержка СВО». Вот я прошу присылать мне всё, что пишут и говорят Стрелков и Пригожин, очень вдумчиво читаю. За войну они? Ну ясен пень. Но при всей их взаимной нелюбви я ни у кого не нахожу более ясных антипутинских и антикремлёвских высказываний. Да и, честно говоря, уже и близких к антивоенным. Интервью Пригожина смотрели? Ну оно же при всем своем людоедстве — антивоенное. Повар там прямым текстом говорит, что всё, война проиграна. Сценарий победы — по его выражению, «оптимальный» сценарий, при котором мы заберём себе то, что уже хапнули, — вряд ли возможен. Элита всё украла, а её дети за границей. Генералы — тупые воры. Оружие наше плохое, снарядов нет. Вообще-то в таком стиле всегда говорил ФБК, но сейчас это говорит главный сторонник войны и один из главных её командиров.\n\nИ наша задача в том, что, когда всё это нам повторит избиратель, спросить у него вкрадчиво: а может, и на хрен её, войну эту? Чего полезли-то? Да, многим не нравится не война, а проигранная война. Или бессмысленная война. Ок, любая антивоенная кампания (см. Вьетнам, см. Ирак) опирается и на это тоже.\n\n\nВот я здесь своих ментов агитирую, как могу. Они, конечно, говорят, что за войну. Ещё бы, разговор записывается на видеорегистратор. Им бесполезно говорить про военные преступления, про Бучу, про агрессию, про санкции. Им до фонаря, кто там страдает. Но на вопрос: «Но снаряды где? Куда ваш Путин, сидящий 23 года у власти, с кучей денег прое…л снаряды, носки, бронежилеты и квадрокоптеры?» — у них ответа нет. «Это же не я про снаряды придумал. Это ваш Пригожин, перед которым вы тут на цырлах ходили, когда он приезжал зеков на войну вербовать. И если ваша власть так всё развалила, что нет ни разведки, ни командиров, ни границы, ни ПВО, ни снарядов, ни носков, — на хрена вы на войну попёрлись-то? Чтоб миллион человек в землю положить?» Они не качают меня на руках после таких разговоров, но они задумываются, и они сомневаются.\n\nИ мы к каждому будем искать свой подход, не пытаясь говорить с программистом из Москвы, молодой матерью из Орла и военным пенсионером из Челябинска одним языком. Это — избирательная кампания против кандидата «Война», и вести её успешно можно, только опираясь на армию из десятков, а лучше сотен тысяч убеждённых, старательных, трудолюбивых людей, верящих в успех. Тех, кто не выгорает через каждые пять минут работы, не падает в обморок от того, что его послал на три буквы собеседник, не демотивируется от встречи со среднестатистическим избирателем и не ждёт, что тот будет логичен, разумен, образован, вежлив и сразу переубедится. Это умная, тонкая и упорная работа вдолгую, и я призываю присоединиться к нам тех, кто хочет реального дела и реального вклада, а не бесконечного нытья в Фейсбуке и Твиттере, где мы пытаемся переубедить самих себя.\n\nМы уже против войны, нас уже несколько миллионов, мы уже научились сами организовывать и финансировать свои действия. Если, давайте пофантазируем, из полутора миллионов уехавших с начала войны и мобилизации, полутора миллионов уехавших после 2014 года и миллиона оставшихся в России, но не боящихся, каждый десятый примкнет к агитации против кандидата «Война», то эта армия из четырёхсот тысяч агитаторов в месяц может охватывать 12 миллионов граждан, даже если один агитатор делает всего лишь один контакт в день, то есть вообще не перенапрягается. Агитационная машина такой силы кардинально изменит общественные настроения в стране за 3–4 месяца.\n\nТеперь фантазировать перестанем. Потому что на практике это маловероятно. Людям лень, у них свои дела. Самые крикливые, требовавшие «настоящего дела», испаряются в первую очередь. Бездельники, как всегда, найдут себе оправдания в «разве это дело, вот эшелоны под откос пускать я бы пошёл, а это фигня». И сосредоточатся на критике, не пустив под откос ни одного эшелона. Ну и так далее. В общем, всё то, что и во время любой избирательной кампании происходит. Тем не менее мы точно понимаем, что есть десятки тысяч людей, готовых посвятить добросовестной и упорной работе на общее благо хотя бы один час в день. Это колоссальная сила. Будет очень непросто организовать такую машину агитации — одну из крупнейших в мире. Но глаза боятся, а руки делают. Уверен, что мы можем поставить себе первой задачей охватить агитацией против войны и Путина 10 миллионов избирателей. Уже это гарантирует заметный сдвиг общественного мнения. Как это отразится на политической ситуации, предсказать не сможет никто. Но работа наша точно не будет напрасной.\n\nПерейдём к конкретике. Что нам доступно из инструментов убеждения внутри России? Митинги/пикеты — нет. Поквартирные обходы — нет. Звонки со своего телефона, если звонящий внутри России, — нет. Колл-центры внутри России — нет. Как видим, основной арсенал традиционных избирательных кампаний нам недоступен. Трезво признаем это.\n\nОднако есть новые возможности, новые технологии. Оффшорные колл-центры, децентрализованные колл-центры. Мессенджеры — агитация через них может быть потрясающе эффективна с учётом того, что Вотсапп и Телеграм есть уже у любой бабуси. Агитация в подконтрольных Кремлю соцсетях тоже возможна, если правильно избегать рисков. Таким образом, грубое описание той агитационной машины, которую мы будем строить, такое: это система, позволяющая вам (агитаторам) включиться в неё в удобное время из любого места и, сохраняя при желании анонимность, связаться голосом или текстом с избирателем внутри РФ, соответствующим нужным параметрам — пол, возраст, город, род занятий и так далее. Система учит вас агитировать, опираясь на предыдущий опыт, подсказывая схему разговора, факты и фразы. В каком-то смысле это похоже на создание и обучение искусственного интеллекта. Мы должны создать и обучить систему коллективного интеллекта, убеждающего избирателей выступать и против ненавистных нам кандидатов — «Война» и Путин.\n\n«Ничего себе», — скажете вы.\n\nНу да, это амбициозная задача. Но, в общем, ничего неосуществимого и ранее не виданного. Маркетологи, рекламщики и политтехнологи делают подобное десятилетиями. Все эти холодные звонки, тёплые контакты и воронки продаж хорошо всем известны. Просто чаще всего маркетологов за такое в тюрьму не сажают. Наша же деятельность, безусловно, будет объявлена незаконной и подрывной. Все силы государственного аппарата будут брошены на борьбу с ней. Ну и отлично, а мы бросим все силы на борьбу с аппаратом войны, коррупции и тупости.\n\nПредстоит большая техническая работа. Ничего подобного пока не существует. Система должна быть очень гибкой и обладать качествами, которые кажутся взаимоисключающими. Это должна быть удобная в работе база контактов, но сделанная так, чтобы она в принципе не могла утечь и стать источником проблем для людей. Любой желающий должен иметь возможность быстро включиться в работу, но провокаторов, жуликов, глупых, вспыльчивых и так далее надо отсекать как можно быстрее. Придётся создавать большое количество одноразовых аккаунтов, но это не должно превращаться в машину спама. Агитмашина должна уметь мгновенно адаптироваться к блокировкам и любым противодействиям, быть максимально креативной. Какие-то элементы подобной штуки мы с коллегами делали или пытались сделать ещё с 2012 года — старожилы помнят проект ДМП, о котором я объявил на одном из митингов.\n\nОднако масштаб проекта таков, что всегда не хватало времени, знаний, денег, кадров. Думаю, что это одна из самых больших моих политических ошибок: я не сделал ДМП приоритетом, и мы не построили её после выборов 2013 года, постоянно отвлекаясь на другие дела. А сейчас у нас просто выбора нет. Ни политического — что может быть важнее, чем остановить войну и власть, живущую войной; ни организационного — сотни тысяч самых активных и грамотных вытеснены за границу. Они готовы что-то делать, но что? Мы тысячи сообщений получаем: «Ребята, дайте нам какую-то работу, полезную, которую можно делать из-за границы или в России, но без лишнего риска».\n\nИтак. Мы начинаем придумывать, мы начинаем строить, мы начинаем нанимать, мы начинаем собирать деньги. Вы нам очень нужны. В первую очередь — те, кто понимает техническую, логистическую и организационную сторону того, что я описал. Мы собираем мнения, экспертизы и идеи. В скором времени организуем хакатоны в разных городах. И, конечно, нам нужны самые стойкие и самые терпеливые, самые понимающие, те, кто станет сердцем и сутью этой системы. Технически строится оболочка, но наполняется она людьми.\n\nДля успешной агитации к моменту создания и запуска полноценной машины мы должны провести тысячи часов разговоров. Отслушать их и проанализировать. Определить параметры микротаргетинга. Создать, попробовать, изменить и улучшить сотни сценариев разговоров для разных целевых групп.\n\nМы ищем 100 первых первопроходцев — волонтёров-агитаторов, готовых заниматься этим потрясающим, но непростым делом. Особенно в неизбежном хаосе и бардаке первых шагов.\n\nНапишите на почту antiwar@navalny.com, если вы:\n\n— IТ-специалист и готовы вложить много времени в работу над созданием технологических решений для нашей агитсистемы;\n\n— маркетолог, социолог, политолог и готовы вложить много времени в работу над созданием скриптов разговоров, воронки вовлечения и т. д.;\n\n— сторонник, готовый адресно поддержать существенными деньгами именно этот конкретный агитационный проект;\n\n— волонтер и хотите быть в первой сотне тех, кто вложит много времени в сами разговоры, в отработку сценариев и поиск слов и подходов, отбирающих избирателей у кандидата Путина и кандидата «Война».\n\nНапишите достаточно подробно о себе, откуда вы и где сейчас живёте, сколько у вас есть времени на эту работу. Мы вскоре с вами свяжемся.\n\nЭто — долгий проект. Военное поражение Путина неизбежно. Но как оно будет выглядеть и каковы будут его последствия, никто не знает. Те, кто составляет собой верхушку власти, готовую устроить войну ради денег и укрепления своих позиций, никуда не денутся. На луну они не улетят. Их ответом на проигранную войну будет истерика и подготовка к новой войне. Именно этим они будут промывать головы гражданам. Никто, кроме нас, в эту борьбу за сердца и умы сограждан не вступит. Значит, надо вступить и победить.'),
('https://navalny.com/p/6638/', 'Я давно хотел это написать, но запрещал себе, потому что вроде как устраиваешь скандал. Но после того, как побывал, хоть и через видеоконференцию, свидетелем на суде у Лилии Чанышевой и увидел ее, такую потрясающую и не сломленную, но и одинокую на этом ужасном закрытом суде в Уфе (закончится все, конечно, огромным сроком), не могу не написать.\n\n\nБуду чувствовать, что своим молчанием предаю ее.\n\n\nЛилия Чанышева была нашим координатором в Уфе. Одним из лучших координаторов, но мурыжат ее сейчас по тюрьмам и СИЗО совсем не на уровне координатора, хотя их всех серьезно преследуют. Она проходит прямо как один из лидеров экстремистского сообщества. Арестовали ее раньше всех, долго держали в СИЗО в Москве, а следователь мне так и говорит: «У нас установка, что Чанышева — это Навальный в юбке».\n\nМы знаем, что так вышло потому, что бандитская мафия в руководстве Башкортостана так бесилась из-за эффективных действий Чанышевой, в частности, по защите шихана Куштау и расследованию коррупции семьи губернатора Хабирова, что они отдельно лоббировали ее посадку на самом высоком уровне. Этим занимался лично Хабиров, а идеологическую базу под посадку готовил такой омерзительный жулик-пиарщик по фамилии Мурзагулов.\n\n\nПри каждом губернаторе, да и при каждом органе власти есть такая вот прикормленная сволочь.\n\n\nХабиров и Мурзагулов\nПишет статьи — публичные доносы, превращающиеся потом в уголовные дела. Обычно я и имен-то их не знаю, но этого Мурзагулова запомнил за то, как он всегда как ручной пудель мотался за Радием Хабировым:\n\n\nИ, конечно, за то, как он особенно изгалялся (1, 2) в травле Чанышевой и нашего башкирского штаба.\n\nИ вот этого Мурзагулова взял на работу Михаил Борисович Ходорковский.\n\n\nКогда мне об этом сказали, я не поверил. Как же так можно сделать? Это ведь не какой-то путинский жулик из начала нулевых. Вот же он прямо недавно посадил честного человека, которого судят прямо сейчас.\n\nНо потом мне прислали и обращение мужа Чанышевой к Ходорковскому, и другие доказательства, а главное, отвратительные кривляющиеся (иначе и назвать не могу) комментарии самого Ходорковского на эту тему. Мол, нужно уметь работать с людьми с той стороны, да и вообще сейчас главный и единственный критерий — война, и если некто против войны, то совсем неважно, что он делал буквально вчера, — он все равно на нашей стороне.\n\n\nhttps://twitter.com/mich261213...\n\n\nhttps://twitter.com/mich261213...\nЯ хочу сказать, что мне очень стыдно за Михаила Ходорковского. За то, как он себя ведет в этой ситуации. За то, как лицемерит. За то, как он оскорбил Чанышеву и в ее лице всю нашу структуру. За то, что он кормит мерзавца Мурзагулова и навязывает его всем в качестве оппозиционера и противника режима.\n\nАргумент про «нужно работать с людьми с той стороны» я уже слышал. Причем от Ходорковского же, когда он в 2016 году взял курировать избирательные компании каких-то жуликов с госканалов. Тимур Валеев, Мария Баронова, помните таких? Все провалили, получили денег и ушли обратно на ту сторону.\n\nА вот про довод «сейчас важна только война и отношение к ней» — это и есть то самое отвратительное кривляние. Успешное упражнение «как перелицемерить самого Путина». Как для Путина «война все спишет», так и здесь война все спишет и отмоет добела каждого негодяя, если он напишет пост в Фейсбуке. Что бы ни говорил Мурзагулов и такие, как он, они не просто не против войны, они устроили эту войну, они всеми силами создавали условия для нее. Потому что прологом к этой войне была зачистка любых активистов, в первую очередь, наших штабов как единственной структуры, которая могла организовать антивоенные митинги по всей стране.\n\nЧанышева сидит потому, что Хабиров и Мурзагулов успешно доказали Кремлю, что она опасна, и Кремль понял, что оставлять ее на свободе во время войны нельзя.\n\nОтдельно скажу для тех идиотов, которые обязательно придут сюда и скажут: «В кризисной ситуации нельзя стоять в белом пальто, нужно быть прагматиком и объединяться со всеми».\n\nОкей. Для вас я надеваю мерзкое, заляпанное, драное, рациональное пальто политического прагматизма. Реалполитик-пальто.\n\nНу и что. Я стою в этом пальто рядом с Ходорковским. Мне очень противно. А какие прагматические выгоды мы имеем? Что добавил нашему движению Мурзагулов? Какие выгоды мы приобрели? Увеличилась ли наша поддержка? Пафоса про людей с той стороны так много, как будто Соловьева с Симоньян перевербовали, но в приобретениях пока числится только самая мелкая, дешевая и никому не известная продажная шушера.\n\nЗато мы нанесли удар по морали активистов. Зато теперь любое взаимодействие с Ходорковским — это недоумение всех региональных активистов и обвинения в предательстве и лицемерии. Это недоумение огромного количества отличных журналистов, оставшихся без работы, тому, что высокооплачиваемую работу у единственного оппозиционера с деньгами получил не приличный человек, а Мурзагулов. Поэтому реалполитик-пальто в этой ситуации я точно снимаю, стоять в нем противно, а выгод нет. И мне стыдно за Михаила Ходорковского. Надеюсь, и вам тоже. И надеюсь, когда ему прямо скажут это многие, он что-то поймет, и ему тоже станет стыдно.'),
('https://navalny.com/p/6634/', 'В преддверии годовщины полномасштабного и неспровоцированного вторжения российских войск в Украину я максимально кратко сформулировал политическую платформу свою и, надеюсь, еще очень многих нормальных людей.\n\n15 пунктов гражданина России, желающего блага своей стране.\n\nЧто это было и что есть сейчас?\n1. Президент Путин развязал несправедливую агрессивную войну против Украины под нелепыми предлогами. Он отчаянно пытается придать этой войне статус «народной», стремясь сделать своими сообщниками всех граждан России, однако его попытки терпят крах. Добровольцев для этой войны почти нет, поэтому путинская армия опирается на заключенных и принудительно мобилизованных.\n\n2. Реальные причины войны — политические и экономические проблемы внутри России, стремление Путина удержать власть любой ценой, а также его одержимость своим историческим наследием. Он хочет войти в историю как «царь-завоеватель» и «собиратель земель».\n\n3. Убиты десятки тысяч невиновных украинцев, боль и страдания обрушены на миллионы. Совершены военные преступления. Разрушены города и инфраструктура Украины.\n\n4. Россия терпит военное поражение. Именно осознание этого изменило риторику властей от «Киев за три дня» до истерических угроз применить ядерное оружие в случае проигрыша. Бессмысленно загублены жизни десятков тысяч российских солдат. Окончательное военное поражение можно отсрочить ценой жизни еще сотен тысяч мобилизованных, но в целом оно неизбежно. Комбинация «агрессивная война + коррупция + бездарность генералов + слабая экономика + героизм и высокая мотивация обороняющихся» результатом выдает только поражение.\n\nЛживые и лицемерные призывы Кремля к переговорам и прекращению огня — не что иное, как реалистичная оценка перспектив военных действий.\n\nЧто делать?\n5. Какие границы у Украины? Такие же, как и у России, — международно признанные, определенные в 1991 году. Мы, Россия, их тоже тогда признали. Эти границы Россия должна признавать и сейчас. Тут нечего обсуждать. Почти все границы в мире случайны и вызывают чье-то недовольство. Но воевать за их изменение в XXI веке нельзя. Иначе мир погрузится в хаос.\n\n6. Оставить Украину в покое и дать ей возможность развиваться так, как желает ее народ. Прекратить агрессию, завершить войну и вывести все российские войска с территории Украины. Продолжать войну — это как раз истерика бессилия, а прекратить ее — сильный ход.\n\n7. Совместно с Украиной, США, Евросоюзом и Британией искать приемлемые пути компенсации ущерба, нанесенного Украине. Например (после смены власти в России и окончания войны), отменив ограничения в отношении наших нефти и газа, но с направлением части дохода от экспорта углеводородов на компенсации.\n\n8. Расследовать военные преступления в сотрудничестве с международными институтами.\n\nПочему прекращение путинской агрессии — в интересах России?\n9. Всем русским присуще имперское сознание?\n\nЭто ерунда. Например, в войне против Украины участвует Беларусь. У белорусов тоже имперское сознание? Нет, у них тоже диктатор у власти. В России, как и в любой стране с историческими предпосылками к этому, всегда будут люди с имперскими взглядами, но это далеко не большинство. Тут нет причин рыдать и причитать. Таких людей надо побеждать на выборах, как побеждают правых и левых радикалов в развитых странах.\n\n10. Нужны ли России новые земли?\n\nРоссия — огромная страна с сокращающимся населением и вымирающей провинцией. Имперскость и тяга к захвату территорий — самый вредный и губительный путь. Российская власть вновь своими руками уничтожает наше будущее ради того, чтобы страна выглядела больше на карте. Но Россия и так большая. Наша задача — сберегать народ и развивать то, чего у нас в избытке.\n\n11. В наследство от этой войны нам достанется клубок сложных и, на первый взгляд, почти нерешаемых проблем. Важно определить для самих себя, что мы действительно хотим их решить, после чего начать делать это честно и открыто. Ключ к успеху — в понимании того, что для России и ее народа будет не просто хорошо, но еще и очень выгодно прекратить войну как можно скорее: только так можно начать движение к снятию санкций, возвращению уехавших, восстановлению доверия бизнеса, экономическому росту.\n\n12. Еще раз подчеркну, что после войны нам предстоит компенсировать Украине ущерб, нанесенный путинской агрессией. Но восстановление нормальных экономических отношений с цивилизованным миром и возвращение экономического роста позволит это сделать, не мешая развиваться нашей стране.\n\nМы на дне, и, чтобы всплыть, нам нужно оттолкнуться от него. Это будет и морально верно, и рационально, и выгодно.\n\n13. Демонтаж режима Путина и его диктатуры. В идеале — через всеобщие свободные выборы и созыв Конституционного собрания.\n\n14. Установление парламентской республики, основанной на сменяемости власти через честные выборы, независимом суде, федерализме, местном самоуправлении, полной экономической свободе и социальной справедливости.\n\n15. Осознавая свои историю и традиции, мы должны быть частью Европы и следовать европейскому пути развития. Другого у нас нет, и другого нам и не нужно.'),
('https://navalny.com/p/6624/', 'Когда я получил предложение написать несколько приветственных слов для конференции о демократии и технологиях, то подумал, что сделаю это, потому что я правильный человек для такого. Информационные технологии играют большую роль в моей жизни. В положении о моей тюрьме так и записано: «Исправление заключенных через постоянный видеоконтроль и информационное воздействие».\n\nТак что я — житель небольшой информационной антиутопии с камерами и электронными замками на каждом шагу. Ну только с поправкой на то, что это Россия, поэтому электронные замки, конечно, сломались, и двери продолжают открывать огромными металлическими ключами.\n\nЯ приветствую организаторов и участников Copenhagen Democracy Summit. Вам явно есть, что обсудить.\n\nАбсолютная банальность — сказать, что новый информационный мир может быть как благом для демократии, так и огромным злом. Тем не менее это так. Всю свою деятельность наша организация построила на информационных технологиях и добивалась этим серьезных успехов, даже когда находилась фактически на нелегальном положении. И информационные же технологии активно используются сейчас Кремлем для арестов участников протестных митингов. Гордо заявляется, что все они будут распознаны даже со скрытыми лицами.\n\nИнтернет дает нам возможность обходить цензуру. И — одновременно — Google и Meta, отключившие рекламу в России, лишили оппозицию возможности вести антивоенные кампании, сделав грандиозный подарок Путину.\n\nУ всех нас много вопросов. Должны ли интернет-гиганты и дальше делать вид, что они «просто бизнес» и «нейтральные площадки». Можно ли и дальше заявлять, что пользователи соцсетей в США и Эритрее, в Дании и России должны действовать по одним правилам? Как интернет должен относиться к указаниям государств с учетом того, что Норвегия и Уганда имеют, кажется, несколько различное представление о роли интернета и демократии?\n\nВсе это очень сложно, очень противоречиво, и все это надо обсуждать. Не забывая о том, что за обсуждением надо вырабатывать и решения.\n\nМы любим технологии. Мы обожаем соцсети. Мы хотим жить в свободном информационном обществе. Так давайте придумаем, как не дать плохим парням использовать информационное общество, чтобы загонять свои народы и всех нас в темные века'),
('https://navalny.com/p/6623/', 'Спасибо большое. Уважаемый суд, вы будете меня перебивать? Я хотел бы уточнить, потому что обычно во время последнего слова меня перебивают. Я вас не слышу или вы просто молчите?\n\nСпасибо большое. Я, честно говоря, на прошлом суде уже даже говорил, что я уже не хочу выступать с последними словами, мне уже вот здесь эти последние слова. Знаете, я так примерно прикинул: 17 января я вернулся в Москву, меня сразу арестовали, и с тех пор я выступаю. Это мое шестое или седьмое последнее слово, что превращается в какую-то немножко комедию. Хотя это, конечно, очень хорошо характеризует происходящее в нашей стране, но тем не менее это смешно, когда человек за полтора года выступает с седьмым последним словом. Однако во время моего последнего «последнего слова» произошел для меня очень воодушевляющий факт, который говорит о том, что надо все-таки их произносить.\n\nКак было устроено наше судебное заседание? Здесь был актовый зал, в моей тюряге, точно так же тут сидела судья, защита, прокурор и технический специалист суда. И штука в том, хитрость, понимаете, — вы же все очень хитрые, и ваша власть очень хитрая и находчивая. В тот день, когда я произносил последнее слово, технического специалиста суда не было. А вот наш майор, который здесь занимается всякой технической фигней, — его переодели в гражданку и посадили на место технического секретаря суда. И что он делал? Когда я выступал с последним словом, каждый раз, когда я говорил слово «война», он нажимал на кнопку, и я это видел, потому что на экране передо мной загорался просто перечеркнутый красный микрофон, говорящий о том, что микрофон был отключен. И он сидел такой довольный, и судья была довольна, и вся ваша система была страшно довольна тем, что у вас есть кнопка, и вы можете на нее нажать, и публика, журналисты, кто угодно не услышат, что же такое я скажу после слова «война».\n\nКонечно, это меня бесило страшно. Ты планируешь, говоришь какие-то важные вещи, тут бах, на кнопку нажимают с такой ухмылочкой — и тебя не слышно. Конечно, бесило. Но, с другой стороны, в чем был воодушевляющий момент? Я подумал и потом понял. Потому что у вас есть все: власть, вы захватили власть в стране, у вас телевидение, все эти продажные журналисты, — все у вас есть, и вы боитесь, что вам какой-то зэк что-то скажет. Настолько боитесь, что приставили специального сотрудника, переодетого в гражданку, чтобы он нажимал на эту самую кнопку. Вы так сильно боитесь этих слов, вообще слова правды, что, конечно, его нужно произносить. К чему я постоянно призываю всех остальных, ну и сам по мере возможности что-то делаю.\n\nОговорка относительно вашего уважаемого суда. Я лично против вас ничего не имею. Я с вами не знаком. Вполне вероятно, что вы хорошие люди, прекрасно относитесь к детям и, скорее всего, даже получали пятерки в институтах, особенно по уголовному процессу. Но когда я говорю «вы», я обращаюсь к суду, конечно же, сейчас, но я имею в виду всю вашу власть: имею в виду Путина, его Совет Безопасности из старых маразматиков, Государственную Думу, где сидят одни жулики, Совет Федерации, где сидят одни старые жулики, всю вашу власть. Вы же выносите мне приговор именем Российской Федерации, поэтому, когда я к вам обращаюсь, я обращаюсь к той самой Российской Федерации, захваченной сейчас теми самыми жуликами, мошенниками и, конечно, убийцами, в нынешнее, настоящее время.\n\nМой приговор предполагает, что я оскорбил суд и я не уважал суд. Знаете, вы сами отлично понимаете, что никто в России не уважает суд. Это в принципе такой орган, который не имеет ни малейшего уважения нигде. Его действительно боятся, потому что какие-то люди в черных мантиях абсолютно беззаконно, как происходит со мной и с многими другими людьми, постоянно могут, действительно, отправить тебя за решетку, поэтому их боятся. Но не уважает российский суд никто, и я его действительно не уважаю.\n\nНо вот эта статья об оскорблении суда — это не научный подход. Если была бы статья о презрении к суду и презрении к этой власти, вот она бы мне отлично подошла, потому что я презираю ваш суд, я презираю вашу систему и презираю вашу власть. И я, самое главное, — что на самом деле, как мне кажется, и является преступлением по сути в ваших глазах, — я эту систему не боюсь. Я не боюсь эту систему. Безусловно, мне не хочется сидеть в этой клетке, вместо того чтобы заниматься какими-то полезными вещами и смотреть, как растут мои дети. Но человеку жизнь дается не для того, чтобы он боялся чокнутого бункерного деда и эту систему, которую он построил. Поэтому я вас не боюсь! И, опять же, всех остальных призываю не бояться. В конце концов, ну что с вами сделают? Ну в клетку посадят. Но ваша система держится именно на страхе, поэтому вас бояться нельзя, бояться вас — это преступление против своего собственного будущего, преступление против своих детей, преступление против своего народа.\n\nПоэтому важно сейчас... Потому что я включаю телевизор — мне показывают, как кто-то там убивает русских, притесняет русских или еще что-то делает с русскими. А именно вы, ваша система, ваш Путин и ваши чиновники являются врагами России сейчас, предателями русского народа и убийцами русского народа в буквальном смысле. За последние годы никто не убил больше русских людей, чем ваш Путин.\n\n— Алексей Анатольевич, прошу вернуться к предмету судебного контроля. У нас обжалуется итоговое судебное решение Лефортовского районного суда.\n\n— Я ровно это и делал. Я поэтому и уточнял: вы, пожалуйста, не станьте той самой кнопкой, которая пытается меня заткнуть. УПК не регулирует, что я говорю в последнем слове. Я говорю про суд, я говорю про приговор, я говорю про основания, которые послужили тому, что мне был вынесен — и не только мне, таким, как я, выносятся эти абсурдные приговоры.\n\nЕсли хотите ближе к делу — давайте, хорошо. Вам не нравится, когда я говорю про войну, — тогда я буду использовать эту войну, идущую сейчас, она важнее в сто раз, чем мой приговор, как метафору для ваших судов и для ваших судебных процессов. Потому что эта война, как и ваши суды, полностью, на сто процентов построена на вранье, на каком-то супернаглом вранье. Ни одного слова правды, ни одного. Как в приговоре нет ни одного слова правды, так и война эта идет. Я же телевизор целыми днями здесь смотрю, это, на самом деле, важная часть наказания — ты должен все время смотреть телевизор. И несколько месяцев назад, три месяца подряд, в каждом репортаже, в каждом сюжете, в каждом репортаже, в каждом выпуске новостей говорили: какая ерунда, что мы введем войска на Украину! Да не может такого быть! Это же американцы врут.\n\n— Алексей Анатольевич, у нас предмет судебного контроля — приговор Лефортовского районного суда города Москвы от 22 марта 2022 года. Слушаем ваше несогласие с вынесенным итоговым судебным актом Лефортовского районного суда Москвы.\n\n— Именно, вот вы просто уже, понимаете… Все-таки выступление в суде. Посмотрите, у вас висит там дяденька, если вы посмотрите по правую руку на стене. Я не могу разглядеть, кто это. Наверное, уважаемый юрист. Насколько я помню, потому что я тогда сидел в «аквариуме», по левую сторону тоже висят какие-то известные люди, и в основном они прославились тем, что не просто бормотали в микрофон, а говорили какие-то важные вещи, выступали с речами в суде. И я сейчас делаю ровно это. Я выступаю с речью в вашем суде, и я, действительно, использую войну как метафору своего судебного приговора. Несмотря на то, что я использую что-то большое для описания чего-то маленького, а можно наоборот. Поэтому я прошу, несмотря на то что вы страшно боитесь, что я тут что-нибудь наговорю, все-таки не работать той самой кнопкой, потому что кто вам поручил это делать — к нему тогда предъявляйте претензии за то, что я здесь говорю. Поэтому вы можете просто меня остановить и выключить, но я собираюсь… я скажу все, что я собираюсь сказать.\n\nТак вот. Месяцы вранья, просто месяцы. А потом щелчок, ввели войска — и просто забыли об этом, понимаете? Память золотой рыбки. И ваша власть пытается сделать так, чтобы все наше население, весь наш народ были как золотые рыбки, в том числе в отношении судебных процессов, в том числе в отношении законов, которые вы меняете каждый день, в том числе чтобы забыли о том, что был закон, который запрещал мне участвовать в выборах, закон имени Навального. Это все абсолютнейшая ложь. Все факты, которые приведены в приговоре, такая же ложная дрянь, как и все те факты, которые используют разжигатели войны, или наоборот. Я, когда слышу по телевизору про НАТО, мне уже просто хочется хохотать, потому что я не золотая рыбка, я помню, как в 2013 году в городе Ульяновске открывали базу НАТО и ульяновский губернатор рассказывал, как хорошо, что мы открываем здесь базу НАТО для работы в Афганистане, потому что это так классно, потому что это будут рабочие места, НАТО нам очень дружеский блок. А знаете, кто еще очень сильно выступал за базу НАТО? Рогозин Дмитрий Олегович. Знаком вам такой человек? А сейчас эти люди пытаются нам доказать, что мы здесь должны устроить кровавую баню, для того чтобы противостоять НАТО. Одновременно с этим в НАТО вступают Финляндия и Швеция, и это уже какая-то ерунда. И я говорю об этом, потому что это, конечно, очень определяюще для всего, что происходит в России, в том числе для российской судебной системы. Потому что наглое вранье в каждом суде. Игнорирование того, что было сказано вчера, просто по щелчку, как будто и не говорили. Вы все переобуваетесь в воздухе на 180 градусов не просто часто, а постоянно. Наша власть, наши суды, наш Путин делают зигзагообразное движение в воздухе, потому что каждый раз ты должен переобуться, чтобы люди забыли о том, что ты врал еще совсем недавно.\n\n— Алексей Анатольевич, у нас предмет судебного контроля — это приговор Лефортовского суда города Москвы. Мы слушаем в последнем слове ваше суждение касаемо вашего несогласия с вашим осуждением, приговором Лефортовского районного суда города Москвы. Пожалуйста, вернитесь к предмету судебного контроля в апелляционном порядке.\n\n— Во-первых, предмет нашего контроля — это приговор, действительно, Лефортовского суда и причины, по которым такой приговор был вынесен. И причины, почему ваш суд происходит или нет, вы отлично знаете. И они заключаются в том, что у власти сейчас в России сумасшедшие богатые люди, которые в том числе устроили войну, в том числе для того, чтобы проводить судебные процессы в тюрьмах, потому что на фоне войны на это уже никто не посмотрит. Поэтому, уважаемый суд, у вашей профессии есть плюсы — неприкосновенность и неприкасаемость, большая зарплата, квартира в Москве. Но, наверное, в вашей работе есть и какие-то минусы. В частности, если лотерея выдала вам такой несчастливый билет, как огласить мой приговор, вам придется меня послушать, извините, пожалуйста. Если вы не хотите меня слушать, вы можете просто меня выключить и тем самым сделать еще одну процессуальную ошибку. Еще немножко, потерпите, пожалуйста.\n\nЯ хочу сказать, что эти ваши процессы, в том числе в отношении меня, — они бессмысленны. Что вы ими хотите добиться? Вы хотите получить какой-то контроль? Вы получите, действительно, кратковременный контроль. Вы хотите остановить прогресс России, вы хотите бороться с наступающим поколением? Что вы хотите сделать? Краткосрочно вы можете запугать кого-то, безусловно, вы разорите сейчас какое-то количество людей, поломаете много судеб, но в целом то, что вы делаете, что делают ваши люди, — это бессмыслица, это просто исторический нонсенс, и, безусловно, вы все потерпите историческое поражение. Как вы потерпите историческое поражение вот в этой тупой войне, которую вы начали, которую начал ваш Путин, потому что у нее нет ни цели, ни смысла.\n\nВообще непонятно, зачем вы ее ведете. Для чего мы ведем эту войну? Чтобы заставить их сделать что? Мы взяли просто сорокамиллионный народ, объявили: «Они нацисты!» — и начали их бомбить. Что они должны сделать, чтобы от них отстали? Вот представьте себе ситуацию: живет в городе Харькове человек, скажем даже, судья, судья в городе Харькове — миллионный город, огромный город. Он живет себе в большом городе, прекрасно работает судьей, собирается с утра отвести ребенка в детский сад, а потом поехать кого-то судить. И в один прекрасный день, 24 февраля, например, его объявляют нацистом, ему в дом прилетает ракета, убивает его ребенка, и он бегает, как ненормальный, и не понимает, что ему нужно сделать, чтобы от него отстали вы и ваша власть. И я, безусловно, в том числе готов сидеть здесь в тюрьме, чтобы доказать всему миру и самому себе, самое главное, что не все в России такие чокнутые, ненормальные, извращенцы и кровопийцы, которые просто непонятно за что, непонятно для чего убивают людей и сами гибнут. Вы знаете, что здесь, во Владимирской области...\n\n— Алексей Анатольевич, мы вынуждены вас вернуть все-таки к предмету судебного контроля и напомнить, что мы слушаем ваши доводы и ваше несогласие с тем приговором суда, который восстановлен 22 марта 2022 года. Вы осуждены и признаны виновным в совершении четырех составов преступления, предусмотренного частью 4 статьи 159 УК РФ и статьей 297 УК РФ. Вернитесь, пожалуйста, к тем суждениям, в чем именно вы не согласны с приговором суда.\n\n— Уважаемый суд, большое спасибо за это мудрое напоминание. Понимаете, я же обычный человек. Я не судья, у которого все очень логично и правильно настроено. Я говорил о том, что я хочу сказать, о своем приговоре, в меру своей способности к суждению. И моя способность к суждению — она и определяет то, что я говорю. Как могу, так и говорю. Знаете, «не стреляйте в пианиста, он играет как может». Так и здесь. Для меня действительно важно сказать здесь и объяснить суду, что у нас поломано все, включая систему правосудия, ровно для того, чтобы начинать потом такие войны. И эти войны, помимо того что, естественно, мы убиваем невиновных людей непонятно за что, — мы уничтожаем тем самым русский народ. Как я уже сказал, во Владимирской области, где я сижу, — здесь все руководство Росгвардии погибло, четыре подполковника, они мертвы. То есть понимаете, это одновременно скрывается и одновременно абсолютно все об этом знают. А почему их убили? Потому что один сумасшедший вцепился в эту Украину. Непонятно, что он хочет сделать, я не знаю, что он хочет с ней сделать. И этот сумасшедший вор нанял больного человека, который командует Росгвардией, и это, между прочим, не мои слова. Это слова бывшего начальника службы безопасности президента Коржакова, который, характеризуя генерала армии Золотова, сказал: «Я не могу назвать его дебилом, потому что он имбецил». И когда один вор нанимает имбецила для того, чтобы вести войну… Да, они убьют много кого на этой войне, но их самих там всех убьют.\n\n— Алексей Анатольевич, хочется все-таки услышать ваше последнее слово по поводу приговора Лефортовского районного суда города Москвы. Еще раз вас призываю: вернитесь к предмету судебного рассмотрения в данном судебном заседании.\n\n— Ваша честь, да не надо меня даже призывать. Я не то что вернуться… Предмет рассмотрения — это вот эта решетка в буквальном смысле. То ли я в него вцепился, то ли меня…\n\n— Алексей Анатольевич, вы осуждены по четырем составам преступления, предусмотренного частью 4 статьи 159 УК РФ и статьей 297.\n\n— Статья 159 УК РФ будет неоднократно применена в отношении тех лиц, о которых я говорю. Вместе со всякими другими статьями. Потому что эти люди, вот ровно для того, чтобы красть, мошенничать, грабить и убивать, — они и держат в тюрьмах таких, как я, ровно поэтому все это и происходит, ваша честь. Вот смотрите, к вопросу о статье 159-й. Вам, наверное, будет интересно. Раз уж вы требуете от меня про 159-ю статью. Я о ней постоянно думал знаете когда? Девятого мая. Девятого мая зэки, как положено, сидели в комнате, очень торжественно... Нам показывают парад. И на параде едет танк «Армата». И нам снова говорят, что танк «Армата» самый лучший, русские войска там… Я думаю: сейчас двадцать второй год, я смотрю танк «Армата». Я в двадцать первом году смотрел танк «Армата», и в двадцатом, и в девятнадцатом, и в восемнадцатом, и в семнадцатом, и в шестнадцатом. Мне кажется, с пятнадцатого года нам показывают танк «Армата». А на войне почему-то я вижу в основном танки Т-72 и немножко более современных, а «Арматы» никакой и близко нет. Так вот статья 159-я: может быть, потому что наш верховный главнокомандующий — вор и сумасшедший, наш министр обороны — вор и пиарщик…\n\n— Алексей Анатольевич, 22 марта 2022 года вы были осуждены Лефортовским районным судом города Москвы за совершение мошеннических действий, за оскорбление. Пожалуйста, призываю вернуться к предмету судебного контроля в суде апелляционной инстанции. Вы еще ни одного довода нам не озвучили конкретного о несогласии с вынесенным итоговым судебным решением Лефортовского районного суда города Москвы. Ваша защита выступала очень четко, обозначили все наши доводы, хочется услышать вашу позицию непосредственно по обжалованному судебному акту.\n\n— Ваша честь, мои адвокаты на воле, потому что выступают очень четко, а я всего лишь зэк за решеткой, потому что не способен выступать четко. Уж я выступаю как могу, мне немножко осталось, просто это важные вещи. Важная вещь заключается в том, что приговор Лефортовского суда выносится именем Российской Федерации и выносится именно таким образом, что власть в Российской Федерации захвачена теми людьми, которые на протяжении многих лет врут нам, в частности, про виды вооружений, устраивают войны, убивают других людей и своих людей, устроили кровавую баню из русских и украинцев. И последствиями этого всего вот эти процессы, как в отношении меня, будут в огромном количестве, и я хочу, чтобы они прекратились. И если я могу хотя бы своим словом сделать хоть что-то, чтобы хоть кого-то убедить, сделать хоть что-то, чтобы прекратить это все, я буду это говорить.\n\nЯ уже близок к завершению, не останавливайте меня больше. Уважаемый суд, я смотрю на вас, и в вашем лице я вижу ту самую «именем Российской Федерации»… Путин, Чемезов, Шойгу, все эти Голиковы, Христенки, все, про кого мы проводили расследования, — все они в вашем лице передо мной. И я хочу сказать одну вещь: как же без вас было бы хорошо! Как же без вас было бы классно жить в нашей стране, потому что впервые за сотни лет у нас ни врагов, ни войны, все нормально, денег море, нефти куча, газ дорогой, вообще Россия — прекрасная, богатая страна без вас. Поэтому я хотел бы, чтобы вас всех не было и чтобы идеология России… Идеология — громкое слово, просто чтобы правила нашей жизни были «живи и дай жить другим». Чтобы все оставили всех в покое, чтобы мы отцепились от Украины, от всех остальных, чтобы вы, судьи, отцепились от обычных граждан, чтобы Росгвардия занималась делами, чтобы ФСИНовцы действительно преступников охраняли, а не политзэков, чтобы полиция занималась расследованиями преступлений, а не ловила людей за лайки, чтобы вас не было и чтобы все друг от друга отстали, и тогда мы будем жить наконец-то хорошо.\n\nНо пока вы есть. И, конечно, самая уместная цитата из самой знаменитой книжки относительно вас, которая сейчас постоянно используется, которая много раз использовалась в таких же ситуациях за многие годы до этого, — «нынче ваше время и власть тьмы». Любопытная особенность, что прямо сейчас я вижу в вас, в вашей власти, что вас это, в общем-то, даже не задевает. Когда вы слышите эту фразу, вы понимаете про себя, что вы власть тьмы и сейчас ваше время, но вы не то что не переживаете — наоборот, это вас воодушевляет, вы такие: «Да! Мы власть тьмы, мы самые крутые, это наше время». И вот в этом ваша суть, что вы все злодеи, понимаете это и упиваетесь этим. Но я хочу сказать, что, конечно же, ваше время будет не всегда и ваше время пройдет. И когда вы все будете гореть в аду, дрова в ваш костер будут подкидывать ваши деды, которые не хотели, чтобы вы использовали их портреты для того, чтобы устраивать новые войны в XXI веке. У меня все, спасибо.'),
('https://navalny.com/p/6599/', 'Спасибо большое, ваша честь, за возможность высказать свое отношение к обвинению. Оно есть у меня. Я постараюсь высказать его довольно внятно и конкретно. Это займет какое-то время. Уважаемый прокурор, истекло уже больше часа. Я буду стараться говорить немножко громче — не думайте, пожалуйста, что я ору, просто мне сказали, что не слышно на трансляции, давайте погромче.\n\nКак вы думаете, какую эмоцию я испытал, когда прочитал это обвинение здесь, в колонии? Гнев, или отрицание, или принятие? Что я почувствовал? Может быть, я разозлился на то, что вы сделали такое очевидно сфабрикованное дело? Нет. Ликование! Я, знаете, изображаю человека, который счастлив: он сидит в кресле вот так, развалился и, как Леонардо Ди Каприо в меме, шампанское держит. Я был счастлив, я на седьмом небе от счастья находился, и сейчас вы поймете почему.\n\nЯ веду свою политическую деятельность много лет единственным и, мне кажется, самым честным способом — собирая пожертвования с людей. Те, кто хочет, перечисляют мне деньги, те, кто не хочет, не перечисляют. Я никогда не брал ни копейки государственных денег и очень этим горжусь. И я, и мои коллеги — мы являемся теми самыми политиками, которые существуют исключительно потому, что у них есть поддержка. Довольно мы давно это делаем. Деньги мы собирали с 2011 года, и с того момента около 300 тысяч человек перечислили нам донаты. В среднем где-то около 500–600 рублей. 300 тысяч человек!\n\n2011 год был достаточно давно — за это время, очевидно, какие-то люди разочаровались в моей деятельности, ну не понравилось, что я им сказал по какому-то их вопросу или еще что-то. Так случается. Сегодня ты голосуешь за одну партию, завтра ты голосуешь за другую партию. Сегодня нам нравится Навальный, завтра нам нравится Путин. Или наоборот, такое бывает. Поэтому, когда я заявил, находясь в Германии на лечении, что я возвращаясь в Россию, а в ответ мне СК заявил: «Мы тебя посадим за то, что ты похитил все пожертвования» и выпустил пресс-релизы, которые мы будем рассматривать на этом процессе, о том, что я похитил и потратил на свои нужны миллиард рублей... Потом был пресс-релиз о том, что я похитил 385 миллионов рублей. И так далее, и так далее.\n\nЯ понимал, что, конечно, очевидно им было… написать то, что мы получили в результате, но 300 тысяч человек! И мы знали, что они допрашивают всех сплошняком. По регионам все люди писали: «Нас вызывают на допрос». То есть всех, кто пожертвовал нам, а мы принимали все пожертвования безналично, и это списки всех жертвователей... Я понимал, что, наверное, довольно много людей, просто потому что они как-то разочаровались во мне или в моих коллегах, могут взять и написать заявление.\n\nНу так и что? После всех этих ваших «украл миллиард и потратил его на себя», «украл 300 миллионов и потратил на себя» — что вы принесли в суд? Вы в суд принесли в буквальном смысле материалы о том, что есть четыре человека. 300 тысяч жертвователей за все эти годы — и четыре человека написали заявления на два миллиона рублей.\n\nИз которых один — это слесарь. Который в буквальном смысле, тут так написано, «я посмотрел ролики Навального, подумал, какой классный Навальный, а потом я взял миллион рублей — миллион двадцать тысяч рублей — и перевел Навальному». А спустя три недели слесарь подумал: «Нет, Навальный не классный, он меня обманул». И он пишет заявление, и на следующий день возбуждается уголовное дело.\n\nИ это говорит о том, что если кто-то когда-то меня спросит: ну какие доказательства ты предъявляешь в пользу того, что у нас Фонд борьбы с коррупцией — самая честная, самая прозрачная организация? Я скажу: мои доказательства — это материалы этого уголовного дела. И вы увидите, что каждый перевод зафиксирован. Здесь просто есть распечатки всех платежей по карточкам: моей, моей жены, моей дочери, большинства сотрудников ФБК.\n\nЗдесь же нет ничего. Есть два потерпевших, которых принесли сами вы, которые перечислили деньги, а потом так разочаровались, что принесли заявление, и есть два запуганных предпринимателя, находящихся под уголовными делами. И все.\n\nВы здесь произнесли фразу, что «похитили денежные средства для личных нужд». Но нет ни одного слова даже в вашем полностью, абсолютно сфабрикованном деле, нет ни одного слова о том, что хотя бы копейка из этих денег поступила ко мне. Что хотя бы копейку я или мои коллеги перечислили на свои нужды.\n\nПоэтому, конечно, я испытал ликование, конечно, я испытал огромную благодарность к тем самым 300 тысячам людей, которых запугивали, уговаривали, и никто из них не написал заявление. Вам пришлось искать липовых подставных людей, чтобы перечисляли нам деньги, потому что из 300 тысяч людей никто не написал. Это лучшие люди. Конечно, я испытываю ликование. Я испытываю ликование, взвешивая нашу систему, при которой «китайская стена» между деньгами-пожертвованиями и личными деньгами ни разу не пересеклась и ни одна копейка никогда не ушла никому и никуда, кроме как на нашу деятельность.\n\nЕсли что-то и есть полезное в этом деле — это то, что, действительно, это глобальный аудит всего, что делал ФБК, который полностью доказал, что мы делали абсолютно все верно, потратили все на ту деятельность, о которой и говорили, на антикоррупционную деятельность, и все мы делали очень хорошо и делали правильно. И наше движение — это потрясающая политическая сила, в которой можно найти 300 тысяч человек, которые тебя не продадут. Даже под нажимом и запугиванием следователей и всех остальных.\n\nЧто касается сути обвинения, она, конечно, заключается в том, что людям запрещается вести политическую деятельность без вашего разрешения. Без разрешения Кремля и всех остальных. Потому что, по сути, что я сделал? Люди поддерживают национального лидера Владимира Владимировича Путина. А я нет! И, наверное, никто не будет спорить с тем, что есть разные политики, люди разных взглядов, они голосуют за представителей разных партий, и я считаю, что у меня есть полное право выходить и заявлять политические требования. Потому что мне не нравится, что здесь происходит, я ненавижу то, что здесь происходит. Ставка фельдшера в колонии — 14 тысяч рублей, понимаете? Со всеми надбавками — 21 тысяча рублей в месяц зарплата. Нефть 93 доллара за баррель, Путин строит себе дворец за 100 миллиардов рублей, а фельдшер получает 21 тысячу. Здесь инспектор, здоровый мужик, получает 25. Поэтому опера ходят и локалки открывают, потому что текучка, потому что работать никто не хочет, потому что зарплаты нет. Вся Владимирская область корячится за 25 тысяч рублей. И мне это не нравится. Я имею право, чтобы мне не нравилось. Я не понимаю, как в стране, которая качает 20 лет нефть и газ, может быть такая нищета.\n\nПоэтому я собираю тех, кому это тоже не нравится, я объединяю их в партию, объединяю в движение, я веду расследования того, куда украли деньги у фельдшера, куда украли деньги у этого инспектора, куда ваши деньги украли, и ваши деньги, и ваши, и мои, и пенсии, и все остальное. Да, мы это расследуем, и для расследований нужны деньги. Какой самый классный и честный способ? Наверное, если обойти людей, просто найти кого-то и сказать: как ты думаешь, на какие деньги должен существовать политик, на бюджетные? Он скажет: собирайте у тех людей, которым нравится этот политик. Я так и решил делать. И я делаю это и буду продолжать это делать. Не существует никакого другого способа нормальной политической деятельности, кроме как человек, который что-то заявляет, чем он доволен и недоволен, объединяет других, собирает с них деньги и продолжает деятельность.\n\nЗал суда, судебное следствие… Давайте здесь проведем судебный эксперимент. Я вот вам сейчас говорю, уважаемый суд, уважаемый прокурор, можете записывать: в 16 часов 25 минут гражданин Навальный, находясь по адресу такому-то в колонии, призвал — вот сюда и вот сюда — всех сотрудников Фонда борьбы с коррупцией продолжать расследования, продолжать публиковать факты коррупции, находить, куда Путин, его группа, его родственники, его вторая жена, третья жена — куда они дели украденные деньги. Расследовать всех этих министров-единороссов, публиковать это, призывать всех остальных распространять это.\n\nЯ призываю всех, как там обычно — «неограниченную группу лиц», людей, граждан России я призываю делать нам пожертвования, чтобы финансировать эту деятельность антикоррупционную. Распространите как следует, потому что это моя политическая деятельность, и я борюсь, да, за то, чтобы сменить власть в стране. Я не хочу, чтобы в Кремле сидели эти люди. Они там десятилетиями сидят! Ну не может такого быть, что у нас каждый министр официально уже долларовый миллионер. Они воры. Я считаю, что они воры. Я прошу делать нам пожертвования, чтобы мы расследовали деятельность этих воров.\n\nЯ тогда на каждом новом судебном заседании буду даже рассказывать о кампании по сбору средств, назовем так: кампания судебных пожертвований. Можете возбудить на меня еще одно уголовное дело, прямо сейчас, понимаете? Как там — «имея преступный умысел на хищение средств, призывал делать фандрайзинг»? Вы можете это так назвать, еще одно дело, как я уже сказал, завести, что угодно можете сделать мне — я не вижу другого способа нормального, честного вести политическую деятельность, кроме как донаты. Если я считаю, что они воры, я об этом говорю, я доказываю, что они воры, я показываю это всем.\n\nЯ буду участвовать в выборах. Да, сейчас есть законодательство, которое для меня прописали, и вы все повторяете, что я не имею права участвовать в выборах. Не только не имею права участвовать в выборах — у вас в материалах написано, цитирую: «Волков сделал ложное заявление о том, что он глава штаба Навального». Ну отлично! То есть теперь даже вы лучше меня знаете, кто начальник моего штаба. Волков — ложный начальник моего штаба. Что угодно можно написать, но ведь есть простая вещь: я хочу участвовать в выборах. Я хочу, чтобы другие кандидаты участвовали в выборах. Я хочу, чтобы была партия, за которую можно голосовать. Я требую допуска на выборы и других людей, и себя. Что в этом несправедливого, что в этом необычного? Вы считаете, что меня нельзя допускать, а я считаю, что это незаконно и неправильно. А я хочу участвовать в выборах и ко всем вновь обращаюсь: если вы считаете, что я и такие, как я, могут участвовать в выборах, давайте вместе объединяться и действовать. Давайте поддерживать нас деньгами. Если нас не допускают — участвуйте в Умном голосовании, для того чтобы мочить единороссов, которые стоят за вами.\n\n(Судья Котова: Алексей Анатольевич, мы сейчас слушаем ваше отношение к предъявленному обвинению…)\n\nТак я об этом и говорю! Вот видите, прозвучало слово «Единая Россия» — и вы уже прямо сталкиваетесь с обвинением, уже ближе некуда. Потому что моя деятельность — это деятельность по организации людей, которым не нравятся «Единая Россия» и Путин, которые ненавидят «Единую Россию» и Путина, которые хотят отстранить от власти «Единую Россию» и Путина, потому что «Единая Россия» и Путин — это ограбление нашей страны.\n\nЭто объективный факт, понимаете? Это объективный факт нищеты. На Покров поглядите! Я Покров не видел… Меня все время возят, знаете, у меня окон нет в машине. Но вы же проехали по городу Покрову. В 100 километрах от Москвы — но вы же видели, что это просто бомжатник какой-то! Почему это происходит? Потому что такая власть. Я буду с ней бороться. Я не вижу другого способа. Я буду продолжать с ней бороться. И я не боюсь ни этого суда, ни колонии, ни ФСБ, ни химического оружия, ни Путина, ни всех остальных. Я не боюсь, потому что считаю унизительным и бесполезным бояться этого всего.\n\nПлохо жить и смириться с этим всем. Смотреть на эту раздолбанную дорогу и смириться с ней, понимаете? Двадцать первый век! Ну да, дорога, конечно, дорога в Покрове — ее нет. Да почему ее нет? Ну да, 14 тысяч фельдшер получает. Да почему я должен это принять? Почему все остальные должны это принять?\n\nМы это понимаем, мы организовываем свою деятельность, и мы продолжим. И я считаю, что я имею право участвовать в выборах. Я буду этого добиваться. Может быть, я когда-нибудь в них буду участвовать, может быть, нет. Может быть, я какое-то место займу, может, не займу, но это будет зависеть от людей, от избирателей, которые за меня проголосуют, если я буду действовать правильно, или не проголосуют, если я буду действовать неправильно и неконкретно.\n\nЯ… даже не знаю, честно говоря, нужно ли мне комментировать другую часть этого экзотического дела, которая касается оскорблений. Я просто вдруг подумал, может быть, это для вас самих развлечение? Потому что, во-первых, объединение в одном деле мошенничества и оскорблений — это странно, во-вторых, вы читали эти оскорбления — как вы смех сдерживали? Значит, оскорбительная фраза: «Ваша честь, Вы нарушаете закон». Ваша честь, посмотрите, пожалуйста, там даже в тексте буковка «В» заглавная. Потому что очевидно, что по интонации фраза «Ваша честь, Вы нарушаете закон» — она оскорбляет кого-то? Нет. Мы с вами обсуждали сегодня допуск адвоката, я вам говорил: «Вы нарушаете закон». Вы мне говорили: «Нет, не нарушаю» — и ссылались на что-то. Мы поговорили. Это оскорбление судьи?\n\n«Я жду, пока вы закончите». Это оскорбление кого? Прокурора, судьи? И, конечно, мое любимое: кавычки открываются — «боже мой» — кавычки закрываются. Боже мой, ваша честь, если вы каждого человека будете судить за «боже мой», я не знаю, что будет происходить. Тогда уже, если кто-то считает, что за фразу «боже мой» нужно судить, нужно было прописывать, что Навальный, находясь в таком-то месте в такое-то время, оскорбил Бога, потому что запрещено упоминать имя Господа нашего всуе. Кого я мог оскорбить фразой «боже мой»?\n\nИ я понимаю, зачем это сделано, потому что это такая попытка запугать, сказать, что если ты будешь что-то говорить, не будешь просто молчать, не будешь покорно кивать, не будешь заискивать, не будешь бояться нас, судей и прокурора с двумя, с тремя, с десятью звездами, то, значит, мы будем тебе клепать уголовные дела.\n\nДа и плевать! Господи боже мой, клепайте! Я все равно не замолчу, понимаете? После того, как уже даже какие-то эфэсбэшники за мной ездили два года и травили химическим оружием, и я лежал в коме 20 дней, а потом еще 20 дней лежал в галлюцинациях, мне бояться вашего суда просто глупо. Я его и раньше не боялся, а теперь я точно не буду бояться.\n\nЗавершая свое выступление, я хочу сказать, что, конечно, мы по пунктам, вот просто по пунктам все это вранье разоблачим в ходе этого процесса. В общем-то, даже разоблачать нечего, вы говорите, что я тратил на личные нужды, здесь даже не упоминается ничего — просто ноль! Мы разберем все по пунктам и по пунктам все опровергнем.\n\nКонечно, я понимаю — это не первый мой процесс, и я не наивный человек, — что приговор будет обвинительный, по нему будет достаточно большой срок. И смысл этого один. Раз я так оскорбил вашего темного лорда Путина, что не просто выжил, но и вернулся, ну вот он сказал: раз он считает, что он такой крутой — пусть тогда он сидит в тюрьме. Вернулся — и будет сидеть там пожизненно, и будет это дело, второе дело, третье, будет бесконечно увеличиваться срок. Ну что ж. Я считаю, что все равно моя деятельность, деятельность моих коллег важнее, чем просто конкретная судьба человека. И я считаю, что худшее, что я на самом деле могу сделать, настоящее преступление, которое я могу совершить, — это вас всех испугаться, и вас, и тех, кто стоит за вами. Я еще раз вам говорю, что я не боюсь, и я еще раз призываю всех остальных тоже не бояться, потому что бояться здесь нечего. Бояться нужно того, что мы всю жизнь проведем вот так, всю жизнь мы проведем в нищете, деградации, отсутствии перспектив и своим детям оставим в наследство вот это вот все — вот это холуйство, раболепство и то же самое тоскливое ожидание лучшей доли, которая не наступит никогда, пока у власти находится эта банда воров.'),
('https://navalny.com/p/6564/', 'Легитимность — очень непростая штука: ее не измерить никак, а от нее очень многое зависит.\n\nКак сообщает Википедия, «легитимность — согласие народа с властью, его добровольное признание за ней права принимать обязательные решения». Откуда она берется? Из выборов. И даже не из самих выборов, а из их восприятия людьми. «Почему мы слушаемся министра или губернатора, хотя мы их знать не знаем, а если и знаем — то справедливо считаем жуликами и ворами? Потому их Путин назначил. А за Путина, как ни крути, люди голосовали» — вот примерно такой мысленный диалог в последние несколько лет был типичен для оппозиционно настроенного обывателя. И во многом это было справедливо. Если, скажем, даже посмотреть на знаменитые графики Шпилькина по выборам в Госдуму в 2016 году и по выборам президента в 2018 году, там видны огромные фальсификации (по 10-12 миллионов голосов вброшено в пользу «Единой России» и Путина), но при этом и еще кое-что видно: если бы не было всех этих фальсификаций, ЕР и Путин все равно победили бы. Их победы не были бы такими разгромными и внушительными, это хуже смотрелось бы с пропагандистской точки зрения, но, тем не менее, большая часть избирателей, которые пришли на участки и которые отдали голоса, тогда поддерживали власть.\n\nЭто больше не так — и это и есть главный политический итог выборов в Госдуму в 2021 году. Объем фальсификаций вырос не так существенно (14 миллионов вброшенных голосов — ужасно, если вдуматься, но не настолько хуже, чем три или пять лет назад), а вот ценность их выросла несоизмеримо. Потому что без фальсификаций на этот раз Единая Россия проваливалась с треском, набирая менее трети голосов по партийным спискам и проигрывая порядка 70-80 одномандатных округов, то есть теряя даже простое (а не только конституционное) большинство в Госдуме. Умное голосование в сентябре 2021 года одержало большую политическую победу.\n\nПонимали ли мы, что фальсификации будут? Конечно, ну мы же не с Марса свалились. Но мы ожидали, что, благодаря Умному голосованию, Кремлю придется пойти на такие существенные и заметные фальсификации, что это значительно подорвет легитимность власти. Что украсть выборы не получится незаметно. Удалось ли нам это? Вот чтобы понять, как российское общество на самом деле воспринимает итоги выборов в Госдуму, мы и провели по горячим следам, почти после выборов, очередной телефонный опрос российских избирателей (по репрезентативной выборке в 1000 человек). Давайте вместе посмотрим на результаты этого опроса.\n\n\nНа первом слайде — ничего удивительного. Ну или все удивительно, тут уж как посмотреть. Самый простой вопрос мы спросили: «дорогой избиратель, как ты проголосовал?». Вопрос задавался через 1-2 недели после тех самых выборов, на которых, по официальным данным Центризбиркома, «Единая Россия» одержала уверенную победу, набрав 49,8% голосов. Но вот незадача: почему-то в телефонном опросе факт голосования за Единую Россию подтвердил лишь 31% респондентов. Поразительное расхождение с официальными данными ЦИК… поразительное совпадение с результатами математического моделирования, отбрасывающими фальсификации. Однако, по все тем же данным Шпилькина получалось, что за КПРФ проголосовало в действительности почти столько же избирателей, сколько за ЕР, а у нас на слайде — большой разрыв. КПРФ вторая, с 19% голосов. Почему так? Смотрим дальше и видим: целых 22% респондентов, принявших участие в выборах, отказываются по телефону сказать, как они проголосовали. То есть боятся.\n\nЭто большая новость: мы не первый год делаем послевыборные опросы для того, чтобы проверить официальные результаты и получить реальную картину произошедшего, но такое видим впервые. Страх. Каждый пятый избиратель — из тех, что уже согласились поговорить по телефону с социологом и ответили положительно на вопрос об участии в выборах, — не решился сказать о том, какой он сделал выбор. Такого не было никогда.\n\nИ объяснение может быть тут только одно: люди боятся. Боятся, потому что ожидают репрессий за «неправильный» ответ. И, конечно, это значит, что внутри этих 22% «сидит» протестное голосование: можно смело предполагать, что почти все эти люди проголосовали не за «Единую Россию», но вряд ли можно что-то уверенно сказать о том, в какой пропорции голоса этих «респондентов-отказников» распределились в реальности между другими партиями. Ясно одно: реальный результат КПРФ, ЛДПР, СРЗП и «Новых Людей» был выше, чем показано на нашем слайде. И, кстати, обратите внимание — вероятно, «Новые люди» в реальности заняли третье(!) место. Так было почти повсеместно в тех регионах Дальнего Востока и Сибири, которые традиционно в наименьшей степени затронуты фальсификациями, — а наш опрос показал, что, видимо, так было и по всей России.\n\n\nДля контроля мы еще задавали вопрос о голосовании по одномандатным округам: здесь картинка в целом получается такая же, а результат «Единой России» даже еще меньше.\n\nНу а теперь о главном: о легитимности власти после этих выборов, об их восприятии избирателями.\n\n\nЧетверть опрошенных говорят, что «недовольны» результатами выборов, а в целом 37% россиян «скорее недовольны или недовольны», в то время как 47% «довольны или скорее довольны». Как-то не наблюдается всенародного единения и ликования. Более того, общий процент довольных результатами выборов — меньше, чем, по официальным данным, получила «Единая Россия». То есть, получается, многие люди (по данным ЦИК) за ЕР голосовали, но при этом не рады победе своей партии… Странно!\n\nНо еще более наглядную картину произошедшего мы видим, когда задаем нашим респондентам прямой вопрос об их оценке честности выборов:\n\n\nИ этот вопрос мы задаем уже много лет каждый раз после крупных выборов. И такого мы не видели никогда. Если совсем коротко, то, Элла Александровна, это фиаско. Вы провалили всё, вас Кириенко с Путиным погонят без выходного пособия. Сколько миллиардов рублей, сколько пропагандистских уловок и ужимок вы вложили в то, чтобы объяснить россиянам, что у нас самые честные, демократичные и замечательные выборы в мире, — а россияне вам не поверили. У вас не получилось. Половина российских избирателей полагает, что официальные результаты выборов не отражают реального мнения граждан — и лишь треть верит в честность выборов и подсчета голосов. Вариант «не отражают», победивший с огромным отрывом, набравший 39% ответов — это те люди, которые понимают, что выборы абсолютно нарисованные. И таких вдвое больше, чем 18%, верящих в честность выборов без тени сомнения.\n\nИздевательство над выборами, борьба с «Умным голосованием», стремление во что бы то ни стало нарисовать «Единой России» красивый результат — все это обошлось Путину слишком дорого. По легитимности российской власти нанесен серьезный удар; после 19 сентября 2021 года большинство россиян знает, что ими правит парламент, который они не выбирали; что депутаты — это воры, укравшие мандаты. Это важное, переломное изменение в общественном мнении, которое будет иметь долгоиграющие последствия; они не обязательно проявятся сразу, но рано или поздно проявятся.\n\nПока ничего особенного в политической жизни страны не происходит, ситуация может казаться стабильной. Но рано или поздно любая власть в любой стране сталкивается с кризисами, в рамках которых ей приходится принимать непопулярные решения; и вот тут-то легитимность начинает играть огромную роль. Власть, лишившаяся легитимности, оказывается колоссом на глиняных ногах: люди попросту отказываются принимать непопулярные решения из рук такой власти. И любой кризис может стать поэтому переломным моментом.'),
('https://navalny.com/p/6563/', 'Это Лилия Чанышева.\n\n\nЛилия живет в Уфе, и очень любит свой родной Башкортостан. Вот она на дне республики, танцует в национальном костюме.\n\n\nОна любит кататься на лыжах и гулять. Играет в футбол и волейбол. Любит хоккей и болеет за родной «Салават Юлаев».\n\nНедавно Лилия вышла замуж.\n\n\nОбычная молодая женщина. Работает, отдыхает, ходит в походы и в театр, ездит в отпуск. Живет совершенно нормальной жизнью.\n\nА это Путин. И он ненавидит Лилию.\n\n\nОн считает, что она враг, предательница, преступница и должна сидеть в тюрьме. И не год, не два, а десять лет.\n\nВы скажете: такая приятная девушка, что она такого сделала? Какие еще 10 лет?!\n\nА дело в том, что Лилия Чанышева — политик. Это ее работа. 4 года она была координатором штаба Навального в Уфе.\n\n\nДелала расследования, выступала на митингах, публично критиковала чиновников. У нее есть своя политическая программа, с которой она шла на конкурс сити-менеджера Уфы.\n\n9 ноября к ней в квартиру вломилась полиция.\n\n\nПровели обыск, увезли Лилию на допрос. С допроса она не вышла — ее прямо там задержали на 48 часов до суда. Обвинение — создание экстремистского сообщества.\n\n10 ноября суд отправил Лилию в СИЗО. Сейчас она единственный человек, арестованный в связи с признанием ФБК экстремистской организацией. То есть самая опасная преступница. Прямо сейчас даже опаснее Навального — ему обвинение еще не предъявлено.\n\nЛилия пришла к нам работать в 2017 году, когда мы только открыли штаб в Уфе. Мы тогда впервые разворачивали такую огромную региональную сеть и было неясно, как вообще мы наладим эту работу, легко ли будет искать людей. А Лилия до этого работала в Делойте — крупнейшей аудиторской компании, входящей в большую четверку. Там у нее была хорошая зарплата, страховка, прекрасные перспективы: могла бы стать главой офиса, переехать на повышение в Москву. Жизнь была стабильна и предсказуема. Но Лилия решила уволиться из Делойта и работать у нас, даже несмотря на то, что мы могли платить ей в 3 раза меньше. Потому что она прямо по-настоящему хотела заниматься политикой.\n\nЗа 4 года, которые прошли с открытия штабов, везде были какие-то перестановки, люди приходили и уходили. Но Лилия все эти годы была координатором штаба в Уфе. Более того, под ее руководством уфимский штаб был одним из лучших.\n\nКогда президентская кампания закончилась, работа штабов изменилась — они сосредоточились на своих регионах, местных коррупционерах. И вот окончательно стало ясно, что Лилия — идеальный политик. Она не просто менеджер, который хорошо выполняет административные функции, она по-настоящему горит работой, умеет ее делать и она ничего не боится.\n\nЛилия организовывала митинги. Лилия защищала Куштау. Лилия несколько раз пыталась участвовать в выборах, но до них ее не допускали. Она делала расследования против башкирских чиновников и главное — местного губернатора Хабирова. Она возглавляла наблюдение на выборах в Новосибирске в 2020 году — именно тогда с помощью Умного голосования нам удалось провести в местный парламент координатора нашего новосибирского штаба, Сергея Бойко.\n\nИ, разумеется, чем больше Лилия делала, тем больше усиливалось давление на нее. Суд обязывал ее выплачивать колоссальные штрафы за организацию митингов. Несколько раз она сидела в спецприемнике. Ей повредили машину, облив краской. В 2019 году она пришла на публичные слушания по проекту бюджета Башкортостана, чтобы выступить там с его критикой, — и ее выдворили из здания Госсобрания силой.\n\nИ ничего из этого Лилию не испугало.\n\nКогда весной против ФБК завели дело об экстремизме, мы закрыли нашу региональную сеть. Заранее, до суда, чтобы обезопасить людей, которые раньше у нас работали. Для Лилии это стало большим ударом, потому что ей, по сути, нужно было отказываться от дела, которое она любила больше всего, и начинать новую жизнь.\n\nОна попыталась. В апреле Лилия вышла замуж. Они с мужем твердо решили не уезжать из России. Лилия занялась плетением макраме на заказ. Они планировали завести ребенка.\n\nНо Путин не дал ей начать новую жизнь. К ней ворвались с обыском, увезли на допрос, задержали до суда, а суд отправил ее в СИЗО как экстремистку.\n\nНа суде Лилия сообщила, что, вероятно, беременна. Срок маленький — 14 дней — и она не успела еще пройти обследование. А тюремные медики, конечно, ничего не устанавливали.\n\nПросто представьте. 4 года человек работает совершенно официально. Снимает офис в центре Уфы. Платит налоги. ФБК и штабы каждый год подают в Минюст отчет как некоммерческая организация. Ни у кого нет вопросов. А потом суд внезапно объявляет эту работу экстремистской. Это безумие, но ЛАДНО. Закон ведь не имеет обратной силы. С июня работа на Навального — экстремизм, но до этого она ведь была совершенно легальна. Как в принципе человека можно посадить постфактум за его работу, к которой на протяжении 4 лет не было никаких претензий? Оказалось, можно. И из всех посадили именно Лилию — за то, что она храбрая, честная и непримиримая.\n\nВ первый раз Лилию арестовали на 5 суток за митинги 28 января 2018 года. И это казалось просто немыслимо — женщину за митинг под арест! Тогда аресты вообще еще казались чем-то экстраординарным, даже к судам над Алексеем еще не привыкли. А тут интеллигентная девушка-аудитор — и в спецприемник.\n\nПотом многое вошло в норму. Административные аресты вошли в норму. Домашние аресты. Мы видели, что ничего не стоит развернуть самолет и посадить человека.\n\nНо вот прямо сейчас на наших глазах пытаются сделать нормой очередной кошмар. Мы прямо в эпицентре этого. От нас зависит, станет ли это обыденностью. Если мы сейчас спокойно проглотим, что Лилию Чанышеву обвинили в экстремизме и посадили за ее официальную работу, то они пойдут дальше. И завтра с любым из нас может произойти что угодно.\n\nЭто, правда, очень важно осознать. На наших глазах молодую женщину суд отправил в СИЗО. На это заседание пустили журналистов — но не пустили ее мужа. Просто так. Решение об аресте судья «принимал» 20 минут — этого бы ему не хватило даже на то, чтобы под диктовку напечатать решение на компьютере.\n\n\nМы видели много политических процессов, но до этого Кремль пытался как-то замаскировать обвинение. Мол, мы вас не за политику сажаем, а за мошенничество. Но тут они уже прямо сказали: мы посадим Лилию Чанышеву за то, что она занималась политикой, обвиняла чиновников в коррупции, находила этому доказательства, организовывала митинги. Мы посадим ее за то, что она хотела честных выборов, нормальных зарплат и справедливости. Мы посадим ее за то, что она не захотела уезжать. Мы посадим ее в назидание всем, чтобы вы сидели тихо и не смели пикнуть, пока мы и дальше будем вас грабить.\n\nЛилии по обвинению в экстремизме грозит 10 лет. Путин может одним движением сломать жизнь ей и ее родным. Но совершенно точно мы можем этому помешать. Мы должны этому помешать. Это не просто какой-то очередной арест, которых теперь стало так много. Это новая ступенька на пути к тому, чтобы уничтожить всех. Чтобы в России остались только чиновники, которые воруют, и нищие, бесправные люди, которые научились только молча выживать и не видят никакой надежды. Если мы сейчас будем стоять в стороне и молчать, то это станет нашим будущим.\n\nПоэтому самое важное, что нужно сейчас сделать. О Лилии нужно говорить как можно больше. Нужно рассказывать всем, чем она занималась, в чем ее обвиняют и почему. Это история в самом деле абсолютно немыслимая, исключительная. Но большинство просто о ней не узнает, если вы им не расскажете. Не хотите объяснять сами — просто пошлите этот ролик. Повесьте его в соцсетях.\n\nЧем больше мы будем об этом говорить, тем больше шансов, что следующие 10 лет Лилия не проведет в тюрьме. Что ее жизнь, жизнь ее мужа и ее родителей не превратится в ад. Но и что наша собственная жизнь не превратиться. Мы не можем допустить такое.'),
('https://navalny.com/p/6530/', 'В этом заключается работа президента Байдена. И европейских лидеров тоже. Они встречаются и ведут переговоры с другими мировыми лидерами. Мы можем, конечно, поговорить о том, насколько подобные встречи выродились в ритуал. Но так или иначе в первой половине 21 века работа президентов во многом состоит из встреч с другими президентами, даже если они очень неприятные люди и лгут в каждом своем слове, как это происходит в случае с Путиным.\n\nДругой вопрос в том, достиг ли Байден каких-нибудь целей? Очевидно, что пока нет. Путин моментально извлек из встречи всю запланированную выгоду, ведь ему ничего и не нужно было, кроме факта этой встречи и телекартинки. Этого достаточно, чтобы внутренняя пропаганда устроила массовое промывание мозгов на тему: «Сколько бы эти сменяющиеся западные лидеры ни критиковали нашего вечного мудрого президента, все равно они идут к нему, садятся за стол переговоров и о чем-то просят. Ведь ни одна мировая проблема без него не решается». Я тут в лагере обязан смотреть государственное телевидение, и поверьте, эта тема была отыграна очень эффективно.\n\nБайден же имел задачу куда более сложную: повлиять на поведение Путина как внутри России, так и на международной арене. И здесь пока последствия ровно обратные желаемому. Уже после встречи Путин фактически криминализировал любую оппозиционную деятельность, объявив ее «экстремистской». Десяткам тысяч людей запрещено быть кандидатами на выборах любого уровня. От текущих выборов в Госдуму с формулировкой «за поддержку Навального и участие в митингах» отстранены все сильные кандидаты. Разгромлены и объявлены иностранными агентами лучшие СМИ. Фактически в России возрожден печально известный советский закон «Об антисоветской агитации и пропаганде». Интернет-сайты блокируются сотнями уже даже без формальных судебных решений. Количество политзаключенных растет. Градус антизападной истерии в СМИ тоже, хотя вроде куда уже дальше. Так что Путин послушал Байдена и сделал все наоборот. Ужесточение репрессий произошло бы и без встречи с Байденом, но ее факт их не предотвратил и не замедлил.\n\nНо здесь важно понять, что какова бы ни была реальная стратегия Байдена, она просто не может дать мгновенного результата. Те элементы стратегии, которые мы можем видеть, например, совершенно замечательное, я бы сказал, историческое заявление о коррупции, говорят нам о том, что она может быть эффективной, а не останется ритуальной пляской, над которой вновь посмеются кремлевские элиты.\n\nНе надо накладывать санкции на Россию. Санкции, гораздо более жесткие, чем сейчас, должны быть наложены на тех, кто грабит Россию, делает ее народ беднее и лишает его будущего. Это так и должно называться: «Пакет санкций в поддержку российского народа, против коррупции, лжи и тирании».\n\nДавайте четко скажем: пока все санкции такого рода специально были сделаны так, чтобы под них не попал почти никто из значимых участников путинской гангстерской группы. Хотите доказательств? Ну так назовите хоть одного реально пострадавшего злодея. Самолеты, яхты, миллиарды в западных банках. Все на месте.\n\nИ то, что до сих пор не введены санкции в отношении олигархов, — страшное разочарование. Режим Путина — это мафиозная группа. Даже по формальным признакам. Обратите внимание, что члены этой мафии переженились, породнились через детей. Это уже набор семейных кланов, где папа — глава ФСБ или бывший глава ФСБ, а сынок — глава государственного банка, «легально» приносящий в семью миллиарды долларов.\n\nИ над этим стоит capo di tutti capi — Путин.\n\nТе немногие индивидуальные санкции, которые введены, почти всегда направлены против muscle этой мафии — полковников и генералов. Да, они злодеи и отдают приказы об убийствах. Но все же это ничего не решающие и легко заменяемые люди. У них нет активов на Западе, они не ездят на вечеринки в Лондон и Нью-Йорк. Пусть они остаются в санкционных списках, но главной его частью должны стать те, ради кого оппозицию отстранили от выборов. Это олигархи. Как государственные — главы госкорпораций и госбанков, так и формально частные, но благосостояние которых прямо нуждается в дальнейшем развитии авторитаризма. Олигарх, покупающий СМИ, чтобы провести там зачистку — уволить независимых журналистов и сменить редакционную политику, — вот более важное оружие Путина, чем кого-то арестовывающий генерал.\n\nСвою колонку, вышедшую на прошлой неделе в трех мировых газетах, я во многом этому и посвятил. Лидеры Запада и, в первую очередь, президент Байден должен проявить реальную решимость в борьбе с коррупцией. В частности перестать уже именовать путинских олигархов «бизнесменами». Любой путинский бандит и мафиози, назвавший себя «бизнесменом», автоматически рассматривается как «почти свой». Человек, с которым можно иметь дело. В Кремле же просто в восторге от того, как легко можно манипулировать американской администрацией фразами вроде: «Ребята, ну за что вводить санкции против олигархов? Что за популизм? Это же просто бизнесмены, хоть и крупные. Они никого не убивали». Они не убивали и не сажали. Но ежедневно убивают и сажают ради них.\n\nИнтересно, что на уровне законодательной власти все это понимают. Заявления лидеров и участников Антикоррупционного кокуса, недавно созданного в конгрессе США, очень верны. Депутаты Европарламента полны решимости вводить санкции против олигархов. Но исполнительной власти по обе стороны океана трудно совладать с армией юристов, лоббистов и банкиров, сражающихся за право владельцев грязных и кровавых денег оставаться безнаказанными.\n\nТакого риска нет, это уже произошло. Несколько лет это одна из основ риторики Кремля: все наши беды от западных санкций, а оппозиция требует новых. Интересно, что непопулярные «антисанкции», введенные самим Путиным, вроде запрета европейских продуктов питания, пропаганда тоже именует «западными санкциями». Так ведь я поэтому и призываю таргетировать персонально злодеев и олигархов. Такие действия Запада будут полностью поддержаны российским обществом и даже вызовут ликование. В глазах обывателя именно такие меры покажут, что Запад не лицемерит, что не «все они заодно» и что наконец хоть кто-то вступился за его интересы.\n\nА пока олигарх Усманов, плативший миллиардные взятки Дмитрию Медведеву, без проблем живет между Монако и Италией на своей яхте Dilbar стоимостью выше 470 миллионов долларов, а Роман Абрамович, финансировавший дворец Путина, заказывает постройку особняков на Upper East Side, никто не поверит в то, что Запад действительно хочет помочь русским спасти свою страну от разграбления группой Путина. Не бывает борьбы с коррупцией без борьбы с коррупционерами.\n\nСложно сказать. С одной стороны, Путин реально опасается, что новые секторальные санкции обрушат экономику России, где и без того реальные доходы населения падают семь лет подряд. С другой — вот это «не поддаюсь давлению» давно превратилось у него в фирменную иррациональную борьбу. Если от меня чего-то требуют, то я сделаю наперекор, даже если это во вред моим интересам.\n\nКак говорят в России: «Назло маме уши отморожу». Еще одна русская поговорка, в контексте вашего вопроса превращающаяся в каламбур: «Поживем — увидим».\n\nПоживем — увидим.\n\nСразу надо оговориться, что у меня нетипичный отряд. Большинство заключенных работают в промзонах. У нас же очень небольшое количество людей в бараке, расположенном на отшибе. Все сделано так, чтобы я был под максимальным контролем 24/7. Общался с минимальным количеством зэков, а все, что говорю, сразу становилось известно администрации.\n\nМой нетипичный для зоны распорядок дня выглядит так.\n\nОбычно я просыпаюсь в 5 утра, хотя подъем в 6. Просто этот час с 5 до 6 — единственная возможность спокойно почитать в тишине. Это мой любимый момент дня, и я заранее грущу, что вскоре он исчезнет вместе с сокращением светового дня.\n\nВ 6.00 звонок и крик дежурного: «Отряд, подъем!». Мы должны быстро встать, одеться, идеально заправить кровати и выйти на улицу. Там мы слушаем гимн и делаем зарядку. Потом утренняя уборка помещения. После нее дежурный выдает бритвенные принадлежности, мы умываемся и бреемся.\n\nПримерно в 7.30 мы строимся в локальной зоне — это огороженный забором узкий участок земли вдоль барака, чтобы строем идти на завтрак.\n\nЕдим очень быстро, чтобы к 8 утра вернуться на главное лагерное мероприятие — проверку. Два раза в день мы стоим строем на улице в течение 30 минут, пока идет пересчет заключенных лагеря. Офицер перебирает стопку карточек, выкрикивает твою фамилию. Ты в ответ выкрикиваешь имя-отчество.\n\nДальше первый раунд «воспитания». До 10 утра мы должны смотреть телевизор. Чаще всего это просто какой-то фильм. С 10 до 12 — свободное время. Хочешь — читай, хочешь — пиши, хочешь — сиди на табуретке. На практике все стремятся занять очередь на кухню, она тут называется «комната приема пищи», чтобы выпить чай или кофе и перекусить. Плюс у тебя всегда есть куча бытовых дел: постирать, подшить, погладить. Я стараюсь обязательно хотя бы 15 минут уделить огромному мешку писем. Отвечать просто нереально, но я все внимательно читаю.\n\nПосле 12 некоторых заключенных выводят на работу, а остальные снова обязаны либо смотреть лекции по телевизору, либо играть в настольные игры: нарды, шахматы. Читать, писать и делать что-то еще во время лекций запрещено. Ты должен сидеть на стуле и смотреть в телевизор.\n\nВ 14 часов обед. Потом снова «воспитательные мероприятия».\n\nС 16.30 до 17 — вечерняя проверка, после которой еще одна уборка помещения. Затем короткий перерыв — можно выпить чая или позаниматься спортом, но только если у тебя есть разрешение и ты член «физкультурного кружка».\n\nВ 18 — главная часть программы превращения преступника в нормального гражданина — «патриотическое воспитание». Мы смотрим фильмы о Великой отечественной войне. Или о том, как однажды, лет 40 назад, наши спортсмены разгромили американцев или канадцев. До того, как я попал сюда, и представить не мог, как много фильмов на эту тему снято на деньги госкорпораций. Именно в 18 часов каждого дня мне особенно четко понятна суть идеологии путинского режима: подмена настоящего и будущего прошлым. Реально героическое прошлое, приукрашенное прошлое, полностью вымышленное прошлое. Все виды прошлого должны постоянно находиться в фокусе внимания, чтобы вытеснить мысли о будущем и вопросы к настоящему.\n\nВ 19.30 снова строимся и идем на ужин. После него снова обязательный просмотр лекций, среди которых попадаются интересные. Уверен, астрофизик Нил Деграсс Тайсон пришел бы в восторг, увидев, как лысые мужчины в тюремных робах сидят перед телевизором и смотрят серию научного сериала, где он рассказывает о квантовой спутанности фотонов. А когда кто-нибудь из них засыпает, его будят окриками: «Не спать, смотреть лекцию!».\n\nВроде весь день ничего не делаешь, но к вечеру ты реально без сил и мечтаешь лечь в кровать.\n\nВ 22.00 командуют отбой, и все ложатся спать.\n\nВ 5 утра следующего дня день сурка начинается снова.\n\nСпасибо волшебной силе социальных сетей. Раньше еда была не очень, но я пару раз написал об этом в инстаграме, и она стала существенно лучше. Основной пункт меню — каша. Если получаешь передачи и имеешь возможность покупать продукты в тюремном ларьке — вполне хватает. Если, как здесь говорят, «двигаешься на одной баланде», то, скажем так, это сильно поможет в борьбе с лишним весом. Но с голоду не умрешь, конечно.\n\nВ любой «красной зоне» все держится на «активистах». В моем отряде их аж треть от общей численности. Конфликты были, особенно в тот период, когда я объявил голодовку. Тюремному руководству это очень не нравилось, и они дали приказ «активистам» давить на меня.\n\nЭто было довольно непростое время, и иногда происходящее действительно напоминало дурацкий стереотипный фильм о русской тюрьме. Раньше моя зона славилась страшными избиениями заключенных. Сейчас людей не бьют, по крайней мере, я о таком не слышал, но, как говорится, «сначала ты работаешь на репутацию, а потом репутация работает на тебя». И тем, кому не посчастливилось услышать на этапе: «В Покров», приезжают заметно присмиревшие и напуганные.\n\nЗона специализируется на насилии психологическом. Это куда более изощренно. Тебя не будут бить — наоборот, бесконечными провокациями тебя самого поставят в условия, когда ты вынужден будешь кого-то избить, ударить, угрожать. И тогда дело сделано — везде видеокамеры, и администрация с огромным удовольствием возбудит против тебя новое уголовное дело по факту нападения, добавив к твоему сроку пару лет. Не поддаваться на провокации — это главное, чему ты должен научиться здесь.\n\nПервые месяцы мне это хорошо удавалось, а сейчас просто стало спокойнее. Я вообще решил, что это будет для меня отличный христианский практикум. А то ведь говорим постоянно про любовь к врагам, а поди-ка попробуй понять и простить тех, кого буквально не выносил совсем недавно. Но я стараюсь.\n\nРаспорядок дня! За последние месяцы не было ни разу, чтобы я ложился позже 22 и вставал позже 5-6 утра. Просто в восторге от этого. Наконец-то живу в устраивающий меня период суток. Люблю рано ложиться и рано вставать, но политика везде, а особенно в России, сильно смещена к вечеру. Все только к обеду просыпаются, а встречи вечером часто заканчиваются глубокой ночью. Да и жена с детьми у меня скорее «совы», они всегда приходили в ужас, когда я предлагал им немного пожить в идеальном для меня графике с 5 утра до 9 вечера. Ну и вот я наконец-то нашел место, где можно реализовать давнюю мечту.\n\nЕще как ни странно, мне нравится тюремная готовка. Раньше я никогда не готовил и не испытывал к этому ни малейшего интереса. А здесь я был просто заворожен той креативностью, которая нужна зэку, чтобы разнообразить свою еду в условиях, когда под рукой часто нет ничего, кроме хлеба, кипятка, самой дешевой лапши и банки консервов. А уж когда приходит передача, и в отряде есть микроволновка, начинается кулинарный фестиваль.\n\nКогда мы готовим, я всегда вспоминаю классическую сцену из «Goodfellas», где боссы мафии готовят пасту в тюремной камере. К сожалению, у нас нет такой классной кастрюли, и паста тоже запрещена. Но все равно весело.\n\nСудя по этим вопросам, вы явно загуглили мой лагерь, обнаружили, что он описывается как одна из самых неприятных зон России, и теперь представляете себе место, где татуированные качки с железными зубами устраивают ножевые бои за право занять лучшие нары у окна.\n\nА представлять надо нечто вроде китайского трудового лагеря, где все ходят строем и все утыкано видеокамерами. Постоянный контроль, культ доносительства. Это «красная зона», действительно одна из самых жестких, созданная для борьбы с «воровскими традициями» и арестантским укладом, который является основой жизни на «черных зонах».\n\nИ в нашем лагере жестокость выражается как раз не в избиении и беззакониях, а в буквальном выполнении закона и инструкций. Жизнь по уставу, возведенному в абсолют, совершенно невыносима. Ведь устав для этого и создан. Именно поэтому зэки делают все возможное, чтобы не попасть сюда, в Покров. А этап в СИЗО на какие-то следственные действия воспринимается как поездка на курорт.\n\nЗдесь невозможно представить мобильный телефон, хотя почти во всех зонах они есть. И татуировочная машинка, даже самодельная — это предмет из какой-то другой вселенной. Можно, конечно, набить тату с помощью иголки и чернил из ручки. Но об этом донесут администрации буквально в течение нескольких минут, и оба участника арт-хэппенинга окажутся в ШИЗО.\n\nТак что татуировки я вижу только те, что били на воле. Единственный факт о них, который меня удивил: я не думал, что люди так часто набивают свастики на видных частях тела.\n\nСделать тату себе? Нет уж. Я и так бритый налысо, худой и в робе. Мне достаточно тюремных атрибутов.\n\nБлагодаря огромному количеству замечательных людей по всему миру, организовавших кампанию за оказание медицинской помощи, — я очень-очень благодарен всем им — ко мне допустили гражданских врачей. Жизнь без риска оказаться в инвалидной коляске из-за отказывающих ног гораздо веселее. А уж когда меня перестали будить по ночам через каждый час, проверяя как склонного к побегу, вообще стало хорошо. Меня и сейчас проверяют, но каждые два часа, и теперь честно стараются не будить. Все познается в сравнении. Вы не представляете, как кардинально возрастает качество жизни, когда тебе просто дают поспать. Теперь я отлично понимаю, почему лишение сна — одна из любимых пыток спецслужб. Следов не остается, а терпеть невозможно.\n\nЗа те 7 месяцев, что я сижу, я ни разу не был в месте, где нет видеокамеры. Все бумаги и записи, которые я несу на встречу с адвокатом и выношу со встречи, тщательно перефотографируются. Я ни разу не говорил ни с одним сотрудником без видеосъемки. Для руководства тюремной системы это и гарантия того, что я не буду влиять «разлагающе» на сотрудников и получать поблажки от тех, кто мне политически сочувствует, и удобный инструмент наказания за малейшие нарушения. Поэтому сотрудники просто вынуждены относиться ко мне более строго, ведь в противном случае нарушение, на которое они закроют глаза, станет уже их проблемой. Разговоры со мной потом пересматривает и переслушивает вышестоящее начальство. Забавно, у нас в отряде, в кабинете начальника, даже установили огромный сервер, чтобы хранить эти видеозаписи.\n\nА еще забавнее то, что когда мы в суде истребуем их, чтобы доказать нарушение моих прав, то нам неизменно отвечают: «В результате технической поломки запись была уничтожена».\n\nОбычно в таких местах интервью в скобочках стоит («смеется»). Вы меня сейчас не видите, но поверьте, я смеюсь.\n\nДолгие годы я должен был оправдываться, отвечая на вопросы «почему вас до сих пор не убили?» и «почему вас не посадили?». Теперь, когда у меня в чекбоксах этих пунктов стоят галочки (там, где про убийство, стоит пометка «ну почти»), я должен оценивать вероятность своей смерти в тюрьме.\n\nНу, очевидно, ответ как в анекдоте: 50%. Или убьют, или не убьют.\n\nДавайте не забывать о том, что мы имеем дело с явно психически нездоровым человеком — Путиным. Патологическим лжецом с маниями величия и преследования. 22 года бесконтрольной власти сделают таким любого, и мы наблюдаем классическую ситуацию полубезумного царя.\n\nКак мы сейчас знаем, убийцы из ФСБ стали ездить со мной по стране буквально со следующего дня после того, как я объявил о том, что иду на выборы президента.\n\nУмный ли это ход? Серьезно, отдать приказ своей спецслужбе убить политического противника химическим оружием? Так себе идейка. Но Путин сделал это потому, что одержим какими-то своими страхами и идеями.\n\nВы же знаете, что в Кремле официально трудятся астрологи и шаманы. Путин помешан на оккультизме и ходит с красной ниткой на запястье. Тоже ничего удивительного — типичный признак деградации и разложения при абсолютной власти.\n\nВот мы пытаемся строить какие-то рациональные версии, а на самом деле когда-нибудь выяснится, что президент России принимает решение на основе советов какого-то старца из леса. Или покемона.\n\nПро индикаторы того, снизился или вырос риск моего убийства, могу сказать одно: мои наблюдения свидетельствуют, что это невозможно прогнозировать по внешним признакам. Тюремная система России — хаос такой же, как и все государственное управление. Тут беспрестанно что-то меняется. Приезжающие строгие проверяющие дают твердые указания, которые вскоре отменяются еще более строгими проверяющими. Моя жизнь — это американские горки, где вчера я враг и мне выносят 28 выговоров за месяц, а сегодня оставляют в покое. А потом снова враг. Это даже не похоже на усилия каких-то злодеев. Просто естественная жизнь Левиафана, каждый элемент которого может принимать какие угодно решения, но результат выходит неожиданным, странным, не устраивающим никого.\n\nИз тюрьмы очень хорошо видно, почему ни один «национальный проект» Путина не увенчался успехом. Тот хаос и бардак, в который превращается государственная система, в принципе не способны порождать какой-то результат.\n\nГосподи, да они даже проект «давайте убьем Навального» не смогли реализовать при всех своих возможностях. Впрочем, не хотелось бы, чтобы это стало первым проектом Путина, доведенным до конца.\n\nОтвет на вопрос «считаете ли вы, что в состоянии победить Путина» мы получаем прямо сейчас. Идет кампания по выборам в Государственную думу, и единственные новости о ней — это отстранение кандидатов от участия.\n\nСначала они сняли, объявив экстремистами, всех, кто работал в системе наших штабов. Потом всех, кто поддержал нас любым образом. Потом даже «системных оппозиционеров», которые могут победить при нашей поддержке. А сейчас уже отстраняют даже не самых проходных кандидатов, испугавшись, что наше «Умное голосование» принесет победу над путинскими кандидатами даже им. Поэтому отвечаю твердо и ни капли не сомневаюсь: да. Если бы мы могли участвовать в выборах, то даже без денег и информационных ресурсов разгромили бы путинскую партию «Единая Россия» прямо сейчас. И на федеральных, и на региональных выборах. В крупных городах к этому не надо было бы даже прилагать больших усилий.\n\nПобедить самого Путина лично — тоже да. Не надо преувеличивать его популярность, цифры соцопросов значат мало в авторитарной стране. Путин — это политик, ни разу не участвовавший в дебатах. Он боится неподготовленных вопросов. Вот на днях он выступал перед «рабочими Башкирии», так выяснилось, что все эти рабочие — местные чиновники, на которых надели форменные комбинезоны.\n\nКартинка выборов с реальной конкуренцией — неважно, я бы в них участвовал или другой настоящий оппозиционер — полностью и очень быстро изменила бы настроение избирателей.\n\nРовно это, например, произошло в Беларуси, где жена арестованного оппозиционера за пару месяцев помешала кампании президента Лукашенко с его рейтингами за 80%, которые гарантировала любая социология.\n\nМы предлагаем альтернативу людям прямо сейчас. Наша программа лучше, и у нас есть образ будущего России, полностью отсутствующий у Путина.\n\nПутин не вечен ни физически, ни политически. Он может быть смещен протестами и революцией, отстранен от власти путем переворота, который легко устроит кто-то вроде мэра Москвы Собянина, сконцентрировавшего в руках огромные медиа- и денежные ресурсы, контролирующего половину кадров Кремля. Рейтинги министров Шойгу и Лаврова гораздо выше путинских, и они легко могли бы стать частью внутриэлитного конфликта.\n\nНо главное в том, что режим Путина — это историческая случайность, а не закономерность. Выбор коррумпированной семьи больного Ельцина. Рано или поздно эта ошибка будет исправлена, и Россия пойдет по демократическому европейскому пути развития. Просто потому, что этого хочет ее народ.\n\nДа, мы призываем участвовать в выборах, несмотря на то, что они все больше и больше превращаются в насмешку. Трехдневное голосование, массовое отстранение кандидатов, изгнание наблюдателей. Мы зовем всех на участки по одной причине: наша стратегия «Умного голосования» работает даже в таких условиях. Не везде — в той части страны, где протоколы просто переписывают, не работает ничего, но в крупных городах точно. Мы тестировали стратегию несколько последних лет. В Москве и Петербурге, в городах Сибири — везде она успешна. Впервые за 20 лет мы можем побеждать кандидатов власти по мажоритарным округам. Да, сейчас не идет речь о том, что мы можем избрать хороших кандидатов — все они отстранены от участия — но мы можем избрать не тех, кого запланировал Кремль. Ключевое слово «избрать» — избиратели, а не Путин, сделают кандидата депутатом Госдумы.\n\nЭто снизит размер путинского большинства, сократит монополию «Единой России», сделает политическую систему более сложной.\n\nНу и главное, благодаря «Умному голосованию» волеизъявление гражданина вновь обретает цену.\n\nК колебаниям рейтингов надо относиться спокойно. Это естественный процесс. Несколько месяцев я не сходил с телеэкранов. Люди устали от этого. Политик в изоляции теряет эффективность и через какое-то время выпадает из актуального политического процесса. Собственно, ровно поэтому авторитарные лидеры и сейчас, и на протяжении всей истории человечества бросали своих политических оппонентов в тюрьмы.\n\nПутин решил свой тактический вопрос: не дал нам отнять у него большинство в Госдуме. Тем самым высоко оценив наш потенциал и возможности «Умного голосования». Но для этого ему пришлось полностью изменить политическую систему. Перейти на принципиально другой, гораздо более жесткий уровень авторитаризма. Превратить в своих яростных, хоть пока и молчащих врагов тех из подконтрольных оппозиционеров, кого ему пришлось выкинуть из политической системы вместе с нами. Он лишил свой режим уже даже и призрачных шансов на развитие и прогресс. Он терроризировал, запугивал, но и радикализовывал общество.\n\nВ краткосрочной перспективе он добился своего. В долгосрочной — ударил по стране и гражданам. И они этого не забудут.\n\nМы будем делать то, что делает все живое на планете Земля. Выживать, эволюционировать, изменяться и становиться сильнее. Ровно это я и пишу в письмах к нашим региональным лидерам.\n\nНичего нового сейчас в России не происходит: затянувшийся период реакции авторитарного режима на фоне нагнетания псевдопатриотической истерики и военных авантюр. Классика из учебника истории.\n\nРезко возросшая эффективность наших штабов и появление большого количества ярких лидеров могло иметь два последствия: либо мы добиваемся участия в выборах и побеждаем путинскую партию, либо они объявляют нас вне закона, пока имеют такую возможность.\n\nРиски политической деятельности резко возросли. Многие оказались в тюрьмах или вынужденной эмиграции. Но ведь оппозиция в России существует не потому, что Алексей Навальный или кто угодно другой командует своими штабами. А потому, что 30% граждан страны — образованные горожане прежде всего — вообще не имеют политического представительства. А вот что происходит с остальными 70%: 25% живут за чертой бедности, а 45% — не более чем на 25 тысяч рублей в месяц. 99% не имеют перспектив лучшей жизни при Путине, зато топ-1% владеет более чем половиной всех богатств России. Путин после 22 лет у власти не может предложить стране ничего, кроме фильмов о победах советских спортсменов в 70-х годах прошлого века.\n\nЯ не сомневаюсь в наших региональных сетях и лидерах — это потрясающе смелые, умные и принципиальные люди. Так или иначе, именно они, появившиеся и выжившие под прессингом, будут во многом определять политику России в ближайшие десятилетия. А пока мы ищем пути эволюции и новые формы работы. Обязательно найдем и станем только сильнее. Учебники истории рассказывают и об этом.\n\nБорьба с коррупцией — специализация нашей работы. Я по-прежнему считаю коррупцию одной из главных проблем России, разъедающей страну, лишающей людей перспектив и препятствующей любым реформам. Это основа нынешней власти.\n\nТем не менее я всегда старался, чтобы мои взгляды на то, как должны выглядеть Прекрасная Россия Будущего, были понятны.\n\nМоя страна прямо сейчас готова стать богатым, успешным государством, идущим по европейскому пути развития. Мы особенные, как и любая нация, но мы Европа, и мы Запад. Основой политического устройства должна стать парламентская демократия, а честные выборы, независимость судов и полная свобода СМИ должны быть священными понятиями в новой России. И, кстати, только эти три фактора могут победить коррупцию, а не расследования и публичные судебные процессы, хотя они тоже нужны.\n\nГлавной целью новой власти должен стать рост доходов населения. Русские слишком бедны. Это нищие граждане богатой страны.\n\nИменно бедность — то, что больше всего волнует россиян. Попав сюда, я еще раз в этом убедился. Мой лагерь всего в 100 километрах от Москвы, но уже здесь главные ощущения, разлитые в воздухе: нищета, отсутствие перспектив. Бедные заключенные, курящие одну сигареты на двоих. Бедные сотрудники с нищенской зарплатой. Медсестры тюремной больницы, зарплата которых так мала, что они стесняются ее назвать. Нехватка и утечка кадров. Чудовищные разбитые дороги. А главное — какое-то тоскливое и очень ясное понимание того, что ничего не изменится. Понимание это, к сожалению, приводит не к протестам с требованиями лучшей жизни, а к равнодушной покорности. Но опыт говорит нам, что покорность может легко смениться яростью. Когда я жил в Москве, цены на растительное масло не очень-то меня волновали. А теперь, когда я должен уместить все месячные покупки в тюремном магазине в 9-тысячный лимит, я очень хорошо понимаю, что такая штука, как рост цен на масло на 40%, может и к революции привести.\n\nПобеда над бедностью может быть связана только с экономическим ростом, а он будет обеспечен в первую очередь двумя вещами:\n\n- тотальным дерегулированием экономики и освобождением бизнеса\n\n- инвестициями в человеческий капитал. В образование и здравоохранение. Источник для этих инвестиций — безумно раздутый военно-полицейский бюджет, пожирающий сейчас 35% общего бюджета.\n\nПроще говоря, правительство должно не бегать по миру, усиливая напряженности и войны, потому что его выжившему из ума лидеру хочется поиграть на «великой шахматной доске», а добиваться того, чтобы средняя зарплата была не 500$, как сейчас, а 4500$, как у наших соседей в Германии и Финляндии.\n\nНовые глобальные вызовы говорят нам о том, что Россия должна стать одним из лидеров в борьбе с изменением климата. Наша историческая миссия — сохранение сибирских лесов и пресной воды, важных для выживания всей планеты.\n\nМы живем в уникальное время. Есть серьезные локальные и региональные конфликты, но нет мировых войн. Нет врагов, желающих эти войны развязать. Мы получаем фантастические доходы от экспорта сырья. Этот шанс надо использовать, чтобы в истории первая половина 21 века осталась как период прогресса и богатства для граждан России, а не годами деградации и прозябания, как сейчас.'),
('https://navalny.com/p/6526/', 'Ровно год назад я не умер от отравления химическим оружием, и, по-видимому, коррупция сыграла немалую роль в моем спасении. Разложившая государственную систему, она разложила и спецслужбы. Когда руководство занято крышеванием бизнеса и вымогательством, то качество тайных операций неизбежно пострадает. Группа сотрудников ФСБ нанесла нервный агент на мое белье так же неправильно, как неправильно, с нарушениями всех инструкций, следила за мной в течение трех с половиной лет, что позволило гражданским активистам-расследователям полностью изобличить ее.\n\nЗато более простые задачи режим, построенный на коррупции, выполняет прекрасно. Судебная система — а это первое, что берут под контроль автократы, желающие ограбить свой народ, — идеально работает в формате quid pro quo. Именно поэтому, вернувшись в Россию после лечения, я прямо с самолета отправился в тюрьму. Приятного тут мало, но зато у меня теперь достаточно времени, чтобы почитать мемуары мировых лидеров.\n\nМировые лидеры ужасно интересно описывают, как решали главные проблемы человечества: войны, бедность, миграцию, изменение климата, оружие массового поражения. Это вопросы так называемой «большой повестки». А вот борьба с коррупцией не очень часто упоминается мировыми лидерами в списке того, что они считают своим наследием. Неудивительно — это вопрос «второстепенной повестки».\n\nЗато — поразительный факт — коррупция почти всегда упоминается там, где мировые лидеры описывают провалы. Свои и (чаще) своих предшественников.\n\n«Мы потратили годы, сотни миллиардов долларов и тысячи человеческих жизней на Ирак/Афганистан/Мали/you name it, но коррумпированное правительство аль-Малики/Карзая/Кейты/you name it своим воровством отвратило от себя народ, открыв дорогу к победе радикалам, вооруженным лозунгами о честной справедливой власти и РПГ.\n\nИ тут возникает очевидный вопрос: ну так раз коррупция мешает нам добиваться решения проблем «большой повестки», то, может быть, настало время ее саму поместить на одно их первых мест этой повестки?\n\nВ общем понятно, почему этого до сих пор не произошло. Коррупция — очень некомфортный вопрос для обсуждения на мировых саммитах. Вы обсуждаете, к примеру, с Путиным, Сирию и кибератаки. Все довольны, всем интересно. На брифинге по итогам каждому есть что сказать.\n\nА теперь представьте встречу с Путиным по вопросу коррупции. Сам ее факт — переход на личности. Все от начала до конца — некомфортная ситуация. Самому богатому человеку мира, ограбившему свою страну, предлагается обсудить, как бороться с ним самим. Очень неудобно и неловко.\n\nА теперь включите новости: именно то, что Запад «не замечал» тотальной коррупции в правительствах Карзая и Ашрафа Гани, не желал говорить об этой лично неприятной для них теме, стало важнейшим фактором победы талибов при полной поддержке населения. Не хотели обсуждать воровство из бюджета — будем обсуждать забивание камнями и казни обезглавливанием.\n\nПосле краха СССР и прекращения глобального идеологического противостояния именно коррупция в ее классическом определении — использование служебного положения для личной выгоды — стала универсальной внеидеологической основой процветания в мире авторитарного интернационала. От России до Эритреи, от Мьянмы до Венесуэлы. И коррупция давно перестала быть внутренней проблемой этих стран. Она почти всегда становится одной из главных причин глобальных вызовов, стоящих и перед Западом. Новая настоящая война в Европе с применением авиации и артиллерии? Это Путин мстит Украине за антикоррупционную революцию, сбросившую с трона его ставленника Януковича. Религиозным экстремистам любого толка легче вести пропаганду, когда их противники ездят на Роллс-Ройсах по улицам своих нищих стран. Миграционные кризисы вызваны бедностью, а та почти всегда — коррупцией.\n\n«Хорошо, что изменение климата не связано с коррупцией», — иронично говорите вы. Приглашаю вас сказать это перед лицом миллионов гектаров сибирских лесов, сгорающих ежегодно из-за варварской сплошной вырубки, нарушающей пожарные правила лесопользования. Не хочется угадать с этим прогнозом, но боюсь, что следующий большой теракт будет не чем-то вроде очередной бомбы от религиозных фанатиков, а, скажем, химическим оружием в водопроводе крупного города или разрушительной атакой на ИТ-инфраструктуру целой страны, и заказчиками этого теракта будет тот или иной владелец золотых дворцов. И сделано это будет для того, чтобы отвлечь внимание мира от этих дворцов, переключив его на глобальные вопросы безопасности.\n\nТак что это не нам должно быть неудобно задавать авторитарным коррупционерам жесткие вопросы с переходом на личности, а наоборот — они должны знать, что их теневые дела всегда будут главной темой обсуждения мировых саммитов. Это станет важнейшим шагом к устранению причин многих вопросов.\n\nНу а что же нам делать? Ведь не могут люди, сидящие в Вашингтоне и Берлине, эффективно бороться с коррупцией чиновников Минска или Каракаса?\n\nЭто верно, но верно и то, что важной особенностью коррупции в авторитарных странах является использование западной финансовой инфраструктуры. И в 90% случаев украденное хранится на Западе. Уж кто-кто, а чиновник, работающий на автократа, точно знает, как важно хранить капитал подальше от своих коллег и босса.\n\nВсе, что нужно для начала работы, — решительность и политическая воля западных лидеров. На первом этапе коррупция должна быть превращена из источника потрясающих возможностей в тяжелое бремя хотя бы для части элит, окружающих автократов. Тем самым элиты будут расколоты, а лагерь тех, кто выступает за модернизацию, прогресс и снижение коррупции, расширится, усилится и получит новые аргументы в национальных внутриэлитных дискуссиях.\n\nЭти несколько шагов совершенно реалистичны, легко исполняемы и могут быть очень эффективным стартом в борьбе с глобальной коррупцией.\n\n1. Запад должен выделить и признать специальную категорию — «страны, поощряющие коррупцию», что позволит принимать единые меры в отношении групп стран, а не вводить санкции в отношении конкретных государств.\n\n2. Главной санкцией и, если хотите, налогом на коррупцию для этой группы стран должно стать «принуждение к прозрачности». Вся документация к контрактам, заключенным между западными компаниями и их контрагентами из стран коррупционных рисков должна быть опубликована, если эти контракты в малейшей степени связаны с государством, чиновниками и их родственниками. Ты сотрудник госкомпании страны с высоким риском коррупции и хочешь купить виллу на Французской ривьере? Покупай, но вся информация по сделке будет общедоступной. Хотите иметь дела с официальным Минском или тетей российского губернатора? Нет проблем, но вам придется опубликовать все документы по цепочке сделки, и взятку, которую вы заплатите через «регионального представителя» или «локального партнера», уже не спрячешь.\n\n3. Борьба с коррупцией без борьбы с коррупционерами — лицемерие, подрывающее доверие избирателей к любым действиям в этой области. До тех пор, пока не будут введены индивидуальные санкции против олигархов, в первую очередь, из окружения Путина — морального лидера всех коррупционеров мира, — любая антикоррупционная риторика Запада будет восприниматься как игра и пустословие.\n\nНет ничего более фрустрирующего, чем чтение очередного «санкционного списка», наполненного никому не известными полковниками и генералами спецслужб, но тщательно очищенного от тех, в чьих интересах действуют эти полковники. Запад должен выйти из семантического плена, где лейбл «бизнесмен» является индульгенцией, сильно затрудняющей попадание в санкционный список. Путинские олигархи, как главы госкомпаний, так и формально частные, но благополучие которых связано с группой Путина, — не бизнесмены, а руководители организованных преступных групп. Однако сейчас, увы, западный истеблишмент выступает такой коллективной собакой Павлова. Показываешь полковника спецслужб — кричат: «Под санкции его!». Показываешь олигарха, содержащего полковника, кричат: «Пригласите его в Давос!».\n\n4. В США, Великобритании и Германии уже есть прекрасные инструменты для борьбы с зарубежной коррупцией, такие как Foreign Corrupt Practices Act, Bribery Act и другие. Отгадайте, сколько дел было начато по заявлениям нашей организации, ныне признанной путинским правительством экстремистской? Правильно, ноль. Горькая правда в том, что даже западные правоохранительные органы обеспечивают иностранным коррупционерам режим благоприятствования. Немного политической воли правительства (и давления общественности) — и ситуация исправится.\n\n5. Препятствование экспорту политической коррупции явно заслуживает создания международного органа или комиссии. Посмотрите, что происходит уже сейчас. Инвестировав сравнительно небольшие деньги, тот же Путин пачками скупает крайне правые и крайне левые движения по всей Европе, превращая их политиков в своих олигархов и агентов. Легализованный подкуп через «членство в советах директоров госкомпаний» и подобные — процветает. Бывший канцлер Германии, премьер Италии, министр иностранных дел Австрии выступают на подтанцовках диктатора, нормализуя и покрывая коррупционные практики. Любые контракты, связывающие бывших и действующих западных политиков с контрагентами из коррумпированных авторитарных стран также должны подлежать публикации.\n\nЭто первые шаги, но и они дадут весомый эффект, создав внутри авторитарных стран элитные группы, для которых борьба за снижение уровня коррупции станет рациональным выбором.\n\nНи денег, ни солдат, ни переустройства промышленности и мировой политики не нужно для того, чтобы начать действовать. Только политическая воля. К сожалению, это часто дефицитный ресурс. Общественное мнение и желание избирателей — вот те финальные ингредиенты, которые могут столкнуть ситуацию с мертвой точки. И когда-нибудь мировые лидеры в своих мемуарах напишут о том, что решили многие важные проблемы «большой повестки», просто устранив первопричину их возникновения. Без войск, миллиардов и зря потраченных десятилетий.'),
('https://navalny.com/p/6487/', 'Я хочу сказать, дорогой мой суд, что ваш король голый. И уже не один маленький мальчик кричит об этом, уже миллионы людей кричат об этом, потому что это совершенно очевидно. 20 лет бездарного правления привели к такому результату. У него есть эта корона, сползающая на уши. Есть вранье по телевизору. И мы потратили триллионы долларов от проданных нефти и газа, а наша страна продолжает отставать и деградировать с каждым годом. Неудивительно, что все экономисты российского происхождения объединяются и пишут письмо о том, что последние годы надо называть потерянными десятилетиями.\n\nИ на самом деле мне для осознания этого, как и всем остальным, никакие экономисты даже не нужны. Я сижу в колонии в областном центре, во Владимире, который находится в 200 км от Москвы. Здесь нет ни одной нормальной ровной дороги, здесь зарплаты 30 тысяч рублей! И это 21 век, в нефтегазовой стране, которая претендует на мировое лидерство. И ваш голый проворовавшийся король хочет и дальше править, до конца. Ему наплевать совершенно на страну. Он вцепился в неё и хочет держать эту власть бесконечно. Но если он и будет править дальше, на смену потерянному десятилетию придет украденное десятилетие.\n\nИ поскольку это всем понятно и все больше людей видит, что король голый, вот этот голый король осознает, что он ничего сделать не может. Он не может сделать ничего, чтобы страна жила лучше. И он судорожно ищет какие-то, знаете, кусочки святости, кусочки чего-то хорошего, чтобы присвоить их себе и украсть. Присвоить и использовать для своих личных целей. И вот он уже много лет делает это с победой нашего народа в Великой отечественной войне. Он пытается её присвоить себе, объявив, что чуть ли не он победил в этой войне, и торгует ей направо и налево. И вот этот процесс об этом: как ваш голый король украл знамя Победы и пытается сделать из него себе стринги.\n\nНо по сути вся ваша власть — это власть тех самых оккупантов и предателей. Я считаю предателями всех вас. Вы во главе с вашим голым королем реализуете ровно тот план, про который мы все читали в учебниках истории. О том, что Россия должна быть захвачена, а русские должны быть превращены в рабов, у которых будут забраны их богатства. Которые будут лишены образования, которые будут лишены здравоохранения, которые будут лишены любых перспектив. Ровно это вы и сделали, вы реализовали тот план. У нас каждый год 200 тысяч человек выезжают за границу. У нас население сокращается сейчас миллионами. Перестаньте меня перебивать! Это мое последнее слово. Оно по материалам дела. Потому что материалы этого дела отлично показывают суть вашей власти.\n\nИ то, что происходит прямо сейчас, когда прокурор Москвы, о котором я лично делал расследование. О его виллах в Испании и гостиницах в Черногории. Я снимал номер в его гостинице в Черногории. И этот прокурор, госслужащий, здесь ворущий миллионами и инвестирующий за границу, пытается в закрытом процессе признать экстремистами и иностранными агентами меня и таких как я, патриотов страны, защищающих нашу страну от вас, предателей. Я хочу сказать, что как бы вы ни прикрывались и ни пытались украсть эту святость и стащить знамя Победы из музея и использовать его для себя, у вас ничего не получится. Ваша оккупационная власть рано или поздно будет отстранена...\n\nВаша честь, перестаньте меня перебивать. Я понимаю, почему вы перебиваете, вы знаете, что это все правда. И я к вам очень добр, с учетом того, что, я повторяюсь, считаю вас всех предателями и оккупационной властью. Но Россию много раз пыталась захватить какая-то оккупационная власть, оккупационная администрация. И ваша — довольна успешная, потому что всё-таки в течение 20 лет вам удавалось грабить наш народ, живущий в этой стране, лишать его будущего и вывозить принадлежащие его богатства за границу. Но это рано или поздно закончится. У меня все!'),
('https://navalny.com/p/6477/', 'Знаете, что оказалось самым важным на первом этапе тюремной голодовки? Проверять карманы.\nВ них подкладывают конфеты. Первую нашли на обыске и сразу ироничные улыбки: А что это у вас, Алексей Анатольевич, конфетка?\nС того момента перед тем, как надеть любую вещь, бывшую без присмотра, я обыскиваю ее и выкидываю конфеты 🤷‍♂️\n\nЯ знал, конечно, что власти в первую очередь захотят дискредитировать голодовку и высмеять ее. Удивляет только примитивность подхода.\nОчень правильным решением было с первого дня голодовки вообще не заходить на кухню без сотрудника с включенным видеорегистратором. Попить можно только на кухне. Я захожу, пью из «бачка для питьевой воды», меня в этот момент снимают, я выхожу. И ни к каким сумкам не подхожу один.\nПоэтому, когда на всех каналах кремлевских СМИ вышли репортажи: «Навальный тайком ест печеньки», и ЗЭК, и даже сотрудники сказали: ну и вранье.\n\nВсе видят, что у меня на 100% хардкорная тюремная голодовка. Без всяких фокусов вроде сладкого чая, подсоленой воды, сока, витаминов и т.д. Только вода.\n\nИ история вам:\nРазговариваю с адвокатом на днях.\nОн: они обычно подсаживают к голодающему человека и загоняют ему море курицы. Чтоб жарил и ел ее на глазах у протестующего.\nЯ смеюсь: не, это слишком, да и невозможно у нас в принципе. На кухне готовить запрещено и не на чем. Только чайник есть. А курица вообще под тотальным запретом. Ее ни купить, ни в передаче получить нельзя.\n\nИ что вы думаете? Через два дня захожу на кухню попить, там стоит радостный «дневальный» и жарит курицу! Плитку притащили! Потом хлеб начали жарить и еще что-то. И так теперь целыми днями. Все правила и распорядки отменили. Пир и либерализация!\n\nЯ ухмыльнулся и подумал, что это у них инструкция такая. А потом подумал еще и понял: нет, это просто то, во что они свято верят. Суть этой власти.\nНикто и никогда не будет отстаивать свои принципы и бороться за свои права, если — вот же она, совсем рядом, вкусная жареная курочка.\n\nВся банда воров от Путина и его министров до холуев внизу системы искренне не верят, что выбирая между идеями и курицей, кто-то выберет идеи. Еда, деньги, роскошь, автомобили и дворцы соблазнят любого. Кто заявляет обратное — прикидывается.\n«Пойдем, Лех, там много еще. Похаваем с пацанами. Слышишь...ммм...какой запах. Брось ты, ничего не добьешься же».\n\nСпасибо, конечно, но я, знаете ли, не готов покорно взирать на наглую рожу, живущую на деньги налоголательщиков и говорящую мне:\nВаш диагноз мы вам говорить не обязаны\nРезультаты вашего МРТ для нас, а не для вас\nНет, ваши лекарства не передадим, можете сколько угодно тыкать в пункты закона о том, что имеете право пригласить врача. Сюда никто не приедет, и ничего вы не добьетесь.\nВ этот момент я понимаю: а ведь прямо сейчас то же самое выслушивают десятки тысяч людей. И в тюрьмах, и на воле тоже. Они буквально умирают без медицинской помощи. Они не известны, их никто не защитит, никто о них не вспомнит, никто не бросит за них вызов этой лживой бесчеловечной системе.\n\nИ сразу выигрываю эпический бой, в которой мой дух противостоит тюремной курице.\nНу и вы тоже не сдавайтесь в важных битвах 😉'),
('https://navalny.com/p/6469/', 'Так часто я выступаю с последним словом! Вот и сейчас наш суд закончится, а потом у меня следующий суд, и там тоже последнее слово. Наверное, если кто-то захочет издать мои последние слова, это получится довольно толстая книжка. Мне кажется, в этом есть какой-то такой своеобразный сигнал, который посылает мне вся эта власть в целом и владелец замечательного дворца Владимир Путин. Все это выглядит странно, но мы так можем, смотри, можем так сделать. Знаете, как жонглер или фокусник, в суде вот он мячик крутит на этом пальце, потом раз — на этом пальце, потом на ноге, потом на голове. И они говорят: «Смотри, мы вот эту судебную систему можем крутить на любой части своего тела. Куда ты против нас лезешь? Мы можем все что угодно, вот смотри, вот прямо так».\n\nНо, если честно, мне кажется, что такая вот их бравада… Ну, то есть, сделать-то они, конечно, со мной могут да, и делают. Но в общем-то, вижу-то это не я один. А обычные люди, которые смотрят на это, они конечно, на них это тяжелое впечатление производит. Потому что каждый же думает: «Ага, а если я столкнусь с судебной системой? А какие у меня шансы здесь вот чего-то добиться?»\n\nТем не менее, последнее слово, значит, нужно говорить последнее слово. Я уже не знаю, о чем говорить-то, ваша честь? Хотите, с вами поговорим о Боге? И о спасении. Выкручу, так сказать, ручку пафоса на максимум. Дело в том, что я верующий человек. Что, в общем-то, служит предметом для постоянных насмешек в Фонде борьбы с коррупцией и вокруг меня. Потому что в основном люди атеисты, я сам когда-то таким был, довольно воинствующим. Но вот сейчас я человек верующий, и мне это сильно помогает, на самом деле, с моей деятельностью. Потому что все становится гораздо, гораздо проще. У меня, знаете ли… Я меньше раздумываю, и меньше дилемм в моей жизни, потому что, ну есть какая-то там, знаете ли, книжка, в которой, ну, в общем-то, более или менее четко написано, что нужно делать в каждой ситуации. Ну не всегда, конечно, легко следовать этой книжке, но, в общем-то, я стараюсь. И поэтому, как я уже сказал, мне легче, чем многим другим, заниматься политикой в России. И тут мне недавно пишет человек: «Навальный, а чего тебе все пишут „держись“, „не сдавайся“, „терпи“, там, „сожми зубы“? Чего тебе терпеть-то? Ведь ты же вроде в интервью говорил, что в Бога веришь, а сказано ведь: „Блаженны жаждущие и алчущие правды, ибо они насытятся“. Ну так вот и отлично тебе, значит». И я подумал — надо же, как человек хорошо меня понимает. Потому что ну не то чтобы мне отлично, но вот эту конкретную заповедь я всегда воспринимал как такую более-менее инструкцию к деятельности.\n\nИ поэтому, ну, безусловно, не очень наслаждаясь тем местом, где я нахожусь, тем не менее не испытываю никаких сожалений о том, что я вернулся, о том, что я делаю. Ну, потому что я сделал все правильно. Наоборот, я испытываю, ну, такое, удовлетворение, что ли? Потому что в какой-то сложный момент я делал как положено по инструкции и заповедь не предал. И важная вещь об этом. Ну, безусловно, для современного человека вся эта заповедь — «блаженные», «жаждущие», «алчущие правды», «ибо они насытятся» — она звучит, конечно, очень напыщенно. Звучит странновато, честно говоря. Ну и люди, которые говорят такие вещи, предполагается, ну, говоря откровенно, выглядят сумасшедшими. Сумасшедшие странные люди, сидят там с всклокоченными волосами в своей камере и пытаются себя подбодрить чем-то, хотя они одиноки, они — одиночки, потому что они никому не нужны. И это вот важнейшая вещь, которую и власть наша, и вся система пытается сказать таким людям: ты одинок. Ты — одиночка. Сначала важно запугать, а потом доказать, что ты один. Ну, потому что какие нормальные люди, ну мы же нормальные люди, адекватные люди, ну придерживаться какой-то там заповеди, господи боже мой? И вот эта вот вещь про одиночество — она очень важна, она очень важна как цель власти. Кстати говоря, по этому поводу сказала один из замечательных философов — Полумна Лавгуд, помните, была такая в «Гарри Поттере»? И, разговаривая с Гарри Поттером в сложные времена, она ему сказала: «Важно не чувствовать себя одиноким, потому что на месте Волан-де-Морта я бы очень хотела, чтобы ты чувствовал себя одиноким». Безусловно, наш Волан-де-Морт во дворце тоже этого хочет.\n\nЗнаете, конвой — замечательные ребята в моей тюрьме, они же нормальные, но они со мной не разговаривают. Им запретили со мной говорить, видимо. Они говорят такие дежурные фразы. Это тоже такая важная вещь, чтобы ты постоянно чувствовал себя одиноким. Но я совершенно не чувствую. И я объясню, почему. Потому что эта конструкция — «блаженны жаждущие и алчущие правды, ибо они насытятся» — кажется экзотической, странноватой, но на самом деле это главная политическая идея, которая сейчас есть в России. Ваша честь, какая существует в России фраза, формулировка самая популярная политическая? Какой самый популярный лозунг в России? Помогите мне кто-нибудь. В чем сила? Правильно, сила в правде — вот эту фразу, которую повторяют все, это же оно и есть. Это и есть та самая заповедь блаженства, только без всякого «ибо, алчущие». Она просто сжата до размеров Твиттера. И вся страна на все лады повторяет, что сила в правде. За кем правда, тот и победит. И это очень важно. И несмотря на то, что страна наша сейчас построена на несправедливости (мы постоянно сталкиваемся с несправедливостью, худший вид несправедливости — несправедливость вооруженная), но тем не менее мы видим, что одновременно с этим миллионы людей, десятки миллионов людей, они хотят правды, они хотят добиться правды и рано или поздно они ее добьются, они насытятся.\n\nВсем же очевидно: ну вот есть дворец. Ты можешь говорить, что он не твой или его нет, но он есть. Есть нищие люди. Ты можешь говорить сколько угодно, что у нас высокий уровень жизни, но страна бедна, и это видно всем. Они должны быть богаты. Вот нефтепровод построили, деньги зарабатывают, а денег нет. И это правда, и против нее не попрешь. И рано или поздно люди, которые хотят правды, — они добьются своего, они насытятся.\n\nИ важная вещь, которую я вам хочу сказать. И в вашем лице — вас, прокурора, — вообще всей власти. Всем людям: важно не бояться этих людей, не бояться тех, кто добивается правды. Потому что многие опасаются : о боже мой, что будет, будет революция, будут кошмары и потрясения. Но подумайте сами, как хороша была бы жизнь без постоянного вранья, без этой лжи. Возможность не врать — это же просто восхитительная ситуация. Подумайте сами, как было бы классно работать судьей, и чтобы вообще не было никакого телефонного права, никто тебе не звонит, и ты просто такой классный судья с большой зарплатой, даже больше, чем у вас сейчас, ты уважаемый столп общества, и никто не может тебе никуда позвонить, никто не может дать тебе указания, как решать дела. И ты приходишь к своим детям и внукам, рассказываешь, что, да, ты — реально независимый судья. И все остальные судьи абсолютно независимые. Классно же было бы, просто было бы отлично быть прокурором, который на самом деле выступает в состязательной системе, ведет интересную такую юридическую игру, защищает кого-то или осуждает каких-то реальных злодеев. И вряд ли, я думаю, что люди поступали на юридический факультет и становились прокурорами, чтобы потом участвовать в фабрикации уголовных дел и подделывать за кого-то подписи. Я не верю, что для этого люди хотят стать прокурорами. И я не верю, что люди хотят стать полицейскими, чтобы потом рассказывать: «А классно мы там на митинге кому-то раскроили голову! Или вот мы конвоировали чувака, а он невиновный. Мы слушаем его предстоящее последнее слово». Никто не хочет этого! Никто не хочет быть таким, все хотят быть нормальным полицейским. Потому что от этого вранья одни минусы, плюсов-то никаких нет. Даже зарплату больше не платят. Минусы есть, а плюсов нет. И для всех, от бизнеса… Вот бизнесмены. Любой бизнес в стране стоит в два раза дешевле, потому что нет судебной системы, потому что несправедливость, потому что какой-то бардак, нищета. И конечно, всем было бы гораздо лучше, если бы этого вранья и несправедливости не было. Было бы гораздо лучше, если бы те люди, которые хотят правды, добились бы этой правды. ФСБшники — это то же самое. Никто, ни один человек на свете не был школьником с горящими глазами, который говорил: «Пойду в ФСБ, и меня там будут отправлять, чтобы я стирал трусы оппозиционеру, потому что кто-то их намазал ядом». Нет таких людей! Никто не хочет этого делать! Все хотят быть нормальными уважаемыми людьми, ловить террористов, бандитов, шпионов, бороться с этим со всем.\n\nИ это очень важно — просто не бояться людей, которые добиваются правды, а может быть, даже как-то их поддержать. Прямо, косвенно, или просто, может быть, даже не поддержать, а хотя бы не способствовать этой лжи, не способствовать этому вранью, не делать мир хуже вокруг себя. В этом есть, конечно, небольшой риск, но, во-первых, он небольшой, а во-вторых, как сказал другой выдающийся философ современности по имени Рик Санчез, «жизнь — это риск. А если ты не рискуешь, значит, ты просто вялая кучка молекул, собранных беспорядочно и плывущих по течению Вселенной».\n\nПоследнее, что я хочу сказать. Мне приходит сейчас куча писем. И каждое второе письмо заканчивается фразой «Россия будет свободной». Это классный лозунг, я его тоже постоянно говорю, повторяю, пишу в ответ, скандирую на митингах, но я постоянно думаю: чего-то мне не хватает. То есть я хочу, конечно, чтобы Россия была свободной, это необходимо, но недостаточно. Это не может являться целью само по себе.\n\nЯ хочу, чтобы Россия была богатой, что соответствует ее национальным богатствам. Я хочу, чтобы эти национальные богатства справедливее распределялись, чтобы все получали свою долю от нефтяного и газового пирога. Я хочу, чтобы мы были не только свободными, но и, знаете ли, с нормальным здравоохранением. Чтобы мужчины доживали до пенсионного возраста, потому что сейчас у половины мужчин в России этого не получается сделать, да и у женщин ненамного лучше. Я хочу, чтобы образование было нормальное и люди могли нормально учиться. Я, конечно, хотел бы, чтобы за одну и ту же работу в России получали такую же зарплату, как в средней европейской стране, потому что сейчас гораздо меньше. Любой: полицейский, программист, журналист, кто угодно, — все получают гораздо меньше.\n\nЯ хотел бы, чтобы много других вещей случилось в нашей стране. Нужно бороться не столько с тем, что Россия несвободная, а с тем, что в целом она по всем направлениям несчастная. У нас все есть, но тем не менее мы какая-то несчастная страна. Русскую литературу откройте, великую русскую литературу, — да боже мой, одни описания несчастий и страданий. Мы очень несчастная страна и из круга этого несчастья не можем вырваться. А, конечно, хочется. Поэтому лозунг я предлагаю изменить и говорить о том, что Россия должна быть не только свободной, но и счастливой. Россия будет счастливой.\n\nУжасно смешно, что снова последнее слово. Весь день прокурор рассказывала, что я требую какой-то исключительности для себя. Мне кажется, то, что у человека может быть два последних слова в один день, — это тоже какая-то исключительность, которой я для себя не требовал, но она у меня есть.\n\nДля последнего слова у меня есть бумажка, вот эта. Она довольно задрипанная, потому что я ее в кармане ношу каждый процесс. И когда я в СИЗО выхожу, меня обыскивают и каждый раз говорят: «Это что такое? Нельзя, можно только материалы дела». А я говорю — это мой материал дела самый главный. Помните, в самом начале вы нам дали полчаса на ознакомление с материалами дела, и я посмотрел классифицированные материалы по делу, описи. Единственная вещь, единственная вещь, которая меня интересовала, я из нее сделал выписку.\n\nИ потом в течение процесса я ее из кармана доставал и украдкой смотрел. Каждый раз в самые такие пафосные моменты, когда прокурорша наша изображала здесь слезы и надрывалась, говорила: «Ветераны, это же наше все, наше государство ради ветеранов и существует. И только мы их любим, а такие, как вы, значит, их оскорбляете».\n\nКогда вы, ваша честь, говорили: «Не смейте. Ветераны — это самое главное. Все в России для ветеранов». И даже потом, когда в своей камере я телевизор включал, там рассказывали про то, как я оскорбил ветерана. Ветеран, конечно, самое важное, что только есть в стране, и вся Россия существует для ветерана. Я снова доставал эту бумажку и смотрел на нее, а это листочек — справка из отдела социального обеспечения, и они там пишут, какую помощь они оказали ветерану Артеменко.\n\nВот вам справка за последние четыре года. Там указано, что последний раз он получил продуктовый набор 06.07, а всего за последние четыре года он получил помощь семь раз. Он три раза получил сертификаты на проднаборы, один раз набор продуктов, и три раза он получил сертификаты на продукты. И если я правильно посчитал, а мне кажется, я правильно посчитал, общая стоимость этих сертификатов — примерно 10 тысяч рублей. Вот и вся ваша забота, ё-мое, о ветеранах, понимаете, лицемерные вы негодяи! Вот в этом и выражается все настоящее отношение к ветеранам. К пожилым людям, ко всем, как это происходит в стране.\n\nЕще одна страница материалов дела — это состав следственной группы. 15 человек из разных регионов, 15 бездельников и охламонов, свезли в Москву, сняли им жилье, дали им командировочные, повышенную зарплату, для того чтобы они расследовали это «преступление», понимаете. На это ушло больше денег, чем ветеран получил от государства помощи за всю жизнь, наверное. Один день этого суда, посчитайте, сколько здесь стоит госслужащих, один день этого суда стоит гораздо, гораздо больше, чем ветеран, участник войны Артеменко получил за последние четыре года помощи от государства, которое осмеливается заявлять о том, что оно заботится о ветеранах. Самое отвратительное, гнусное, омерзительное, что вот есть в этой власти, — это то, что она существует для того, чтобы грабить этих людей. Самых несчастных людей — ветеранов, этих пенсионеров. Они же источник этих денег. Откуда же дворцы берутся и ершики стоимостью там по 50 тысяч рублей за штуку? Мы знаем, что деньги не берутся ниоткуда. Они взяты оттуда. Они украдены у этого ветерана. Кого-то не долечили, кого-то не доучили, кому-то не купили инвалидное кресло, кому-то не дали лекарств — для того чтобы построить дворец, для того чтобы тещам Путина купить квартиры, для того чтобы обеспечить, чтобы у Медведева было четыре дворца.\n\nИ каждому нужно дать, но чтобы дать, нужно у кого-то отнять. И они отнимают, но для того, чтобы защитить свою возможность воровства. Они используют тех, кого обокрали. Вот это и есть самое отвратительное. Они каждый божий день, вы каждый божий день обкрадываете этого ветерана, а потом его же и вытаскиваете, трясете им и говорите: «Мы будем у власти всегда, потому что мы его защищаем». И это просто реально чудовищно.\n\nВот здесь выступал такой чучундрик, который написал на меня заявление. И он сказал лучшую фразу за весь процесс, он говорит: «Ну я смотрю этот ролик, и там человек пожилой, и видно, что он бедный, значит, он пенсионер и ветеран». А что же он бедный-то? А почему в нашей стране для него это абсолютная ассоциация у любого? Ну ветеран, ну, значит, бедный. Ветеран войны — значит, очень старый и, скорее всего, нищий. И конечно, человек видит сразу же, у него просто картинка совпадает. Бедный — это значит ветеран. Почему же, я не понимаю?! Если это участник войны, и вы здесь наворовались все и рассказываете, как вы их любите, как вы их защищаете, почему я прямо просто сразу с первого взгляда вижу, что он бедный? Почему, если вы возьмете все страны антигитлеровской коалиции, ладно страны антигитлеровской коалиции, давайте возьмем тех, кто проиграл войну: Италия, Япония, Германия. Значит, в России — самая маленькая пенсия у солдата-победителя. Самая маленькая, и вы ее не хотите повышать! Я писал такой закон и просил, чтобы повысили им пенсию, для солдатов, [как] для солдатов немецкого бундесвера. Нет, нельзя повысить!\n\nПотому что не хватит на дворец. Потому что деньги не берутся ниоткуда. Для того, чтобы в Геленджике или на южном берегу Франции возникали эти классные дома единороссов, нужно обокрасть кого-то. И, конечно, лучше всего, удобнее всего обокрасть ветерана или пенсионера. Наша власть, вы знаете, от Путина до «Единой России» и вот до вас всех, она превратилась в такую, знаете, огромную свинью, которая из корыта, наполненного деньгами, нефтедолларами, хлебает, засунув туда свою голову, а когда ей стучат под ухом и говорят: «Алле! Это же здесь для всех!» А вы голову засовываете и говорите: «Что? Не позволим оскорблять ветеранов!» — и обратно голову. И все говорят: «Может быть, хватит красть?» Вы снова свою эту вытаскиваете физиономию и говорите: «Что? Не позволим пересматривать итоги Второй мировой войны!»\n\nДа какое вы, ваш Путин и ваша «Единая Россия» имеете вообще отношение к этой войне? Просто послушайте: «Мы победители», понимаете? Из окопов не вылезали.\n\nВы используете это, потому что вам не хочется говорить про настоящее. Вы используете это, потому что очень неловко обсуждать любую текущую проблему. Потому что с вами же, опять же, хотят поговорить про коррупцию, про бедность, про неравноправие, про развал здравоохранения, а сказать об этом нечего. И когда конкретные люди типа меня начинают задавать слишком острые вопросы, сразу возникает: «Про какой дворец? Какой там дворец, какой Димон, какой Путин? Давайте поговорим о том, как он оскорбил ветерана!» И ровно для этого придумали этот процесс. Потому что не хочется вам говорить ни о чем другом.\n\nВот здесь прокурор... Один из самых, конечно, гадких моментов процесса — когда прокурор читала вот эти вот сфабрикованные показания. Ну все было понятно нам, десять страниц показаний, они же, подумайте, сначала собрались и сказали: «Ну, нужны теперь показания, которые он будет читать в суде». Но дедушка старый, и уже, конечно, он говорить не может. «Ну ладно, давай, черт с ним, возьмем его старые мемуары, подпишем за него». Посмеялись же еще над дедом: «А чего, дед старый, ему все равно». А потом она читала, делала паузы и, значит, изображала, что голос у нее дрожит.\n\nА я вот хочу напомнить, например, о недавнем таком важном эпизоде в жизни нашей страны, о котором я упоминал в ходе этого процесса, но никогда о нем не поздно будет вспомнить еще раз. Когда в Краснодарском крае под непосредственной крышей бывшего генпрокурора Чайки, который до сих пор работает в прокуратуре, и его окружения, которое до сих пор работает в руководстве прокуратуры, действовала банда Цапка, которая терроризировала целую станицу, насиловала школьниц, убивала людей. А поймали их только после того — потому что их крышевала прокуратура и суды, — только после того, как они вырезали целую семью огромную, убили взрослых и детей, сложили их во дворе, подожгли — это официальные материалы дела, — а новорожденного ребенка живого бросили в этот костер. Понимаете? И про этот эпизод... Это все случилось, потому что за ними стояла прокуратура, за ними стояли суды, за ними стояла «Единая Россия». Это члены партии «Единая Россия» все делают. И этот эпизод, понимаете ли, не хотите зачитывать. И голос у вас не дрожит, хотя это самое настоящие фашистские преступления. Голос у вас не дрожит, а наоборот, очень гордым голосом вы говорите: «Хм, а кто последний в кассу?» И вы готовы обслуживать вот этого огромного хряка, который засунул свою морду в общие нефтедоллары, потому что надеетесь, что вам перепадут какие-то крошки.\n\nЯ включаю телевизор и вижу репортаж, стыдный репортаж. Это же прямо показывают тот эпизод, когда, значит, ветеран узнал о том, что его оскорбил Навальный, и ему стало плохо. И так случилось удивительным образом, что в этот момент, конечно, дома у ветерана была съемочная группа, которая снимает его камерой. Лежит этот человек — и это показывают, я вижу, по телевизору, — он лежит в трусах на одеяле, рядом внук стоит, и в глазах его счастье, и он доволен, и у него прямо на лице написано: боже мой, я наконец-то монетизировал деда. Получилось. Он снова оказался полезен нашей семье. Все счастливые, камеры снимают, за камерой, наверное, стоит Маргарита Симоньян, которая говорит: давай, дед, не рыпайся, лежи. А этот 95-летний человек лежит в трусах на одеяле, и у него на лице написано: что вообще происходит? Почему они это все делают? А они: «Ну, нормально! Нормально, так трогательнее получится!»\n\nВы всем этим процессом, цель которого понятна совершенно, вы реально унизили и оскорбили вот этих всех ветеранов в десять раз вообще хуже, чем вы все делали до этого. Вы реально унижали достоинство, использовали человека как куклу, который абсолютно не втупливает... он то хочет выступать, то не хочет выступать. Да вы просто открыто подписываетесь за него. То есть он в вашем представлении вообще не человек. Ну вы понимаете, живой человек, дееспособный — как же можно взять и за человека взять и подписать десять разных заявлений просто разным почерком? Потому что для вас он вообще не человек. Для вас и для вашей власти это просто механизм, кукла, которую вы хотите использовать для себя.\n\nЯ начал с этого свое выступление в первый день и продолжу: да, я, безусловно, считаю, что за это за все вы будете гореть в аду. Я надеюсь, вы люди более-менее молодые, вы не только в аду будете гореть, но еще и ответите перед нормальным человеческим судом за то, что вы сделали. Я понимаю, что вы устроили. Чтобы один суд, вступило в силу решение суда, сейчас другой суд, просто для того чтобы вечером сказать, что вот этот вот суд, который был последний, и вот вы снова... его посадили на 3,5 года, а сейчас вы меня еще раз посадите на 3,5 года, чтобы сказать: «Навальный сидит не потому что скрывался от следствия, а потому что оскорбил ветерана». И вы будете дальше ездить, что называется, по ушам телезрителей всякой своей ерундой. И ваш замысел понятен, и, безусловно, какое-то количество вашей телепублики вы обманете, но я уверен и знаю просто это, что ваш план не сработает. Огромное количество людей, которые следят за этим процессом, — они же видят, как это все происходит. Им так же противно и тошно, как и мне, потому что они, в отличие от вас, считают ветеранов, пенсионеров и кого угодно людьми, нормальными людьми, над которыми нельзя издеваться, которых нельзя сначала притаскивать для участия в ваших гадких роликах, потом за них нельзя подписываться, подделывать заявления, нельзя их вот так вот снимать в беспомощном состоянии. И то, что вы задумали, я уверен, у вас не получится, и так или иначе правда возьмет свое, и каждый ответит по заслугам. Спасибо.'),
('https://navalny.com/p/6463/', 'Привет, это Навальный. Я, конечно, много раз слышал эту фразу: «Невозможно лишить свободы того, кто свободен внутри». И вот сейчас сам себя честно спрашиваю: «Ну как, работает?». Поверьте, работает.\n\nЖелезные двери за мной захлопываются с оглушительным лязгом, но я чувствую себя свободным человеком. Благодаря уверенности в своей правоте. Благодаря вашей поддержке. Благодаря поддержке своей семьи.\n\nДемонстративное беззаконие, происходящее со мной с момента, как я сошел с трапа самолета, — это месть Путина лично мне. За то, что выжил, за то, что осмелился вернуться. Теперь еще и за то, что рассказал о его дворце. Но в еще большей степени это послание Путина и его друзей всей стране: «Видали, как мы можем, а? Наплюем на законы и свернем в бараний рог любого, кто осмелится идти против нас. Мы и есть закон.»\n\nИ я в очередной раз (понимаю, надоел, ну а что мне еще остается?) призываю: не дайте себя запугать. Не дайте обмануть себя с помощью иллюзии силы и крутизны.Какая же это сила, если им приходится тайком травить людей и беззаконно их сажать? Где их всенародная поддержка, если выборы они могут выиграть, только не пуская на них независимых кандидатов, а страх перед протестующими вынуждает их закрывать станции метро?\n\nОни могут удерживать власть, используя ее для личного обогащения, лишь опираясь на наш страх. Зато мы, поборов страх, можем освободить свою Родину от кучки воров-оккупантов. И сделаем это. Обязаны сделать. Для себя и будущих поколений.\n\nПравда на нашей стороне. Оставайтесь свободными.'),
('https://navalny.com/p/6462/', 'Я хотел бы начать с обсуждения правового вопроса, который мне кажется главным и немножко упущенным из этого обсуждения. Потому что выглядит все немножко странновато. Вот сидят двое, и один из них говорит: а давайте посадим Навального за то, что он являлся не по понедельникам, а по четвергам. А второй говорит: а давайте посадим Навального за то, что он, выйдя из комы, немедленно не приехал к нам в инспекцию. И идет это обсуждение, все обсуждают понедельники, четверги, когда кто должен был отправить какую бумажку и так далее.\n\nНо я хотел бы сказать пару слов о таком небольшом слоне в этой комнате. Я хотел бы, чтобы все еще раз обратили внимание, вся пресса, которая пишет про этот процесс, все люди обратили внимание на то, что суть дела заключается в том, чтобы меня посадить по делу, по которому я был уже признан невиновным, по делу, которое уже признано сфабрикованным.\n\nЭто не мое мнение, потому что, если мы откроем любой учебник по уголовному праву — я надеюсь, ваша честь, вы это делали пару раз в жизни, — мы увидим, что Европейский суд по правам человека является частью судебной системы России, потому что Россия является членом Совета Европы, и эти решения обязательны. И я, пройдя все необходимые стадии судебного процесса, обратился в Европейский суд, и Европейский суд вынес решение, в котором черном по белому написал,что даже состава преступления [в деле «Ив Роше] нет. Дело, по которому я нахожусь в этой странной клетке, полностью сфабриковано. Мало того, Российская Федерация, в общем-то, признала это решение — половинчато. Мне даже выплатили компенсацию по этому делу, тем самым признав решение ЕСПЧ.\n\nНесмотря на это, мой брат три с половиной года отсидел в тюрьме по этому делу, которое было, еще раз, признано сфабрикованным решением суда, обязательным на территории России. Я по этому делу просидел год под домашним арестом. Когда мой испытательный срок заканчивался, меня за неделю до этого арестовали, привезли в Симоновский суд и без защиты, всучив какого-то адвоката по назначению, продлили мне его еще на год.\n\nДавайте немножко математики. В 2014 году меня осудили, дали мне 3,5 года, дали испытательный срок, а сейчас 2021 год, но меня все равно продолжают судить по этому делу. Меня уже и невиновным признали, и состава преступления в этом деле нет, и все равно с упорством маньяка наше государство требует меня посадить по этому делу.\n\nПочему же по этому делу? Уж чего-чего, а недостатка в уголовных делах в отношении меня точно нет. Еще одно совершенно недавно возбудили. Тем не менее кому-то очень хотелось, чтобы я ни одного шага не сделал по территории нашей страны, вернувшись, как свободный человек. Чтобы с момента пересечения границы я оказался арестантом. И мы знаем кому. Мы знаем, почему это случилось. Причина этого всего — ненависть и страх одного человека, живущего в бункере. Потому что я нанес ему смертельную обиду тем, что выжил после того, как меня пытались убить по его приказу.\n\n(Прокурор Фролова просит сделать Навальному замечание).\n\nМне не нужно ваше замечание, ваша честь! Мне теперь этот прокурор будет мешать договорить свое отношение ко всему тому, что происходит?\n\n(Прокурор спрашивает, какой вопрос сейчас рассматривается; адвокат Михайлова просит сделать замечание прокурору).\n\nЯ говорю по представлению, ваша честь. То, что такой суд по представлению, — это мое мнение. Я свое мнение относительно представления высказываю в полном соответствии с законом. И то, что представитель обвинения пытается меня перебивать на каждом слове и закрывать мне рот, тоже отлично характеризует все происходящее. Потому что то, что я говорю, имеет прямое отношение к происходящему здесь. Продолжим.\n\nЯ нанес смертельную обиду тем, что выжил благодаря хорошим людям, пилотам и врачам. Потом я еще сильнее его обидел тем, что не спрятался, живя где-то под охраной в каком-то бункере поменьше, который я мог бы себе позволить. А потом случилось вообще страшное. Мало того, что я выжил, мало того, что я не испугался и не спрятался, — я участвовал еще и в расследовании своего собственного отравления, и мы показали и доказали, что именно Путин, используя ФСБ, осуществил покушение на убийство. И не я один был такой, сейчас это многие знают и еще узнают, и это сводит с ума маленького вороватого человечка в бункере. Вот именно этот факт, то, что все вскрылось, понимаете? Нет никаких рейтингов, нет никакой огромной поддержки — ничего этого нет, потому что выяснилось, что для того, чтобы совладать с политическим оппонентом, у которого нет ни телевидения, ни политической партии, нужно просто попытаться убить его химическим оружием. Конечно, он просто сходит с ума, потому что все убедились, что он просто чиновничек, которого случайным образом поставили на президентский пост, который никогда не участвовал ни в дебатах, ни в выборах, и это единственный метод его борьбы — пытаться убить людей.\n\nИ сколько бы он ни изображал из себя великого геополитика, великого мирового лидера, его главная обида в отношении меня теперь заключается в том, что в историю он войдет именно как отравитель. Знаете, был Александр Освободитель или Ярослав Мудрый, и будет у нас Владимир Отравитель Трусов. Именно так он войдет в историю.\n\n(Навальному пытаются возразить).\n\nВаша честь, это же имеет прямое отношение [к представлению]! Я здесь, стою на этом месте, меня охраняет полиция, Росгвардия появилась, половина Москвы оцеплена именно потому, что маленький человечек в бункере сходит с ума оттого, что мы доказали и показали, что он не геополитикой занимается, а проводит совещание, на котором решает воровать у политиков-оппонентов трусы и мазать их химическим оружием.\n\nГлавное в этом процессе даже не то, чем он закончится для меня, посадят меня или не посадят. Посадить меня несложно — по этому или другому делу. Главное, для чего это происходит, — чтобы запугать огромное количество людей. Это же так работает: одного сажают, чтобы испугать миллионы. У нас 20 миллионов человек за чертой бедности, у нас десятки миллионов живут, не имея ни малейших перспектив. У нас десятки миллионов людей относятся именно к тем, про кого мы каждый день говорим: в Москве жизнь еще более-менее, а отъедете за 100 километров — там вообще полный швах. И у нас вся страна живет в этом полном швахе, не имея ни малейших перспектив, получая 20 тысяч рублей. И они все молчат, и их пытаются заткнуть вот ровно такими показательными процессами. Посадить вот этого, чтобы испугать миллионы. Кто-то вышел на улицу — посадить еще пять человек, чтобы испугать 15 миллионов.\n\nГлавное, что я хочу сказать: этот процесс, я очень надеюсь, не будет воспринят людьми как сигнал того, что они должны больше бояться. Это же не демонстрация силы — Росгвардия, клетка. Это же демонстрация слабости, просто слабости. Миллионы и сотни тысяч посадить нельзя, и я очень надеюсь, что люди будут все больше и больше осознавать это. И когда они осознают — а такой момент придет, — все это рассыплется. Потому что вы не посадите всю страну. Всех этих людей, которых лишили перспектив, лишили будущего, которые живут в богатейшей стране и получают ноль от национальных богатств, ноль! Мы только по количеству миллиардеров растем, а все остальное падает, понимаете? Я сижу в своей камере и слышу по репортажам, как подорожало масло, подорожали макароны, подорожали яйца. 2021 год, страна — экспортер нефти и газа, а у нас вся страна говорит о том, что макароны подорожали и мы жить больше не можем! Вы этих людей лишили перспектив, и вы этих людей пытаетесь запугать. Я призываю всех не бояться. Вся эта власть основана ровно на этом.\n\n(Судья говорит Навальному, что он не на митинге, и предлагает вернуться к представлению).\n\nЯ не просто вернусь — я уже прямо там нахожусь, я в центральной части этого нахожусь. Вы говорите, что я ничего не сказал по поводу представления. Вот это вот все — представление и есть. И все, что я говорю, — это мое отношение к представлению, которое вы устроили.\n\nБывает такое, когда беззаконие и произвол являются сутью политической системы, и это ужасно, но бывает еще хуже: когда беззаконие и произвол наряжают себя в мундир прокурора или судейскую мантию. И в этом случае долг каждого человека — не подчиняться тем законам, которые обряжены в эти мантии. За вами, внутри вас, — произвол и беззаконие, и долг каждого человека — не подчиняться вам, не подчиняться таким законам.\n\n(Судья снова требует вернуться к представлению и говорит: «Давайте мы не будем политикой заниматься!»)\n\nКак же мы можем не заниматься политикой?! У меня сложилось мнение относительно этого представления, и я вам его высказываю. Другого мнения у меня нет, и будьте добры меня выслушать. Еще раз хочу сказать: когда произвол и беззаконие оделись в ваши мундиры и изображают из себя закон, долг каждого честного человека — не подчиняться вам и бороться с вами всеми силами. И я, как могу, борюсь и буду продолжать это делать. Хотя с учетом того, что сейчас я полностью оказался под контролем людей, которые обожают все намазывать химическим оружием, наверное, за мою жизнь никто не даст и три копейки. Но тем не менее даже сейчас, даже со своего места я говорю, что я буду с вами бороться, и призываю всех остальных не бояться вас и делать все, чтобы восторжествовал закон, а не ряженые в мундирах и мантиях. И я приветствую всех тех, кто борется и не боится, всех честных людей. Я приветствую и благодарю сотрудников ФБК, которые тоже сидят под арестом, и всех остальных по всей стране, кто не боится и выходит на улицы, потому что у них есть такие же права, как у вас. Потому что наша страна принадлежит им в той же самой степени, как и вам, как и всем остальным. Мы такие же граждане, и мы требуем нормального правосудия, нормального отношения к нам, участия в выборах, участия в распределении национальных богатств. Да, мы всего этого требуем!\n\nВ России сейчас много хороших вещей. А самая хорошая вещь — это вот те самые люди, которые не боятся, не опускают глаза, не смотрят в стол и никогда не отдадут нашу страну кучке продажных чиновников, решивших обменять нашу Родину на свои дворцы, виноградники и аквадискотеки.\n\nЯ требую немедленной свободы для себя и для других арестованных. Я не признаю это представление. Оно полностью лживо, оно не соответствует закону. И я требую немедленного освобождения.')]

replace_texts_by_urls(file_path, url_text_pairs_site)


Replaced text for the entry with URL: https://navalny.com/p/6653/
Replaced text for the entry with URL: https://navalny.com/p/6655/
Replaced text for the entry with URL: https://navalny.com/p/6654/
Replaced text for the entry with URL: https://navalny.com/p/6651/
Replaced text for the entry with URL: https://navalny.com/p/6647/
Replaced text for the entry with URL: https://navalny.com/p/6644/
Replaced text for the entry with URL: https://navalny.com/p/6642/
Replaced text for the entry with URL: https://navalny.com/p/6638/
Replaced text for the entry with URL: https://navalny.com/p/6634/
Replaced text for the entry with URL: https://navalny.com/p/6624/
Replaced text for the entry with URL: https://navalny.com/p/6623/
Replaced text for the entry with URL: https://navalny.com/p/6599/
Replaced text for the entry with URL: https://navalny.com/p/6564/
Replaced text for the entry with URL: https://navalny.com/p/6563/
Replaced text for the entry with URL: https://navalny.com/p/6530/
Replaced t

In [410]:
# Function to check for posts with specific phrases or urls

file_path = '../data/fbk_oleg_archive.csv'
searched_phrase = "Обыски, задержания и аресты — это так Путин"
author = 'Oleg Navalny'
date_range = ['2021-01-18', '9999-12-31']
domains = ['navalny.com']
#key_urls = ['https://t.me/navalny/2970']

found_entries = search_text_in_archive(
    search_phrase= searched_phrase, 
    file_path=file_path, 
    author= author, 
    #date_range= date_range
    #domains= domains 
    #key_urls=key_urls
    )

Specified phrase or URL not found.


In [436]:
# Function to count posts with specific author, url domain and dates

import csv
from datetime import datetime

def count_posts_by_author_and_domain(file_path, author, domain=None, date_range=None):
    # Read the existing archive
    with open(file_path, 'r', encoding='utf-8') as file:
        archive = list(csv.DictReader(file))
        
    # Function to check if a date is within a date range
    def is_within_date_range(date, date_range):
        date_format = "%Y-%m-%d"
        post_date = datetime.strptime(date, date_format)
        start_date, end_date = [datetime.strptime(d, date_format) for d in date_range]
        return start_date <= post_date <= end_date

    # Filtering logic based on author, domain, and optional date range
    if date_range:
        if domain:
            filtered_posts = [row for row in archive if row['author'] == author and domain in row['url'] and is_within_date_range(row['date'], date_range)]
        else:
            filtered_posts = [row for row in archive if row['author'] == author and is_within_date_range(row['date'], date_range)]
    else:
        if domain:
            filtered_posts = [row for row in archive if row['author'] == author and domain in row['url']]
        else:
            filtered_posts = [row for row in archive if row['author'] == author]

    # Sort the posts by date
    filtered_posts.sort(key=lambda x: x['date'])
    
    count = len(filtered_posts)
    print(f"Number of posts by {author} with domain {domain if domain else 'any'}: {count}")
    for post in filtered_posts:
        print({key: post[key] for key in ['url', 'date', 'text', 'author', 'source_type']})

    return count

# Usage
file_path = '../data/fbk_oleg_archive_clean.csv'
author = 'Navalny'
domain = 'facebook.com'
date_range = ['2008-01-18', '2021-01-18']
#date_range = ['2021-01-18', '9999-12-31']
count_posts_by_author_and_domain(file_path, author, domain, date_range)



Number of posts by Navalny with domain facebook.com: 239
{'url': 'https://facebook.com/navalny/posts/3462232960462429', 'date': '2020-07-20', 'text': 'Очень интересно. Вы, кстати, знали, что Белоруссия - лидер по количеству шенгенских виз на душу населения?\nИ степень урбанизации там выше, чем в России.', 'author': 'Navalny', 'source_type': ''}
{'url': 'https://facebook.com/navalny/posts/3465014990184226', 'date': '2020-07-21', 'text': 'https://navalny.com/p/6401/', 'author': 'Navalny', 'source_type': ''}
{'url': 'https://facebook.com/navalny/posts/3464671626885229', 'date': '2020-07-21', 'text': 'Дегтярёва ждет нелегкое губернаторство', 'author': 'Navalny', 'source_type': ''}
{'url': 'https://facebook.com/navalny/posts/3468427916509600', 'date': '2020-07-22', 'text': 'https://youtu.be/lLBvnZIxEuk', 'author': 'Navalny', 'source_type': ''}
{'url': 'https://facebook.com/navalny/posts/3467999193219139', 'date': '2020-07-22', 'text': 'Абсолютная беспочвенность обвинений была очевидна с сам

239

In [394]:
# Remove some unrelated sentences and links from Navalny's after-jail posts

import csv
import re
from datetime import datetime

def is_within_date_range(date, date_range):
    date_format = "%Y-%m-%d"
    post_date = datetime.strptime(date, date_format)
    start_date, end_date = [datetime.strptime(d, date_format) for d in date_range]
    return start_date <= post_date <= end_date

def modify_text_by_multiple_key_wordings(file_path, author, key_wordings_and_texts_to_delete, date_range=None):
    # Read the existing archive
    with open(file_path, 'r', encoding='utf-8') as file:
        archive = list(csv.DictReader(file))

    modified_entries = 0  # Counter for modified entries

    # Loop through the archive to find entries matching the author and key wording
    for row in archive:
        if row['author'] == author:
            if date_range and not is_within_date_range(row['date'], date_range):
                continue
            for key_wording, text_to_delete in key_wordings_and_texts_to_delete:
                if key_wording in row['text']:
                    if key_wording == 'http':
                        row['text'] = re.sub(r'https?://[^ ]+', '', row['text']) # for both https and http links
                    else:
                        row['text'] = row['text'].replace(text_to_delete, '')
                    modified_entries += 1

    # Write the modified archive back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['author', 'text', 'date', 'url', 'source_type'])
        writer.writeheader()
        writer.writerows(archive)

    print(f"Modified {modified_entries} entries.")
    return modified_entries

# Usage
file_path = '../data/fbk_oleg_archive_clean.csv'
author = 'Oleg Navalny'
date_range = ['2015-10-24', '9999-12-31']

# Instagram links to delete from texts
insta_links_to_delete = [('https://www.instagram.com/p/', '')]

# All links to delete from texts
all_links_to_delete = [('http', '')]

# Sentences to delete
sentences_to_delete = [('Сегодня в Бабушкинском суде', 'Сегодня в Бабушкинском суде у Навального состоялось сразу два заседания — апелляция по «делу Ив Роше» и приговор по делу о клевете. Первый суд оставил в силе замену условного срока на реальный, несмотря на требование ЕСПЧ немедленно освободить Алексея. Второй приговорил Навального к штрафу в размере 850 тысяч рублей.В обоих процессах Алексей говорил последнее слово. Мы публикуем оба.Апелляция по «делу Ив Роше»'),
('Этот пост Алексей прислал несколько дней назад', 'Этот пост Алексей прислал несколько дней назад. Вчера стало известно, что он «убыл» из СИЗО «Матросская тишина». Где он сейчас, мы не знаем. При этом, согласно решению ЕСПЧ, Алексей должен быть немедленно освобождён.\n\n'),
('Посмотреть эту публикацию в InstagramПубликация', 'Посмотреть эту публикацию в InstagramПубликация от Алексей Навальный (@navalny)'),
('В газетах The Guardian, Frankfurter', 'В газетах The Guardian, Frankfurter Allgemeine Zeitung и Le Monde вышла статья Алексея, приуроченная ко дню его отравления год назад.Мы публикуем оригинал этой статьи на русском:'),
('Сегодня в газете New York Times вышло', 'Сегодня в газете New York Times вышло интервью Алексея — о внешней и внутренней политике, о его пребывании в тюрьме и тюремном быте, о рейтингах власти и оппозиции и о предстоящих выборах.В материал NYT вошли цитаты Алексея. Мы публикуем оригинал этого интервью на русском языке. Ниже — все вопросы, заданные журналистами, и ответы в том виде, в котором Алексей их написал.'),
('15 февраля в покровской колонии', '15 февраля в покровской колонии начался суд над Алексеем. По двойному делу о «мошенничестве» и «оскорблении суда» ему могут продлить срок еще на 15 лет. Публикуем речь, которую он произнес в первый день этого нелепого процесса.'),
('Ваша честь, вы видите каждый день,', 'Ещё\n\nСегодня прошла кассация на приговор Алексею Навальному по делу о «мошенничестве». Весной его судили прямо в колонии и дали 9 лет лишения свободы.\n\n'),
('Преступная власть проигрывает', 'Речь Навального в суде:'),
('Преступная власть проигрывает', 'Речь Алексея Навального на заседании кассационного суда по делу о мошенничестве и неуважении к суду — о путинской войне, мобилизации, борьбе с несогласными, будущем России и происходящей катастрофе.Поддержите нашу работу с помощью спонсорства на YouTube или через наш сайт.'),
('В Washington Post на русском и английском', 'В Washington Post на русском и английском, а во Frankfurter Allgemeine Zeitung на немецком вышла статья Алексея Навального о том, что должно стать настоящим концом путинской преступной войны и какой должна быть Россия будущего:'),
('Алексея Навального в пятый раз подряд', 'Алексея Навального в пятый раз подряд отправили в ШИЗО - за то, что он сказал о мобилизации в суде 21 сентября.Послушайте, что он сказал в суде сегодня:'),
('Сегодня суд оставил в силе приговор Алексею — 9 лет', 'Сегодня суд оставил в силе приговор Алексею — 9 лет строгого режима.Мы публикуем полную расшифровку последнего слова Навального в суде.— '),
('Путин вошел в топ-100 самых влиятельных людей 2022', 'Путин вошел в топ-100 самых влиятельных людей 2022 года по версии журнала TIME. Они попросили Алексея Навального написать колонку об этом из тюрьмы.Ниже оригинал текста Алексея:'),
('Суд над Алексеем Навальным и Даниэлем Холодным закончен', 'Суд над Алексеем Навальным и Даниэлем Холодным закончен. Обвинение требует Навальному по совокупности приговоров 20 лет лишения свободы с отбыванием в колонии особого режима. Холодному - 10 лет лишения свободы с отбыванием в колонии общего режима. Оглашение приговора - 4 августа в 16.00.\u2028\u2028Последнее слово Алексея Навального:\n\n«'),
('Алексея Навального в пятый раз подряд', 'Алексея Навального в пятый раз подряд отправили в ШИЗО - за то, что он сказал о мобилизации в суде 21 сентября (ссылка на это выступление в сторис)\n\nПослушайте, что он сказал в суде сегодня:\n\n'),
]

oleg_link = [('youtu.be/_Ss1PUOIdpA', 'youtu.be/_Ss1PUOIdpA')]

modify_text_by_multiple_key_wordings(file_path, author, oleg_link, date_range)


Modified 1 entries.


1

In [381]:
# To manually add some latest Navalny's posts

import csv

def add_entries_to_archive(file_path, new_entries):
    # Open the file in append mode ('a')
    with open(file_path, 'a', encoding='utf-8', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['author', 'text', 'date', 'url', 'source_type'])
        
        # Write the new rows to the file
        writer.writerows(new_entries)

    print(f"Added {len(new_entries)} new entries to the archive.")

# Usage:

new_entries = [
    {'author': 'Navalny', 'text': 'Многие удивятся, но выборы все ещё существуют. 10 сентября они пройдут в 40 регионах, в том числе в Екатеринбурге, в Омской области, в Хабаровске, в Красноярске, в Ярославской и Архангельской областях, в Хакасии и Якутии. Таким образом, нам надо определиться с отношением к этим выборам, и я позволю себе внести предложения. Начиная с 2011-го именно предложенные нами тактики голосования признавались наиболее эффективными. Хотя каждый раз их приходилось отстаивать в ожесточённых дискуссиях. В авторитарной стране, которой является Россия, сам по себе факт получения большего количества голосов совсем не гарантирует ни успеха, ни победы. Что мы, в общем-то, и наблюдаем, учитывая, где я пишу этот пост. Поэтому мы расцениваем участие в выборах как важнейший, но далеко не единственный элемент политической борьбы. Фразу «власть в России сменится не в результате выборов» повторили все много раз, но впервые произнёс её я в 2011 году. Однако это совсем не отменяет важности правильной предвыборной тактики. Главное, что надо понять и запомнить (это лежит в основе каждой нашей выборной стратегии): все выборы различаются. Надо принимать решения, исходя из ситуации. Универсальных правил нет. Мы должны адаптироваться, отвечая на новые трюки и изобретения власти. Ведь посмотрите, с первого года президентства Путина правила меняются в каждом выборном цикле, чтобы наша ранее эффективная стратегия не сработала. И мы будем идиотами, если продолжим делать одно и то же, а не менять стратегию в зависимости от новых правил. В 2011 году наша кампания «голосуй за любую партию, кроме „Единой России“» нанесла поражение партии власти. Они получили менее 50% голосов и для того, чтобы не потерять большинство в Государственной думе, пошли на совершенно открытые фальсификации в Москве. Что и привело к широкому протестному движению впервые с перестроечных времён и к формированию оппозиции нового типа. В 2013 году мы впервые в истории России провели избирательную кампанию на выборах мэра Москвы, опираясь на волонтёров и активистов, и добились высокого результата без олигархических денег и административного ресурса. И вновь от второго тура и невероятного поражения единоросса Собянина спасла только фальсификация. В 2015 году мы обратили всеобщее внимание на региональные выборы, которые до этого столичной оппозицией игнорировались. Однако тут уже власть, поняв, что мы и в регионах её потреплем, не пустила на выборы людей из наших списков, несмотря на протесты и голодовки. Самой успешной стратегией стало Умное голосование, впервые использованное в 2019 году. До этого в России существовало железное политическое правило: оппозиционный кандидат не может выиграть в одномандатном округе. Для этого надо получить не менее 35–40% голосов, а это невозможно без помощи губернатора или мэра. Однако в 2019 году мы просто с первой попытки разгромили в Москве и Собянина, и Путина, и «Единую Россию». Несмотря на то, что почти никого из независимых команд кандидатов на выборы не пустили, Умное голосование, поддержавшее сильнейших кандидатов от системной оппозиции, выиграло тогда вместе с ними 24 округа из 45, лишив «Единую Россию» большинства в столичном парламенте. Как обычно, власть спасли фальсификации. Мэрия Москвы украла четыре мандата и сохранила свое большинство. Но всё же в столичный парламент прошло рекордное количество реально оппозиционных кандидатов. Поняв, что выиграть у нас с вами в крупных городах они уже не смогут, в ответ на Умное голосование власти изобрели «Дистанционное электронное голосование» (ДЭГ). Это система, которая просто рисует нужным кандидатам несуществующие голоса, и проверить это никак нельзя: наблюдают за системой сами организаторы фальсификаций. Несмотря на это, Умное голосование отлично себя показало и на региональных выборах 2020 года. Меня тогда и отравили, пытаясь убить в Томске, куда я приехал, чтобы поддержать независимых кандидатов в Городскую думу. Они, кстати, в итоге стали депутатами, о чём я узнал, уже лежа в больнице. По одномандатным округам кандидаты Умного голосования победили в Томске в 19 округах из 27. И даже на последних, уже военного времени, выборах в сентябре 2022 года — на выборах депутатов муниципальных советов Москвы, — в соответствии с рекомендациями Умного голосования, которое поддерживало только антивоенных кандидатов, проголосовало более 400 тысяч избирателей (из 700 тысяч, пришедших физически на избирательные участки). Опять пришлось воровать мандаты с помощью ДЭГа — якобы в электронном виде голоса отдали 1,7 млн избирателей, — и всё равно 63 кандидата, поддержанных нами, получили депутатские мандаты. И вот перед нами вопрос о том, как вести себя на выборах 2023-го. С 2011 года мы постоянно боремся с двумя типами дуриков. Первые кричат, что все выборы — это плохо, это работа на Путина, выборы ничего не решают и всё надо бойкотировать. Вторые — такие же дурики, но в противоположной крайности. Эти мчатся участвовать во всём, что власть назвала голосованием, даже если оно таковым совсем не является. Этим хоть на пчелиный улей, пенёк или на деревянный туалет повесь табличку «выборы», они сразу выстроятся в очередь. А каждые выборы или «выборы» надо анализировать, чтобы понять: чего мы можем добиться? Мы ослабим власть или усилим её, участвуя в выборах? Есть ли тут возможности хоть для какого-то контроля? Есть ли возможность для агитации? Чего от нас ждут? В чём подвох (а он всегда есть)? Исходя из этого, надо где-то голосовать и агитировать изо всех сил. Где-то просто наблюдать. Где-то игнорировать. Где-то вести кампанию забастовки и активного бойкота. В чем особенность выборов-2023? 1. Наша лучшая стратегия, Умное голосование, на них неприменима. По крайней мере, в прежнем виде. Объясняю. Ситуация 1. Есть отличный кандидат, Умное голосование поддерживает его. Его моментально не просто снимают с выборов, а сажают. Ситуация 2. Есть просто кандидат-неединоросс, который может победить с помощью Умного голосования. Мы поддерживаем его. Кандидата моментально везут на фронт, где он фотографируется, обвешанный автоматами, гранатами и отрезанными головами. После этого мы с вами занимаемся не выборами и агитацией — мы отбиваем в твиттере истерики о том, что Умное голосование поддерживает людоедов. Сама власть истерики и инициирует. Через «твёрдых собянинцев» в том числе 2. Уровень фальсификаций, в том числе через электронное голосование, так высок, а наблюдение так ограничено, что в этих выборах почти не осталось выборов. Сразу оговорюсь: не везде. Где-то можно и нужно участвовать. Где-то мы и Умное голосование применим втихаря, насколько это возможно. Но в целом с возможностью защитить результат всё не очень. 3. Снова: все выборы разные. Например, выборы в Хакасии, где даже дебаты проходят между губернатором-коммунистом и соперничающим с ним единороссом, присланным из Москвы, интересны и конкурентны. Или выборы в Гордуму Екатеринбурга, где воюют между собой мэр города и губернатор области. А выборы мэра Москвы-2023 устроены так, что как ни проголосуй — проголосуешь за Путина. Либо за Собянина, либо за «путинское крыло КПРФ», представленное Зюгановым-внуком. Если пытаться устраивать протестное голосование в Москве, то вот заранее подготовлен канал для такого протеста: Зюганов-внук. Кампанию не ведёт. Власть хвалит. Делает что велят. Протестное голосование может немного добавить ему голосов, но это полная калька кремлёвской тактики с Собчак на выборах президента-2018: «Ах, у нас есть оппозиция, ну так вот тебе, оппозиция, кандидат. Давай, агитируй за него, а мы потом посмеёмся над результатами». Ну и интерес к московским выборам соответствующий. Это третья по важности избирательная кампания в России. А я уверен, что большинство из вас не знает кандидатов, кроме Собянина и Зюганова. Так можно ли чего-то добиться на выборах-2023? Я считаю, что можно. Просто наши цели возвращаются к самым базовым: 1. Мы, оппозиционные избиратели, должны сохранить привычку и правило ходить на выборы. Рано или поздно в России они пройдут относительно свободно. Мы должны на них победить. Этого не получится, если мы убедим себя, что выборы не имеют смысла и значения, и привыкнем на них не ходить. 2. Надо поддерживать желание людей баллотироваться. Немало хороших кандидатов в регионах. Есть прямо отличные. Просто лично я нахожусь в ситуации, когда мне лучше их фамилии не называть. Но в любом случае им нужны наши голоса и поддержка. И точно не нужно нытьё о том, что всё бесполезно. 3. Крайне важно сохранить остатки инфраструктуры наблюдения. Как я сказал, рано или поздно в России пройдут относительно свободные выборы, но и на этих выборах у нас вновь попытаются украсть голоса. Нам нужны будут опытные наблюдатели. Наконец-то перехожу к тому, что делать. Я считаю, что в нынешней ситуации нам надо сделать шаг назад и вернуться к стратегии голосования «за любого кандидата против „Единой России“». Идите на выборы с целью нанести максимально возможный урон партии власти и её кандидатам. Резюме: 1. 10 сентября надо идти на участки и голосовать. 2. Выберите в бюллетене кандидата, который не является членом «Единой России», и голосуйте за него. Хорошие кандидаты есть, и много. 3. Те, кто участвует в выборах как независимые кандидаты, — молодцы, и их стоит поддерживать. 4. Умное голосование мы делать будем, но точечно и непублично. 5. Обязательно стоит участвовать в наблюдении за выборами, для этого использовать любые возможности. Наблюдательские организации существуют до сих пор, и многие толково работают. 6. Выборы мэра Москвы 2023 значения не имеют. Они фиктивны. Можно проголосовать, ради любопытства сходив на участок, можно игнорировать. Они не стоят того, чтобы их вообще обсуждать. 7. Тактика по президентским выборам в марте 2024 года может быть определена только после того, как будет ясен общий расклад с правилами, кандидатами и наблюдением. Стратегически мы считаем Умное голосование и его участников главным достоянием и лучшей возможностью оппозиции. Но ключевое слово здесь — «умное». Оценив новые обстоятельства, войну и политическую ситуацию, мы должны поступить умно и сделать шаг назад, голосуя за любого кандидата и за любую партию против «Единой России».', 'date': '2023-08-21', 'url': 'https://navalny.com/p/6653/', 'source_type': ''},
    {'author': 'Navalny', 'text': 'В связи с терактом, который устроил Путин, чтобы убить своего солдата Пригожина, хочется сказать о порядочности, хитроумии и стратегии. Порядочность. Буквально пару дней назад писал одному человеку, объясняя, в чем секрет выживаемости Пригожина. Скорее всего, всё просто. Личная договорённость и путинскоe пацанское слово. Пригожин обещал Путину, что в лепёшку расшибется для него и его войны. Будет его самым надёжным командиром, готовым пойти на всё ради победы. Путин, глядя на, мягко говоря, отсутствие энтузиазма у Министерства обороны, это оценил. Он дал Пригожину небывалого уровня поддержку (шутка ли, тысячи зеков за один день из тюрем вынимали). Ну и был — предполагал я — некий классический диалог: — Я за вас на всё пойду, Владимир Владимирович. Победа будет за нами любой ценой. Но и вы уж меня не подведите и на съедение не отдавайте. — Женя, вот тебе моя рука и моё слово. Выполни задачу, а я всегда за твоей спиной. Поддержка гарантирована. Во время «мятежа» Пригожин психанул «по делу», и сути отношений это не изменило. Путинское слово продолжает действовать. Какой глупостью было написать это. Какая наивность — предположить, что Путину может быть свойственна порядочность любого вида: офицерская, пацанская, бандитская. Там же нет ничего, кроме постоянного вранья по абсолютно любому поводу. Ври, воруй, отнимай, убивай, убегай. Хитроумность. В этом пункте снова стоит посмеяться над теми, кто верит в ювелирную работу спецслужб, многоходовые планы и «захотят убить — там умеют убивать так, что комар носа не подточит». Вот вам тонкая работа путинских убийц. Сбить/взорвать над Россией пассажирский самолёт, то есть устроить настоящий теракт, убив в нём ещё и абсолютно не причастных пилота, стюардессу и т. д. Стратегия. По стране ходят 50–60 тысяч бывших зеков и просто бывших участников войны, облепленных нашивками «Вагнер». Ещё несколько сотен тысяч поклонников этого культа. Его официально воспевали, и он довольно популярен: «Вагнер», «оркестр», «музыканты», «наши в Африке», продвигают российские интересы и так далее. Пригожина и Уткина награждают звёздами героев. Два президента, России и Беларуси, дают им гарантии безопасности. А теперь — что произошло глазами поклонников «Вагнеров»: русские офицеры-герои не щадили себя на войне в Украине. Это супермегаспецназовцы-профи. Их обманули предатели-генералы. Их сослали в Африку — они и там заявили, что будут укреплять мощь России, начинать экспансию и вновь заставят считаться с русским оружием. Но предатель и трус Путин в своём Кремле завидует популярности этих русских героев в народе и войсках и ненавидит их за это. Поэтому отдал подлый приказ убить их. Таким образом, Пригожин и Уткин в глазах многих из тех чувачков, что любят к месту и не к месту наряжаться в камуфляж, превратились в мучеников. В «легенды» этой войны. В глазах историков, кстати, тоже легенды и самые интересные персонажи. Фильмы, книги и конспирологические теории о том, что они выжили и воюют в джунглях Конго. Но дело даже не в том, что для части самых агрессивных сторонников войны с Украиной история этой войны — это теперь сага о том, как Путин предал и убил своих верных офицеров. А в том, что именно из таких ингредиентов создается блюдо «гражданская война». Создали банду. Вооружили банду. Распустили банду. Убили главарей. Метафора «пауки в банке» часто применяется к тем, кто сидит в Кремле. Но больше подходит другая — «обезьяна с гранатой».', 'date': '2023-08-24', 'url': 'https://navalny.com/p/6655/', 'source_type': ''},
    {'author': 'Navalny', 'text': 'С маршем на Вашингтон за рабочие места и свободу, прошедшим в этот день в 1963 году, меня разделяют десятилетия и тысячи километров. Но последние полгода это историческое мероприятие стало мне так близко, что иногда мне кажется, что я там был. Дело в том, что, сидя в ШИЗО без ручки и бумаги, занимаешь себя тем, что учишь что-то наизусть. В какой-то момент я выучил легендарную речь Мартина Лютера Кинга I have a dream и произнёс её, гуляя по камере, кажется, раз двести. Каждый раз восхищаюсь ею, как в первый, и мне не надоедает это.  Речь обращена к американцам и посвящена американской политике. Но она, как и сам Кинг, безусловно, достояние всего человечества. Поздравляю всех с годовщиной того, что они у нас есть.', 'date': '2023-08-28', 'url': 'https://t.me/s/navalny/3488', 'source_type': ''},
    {'author': 'Navalny', 'text': 'Начали приходить письма с отзывами о моем посте про страх и ненависть. Читаю с огромным интересом. Большое спасибо всем, кто обратил внимание на эту дискуссию. Повторюсь, она не теоретическая, не историческая, не политологическая. Это самая актуальная дискуссия о том, как победить «путинский режим». Ведь этот режим — это не только Путин персонально, но и часть элиты (в широком смысле), которая считает текущий способ управления нашей страной подходящим. «Есть, конечно, перегибы, но в целом приемлемо. Иначе нельзя. Такой народ» (c). А у нас — оппозиции — другая точка зрения… «Эээ…а какая?» — уточняют у нас избиратели. А мы, выясняется, до сих пор не определились с вопросом, нужно ли обсуждать подделку выборов и коррупцию. Победа над «путинским режимом» в конечном итоге означает победу над его сторонниками на выборах. А у нас на знамёнах Чубайс, Собчак, ельцинская Конституция и союз с ЧВК Вагнера имени Ходорковского. Класс. Гарантированные 1,5% на несвободных выборах и 0,5% на свободных. Обо всём этом я постараюсь подробно написать в своей реакции на реакции, когда все прочитаю и обдумаю. А пока два очень важных замечания по ходу дискуссии. Первое. Эту часть я переписывал три раза, каждый раз снижая уровень оскорблений и персональных нападок, пока не дошел до доброжелательного и конструктивного тона (кажется). Я веду блог лет 20 и написал тысячи постов. Я сижу уже 2,5 года и тоже сумел что-то написать. Но вот во второй раз в своей жизни (первый после поста «Мне стыдно за Михаила Ходорковского») я сталкиваюсь с реакцией в виде откровенной и наглой лжи о том, что тексты якобы пишу не я. Ложь эта варьируется от прямой («это пишет не Навальный»): до ещё более тошнотворного виляния и намёков в стиле «многие говорят, что писал не он, но опровержений нет, поэтому, наверное, он, но вообще кто его знает». Обратите внимание, что это не просто враньё в соцсетях, а кто-то даже оплатил и разместил эту заказную статью. Тут надо понять, что во лжи обвиняют меня. Я ж, выходит, вам вру. Кто-то пишет за меня, а я присваиваю себе его тексты. И не технические какие-то, а программные. Не надо придуриваться и предполагать, что я могу этого не знать. Уровень моей изоляции весьма высок. Но я, как и любой зек, имею право на переписку и на любое своё высказывание. Я уже высказывался о «сомнениях в авторстве» тех слов ясно и повторяю последний раз доброжелательно и конструктивно, но твёрдо. Часть из тех, кто повторяет эту ложь, просто глупенькие. Им простим. Часть — лицемеры. Знают, что врут, но придуриваются. Это тоже ладно. Но организаторы и постоянные настойчивые промоутеры этой идеи не только отвратительные негодяи и бессовестные люди, но и публичные доносчики (стукачи). Объясню. Мне за что только что 19 лет дали? За слова. В буквальном смысле. За посты, видеоролики, которые путинский суд признал экстремистскими. Значительная часть из них размещена ПОСЛЕ того, как я сел. А в идущем сейчас следствии по делу о терроризме суть обвинения та же — публикации в интернете. Но все они размещены уже после моей посадки. Вы не представляете, до какой степени совпадают интересы и аргументация тех лжецов и жуликов, что обсуждают моё авторство в соцсетях, и путинских следователей. — А вы это пишете? Подтвердите авторство. А как вы это пишете? Нет, ну давайте обсудим. Как вот технически это делается? Телефон у вас есть или передаёте через кого-то? Через адвокатов или через письма? Нам всё-всё надо узнать. Объяснитесь перед нами. Потому то, что делают эти мерзавцы, и есть публичный донос. Они отлично всё знают, всё понимают, но продолжают своё: «А чем ты докажешь, что это ты? А ну-ка раскрой нам все детали. А то мы так и будем продолжать каждый раз наш блядский цирк с тем, что пишешь не ты». «А вы их, товарищ Бастрыкин и товарищ Бортников, проверьте. Вы всё проверьте и обратите внимание. Как это он сидит и продолжает нехорошие вещи и про товарища Путина писать». «Мы его сейчас вынудим всё рассказать, а вы уж примите меры. Всю цепочку накройте. Безобразия эти прекращайте». Это всё именно так и есть. Не выглядит публичным доносом, а с учётом настойчивости им является. И мне самому, положим, нечего терять, но ведь я-то не один. Поэтому у меня такое предложение ко всем нормальным людям: а) если видите какие-то очередные рассуждения об авторстве текстов политзеков (кроме очевидных случаев взлома соцсетей, но это сразу ясно), тем более с «а докажи нам, что это ты» и обсуждением механизма, — знайте: перед вами лжец, негодяй и доносчик. Никогда больше не доверяйте такому человеку. б) Видите такое — заходите в комментарии и возите негодяя лицом об стол. Вопрос этот касается далеко не меня одного. Политзеков много и будет больше. Все эти тупые обсуждения осложнят жизнь и им, и их родственникам, и адвокатам. Важное замечание номер два. Чуть ли не самая популярная реакция от тех, кто посчитал мой пост критикой в свой адрес, выглядит так: «Всё Навальный неправильно написал, но я не могу критиковать его, пока он сидит в тюрьме». Призываю абсолютно всех: давайте перестанем так рассуждать, это глупости. В тюрьмах сидит полмиллиона человек. Их всех критиковать нельзя? И не все политзеки обязательно являются какими-то хорошими и святыми людьми. Само по себе сидение в тюрьме не делает никого лучше или хуже. Судить человека надо по его делам, а не по месту нахождения. С сидящими не нужно спорить только по всяческим тюремным темам, потому что вы в них ничего не понимаете, если сами не сидели. И даже если прочитали кучу книг об этом и родственник ваш сидел — всё равно не понимаете. В этом я на собственном примере убедился. Да и не надо вам в этом ничего понимать, оно того не стоит. И самое важное. Вы же меня так самого главного лишаете. И самого дорогого. Критики. Как мне иначе-то думать, работать и улучшать себя? Критика и конкуренция — основа прогресса, без них мне (и ФБК) никак. Например, я ругаю Алексея Венедиктова и продолжу это делать. Но, объективно говоря, именно он оказал мне (и ФБК) самую ценную услугу за последний год. Он опубликовал известное письмо Леонида Волкова. Спровоцировал огромную волну критики. Была важнейшая дискуссия и вообще, и внутри ФБК. Мы были вынуждены справляться с кризисом. Лёня ушел в отставку с поста главы международного ФБК. Мы изменили некоторые наши внутренние процедуры. Мы думали, как исправить проблему, и исправили её. Леонид очень подробно и публично объяснил свои действия (раз, два, три). Я об этом тоже сделал заявление. ФБК получил критический опыт, исправил ошибку и стал лучше. Волков отреагировал как ответственный политик, исправил ошибку и стал лучше. Сейчас он очень успешно работает над нашим важнейшим проектом. Я тоже тогда получил свою долю критики и навсегда учту этот опыт в своей работе. Вы, в свою очередь, тоже получили полезные знания: увидели, как ФБК справляется с кризисной ситуацией. Исправляет ли ошибку. Действует ли ответственно. Теперь вы имеете большее представление о том, чего от нас ожидать и как мы будем действовать, в том числе оказавшись во власти. Вы можете сравнить наше поведение в кризисной ситуации с поведением других политиков и политических организаций. Критика и конкуренция оказались полезны всем. В ситуации с моим постом (программным) то же самое. Я нуждаюсь в вашем мнении. Если вы согласны — это важно, и я это отмечу. Если не согласны — я либо опровергну ваши аргументы, либо соглашусь с ними. Знаете, для меня давно фраза «оппозиция снова ругается» — маркер идиота. Это политические дебаты, даже если они в твиттере или комментах фейсбука. Без них нет жизни и нет прогресса. Если вам хочется, чтобы люди с разными точками зрения не спорили между собой, то вам в «Единую Россию» или в КПСС. Так что критикуйте меня, пожалуйста, если не согласны, я без этого никуда.', 'date': '2023-08-29', 'url': 'https://navalny.com/p/6654/', 'source_type': ''}
]

file_path = '../data/fbk_oleg_archive_clean.csv'  # Replace with your actual file path

add_entries_to_archive(file_path, new_entries)


Added 4 new entries to the archive.


In [427]:
# Exctract Navalny_jailed and Oleg Navalny (jailed & free) texts for further analysis

import pandas as pd
from datetime import datetime

def extract_texts_by_author_and_date(csv_path, output_txt_path, author, date_range=None):
    # Read the CSV into a DataFrame
    df = pd.read_csv(csv_path)
    
    # Filter by author
    df_filtered = df[df['author'] == author]
    
    # Filter by date range if provided
    if date_range:
        start_date, end_date = date_range
        df_filtered['date'] = pd.to_datetime(df_filtered['date'])
        df_filtered = df_filtered[(df_filtered['date'] >= start_date) & (df_filtered['date'] <= end_date)]
    
    # Extract the texts
    texts = df_filtered['text'].tolist()
    
    # Save to a text file
    with open(output_txt_path, "w", encoding='utf-8') as f:
        for text in texts:
            f.write(f"{text}\n")
    
    print(f"Successfully saved {len(texts)} texts by {author} from {start_date} to {end_date} into a text file.")

# Extract Navalny_jailed texts
csv_path = '../data/fbk_oleg_archive_clean.csv'
author_N = 'Navalny'
date_range_N = ['2021-01-18', '9999-12-31']  # format: YYYY-MM-DD
output_txt_path_N = '../data/NavalnyJailed_raw.txt'
extract_texts_by_author_and_date(csv_path, output_txt_path_N, author_N, date_range_N)

# Extract Oleg_Navalny_jailed text
author_O = 'Oleg Navalny'
date_range_OJ = ['2015-10-24', '2018-07-01']  # format: YYYY-MM-DD
output_txt_path_OJ = '../data/OlegJailed_raw.txt'
extract_texts_by_author_and_date(csv_path, output_txt_path_OJ, author_O, date_range_OJ)

# Extract Oleg_Navalny_free text
author_O = 'Oleg Navalny'
date_range_OF = ['2018-07-01', '9999-12-31']  # format: YYYY-MM-DD
output_txt_path_OF = '../data/OlegFree_raw.txt'
extract_texts_by_author_and_date(csv_path, output_txt_path_OF, author_O, date_range_OF)

C:\Users\User\AppData\Local\Temp\ipykernel_12748\3512343986.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date'])


Successfully saved 243 texts by Navalny from 2021-01-18 to 9999-12-31 into a text file.


C:\Users\User\AppData\Local\Temp\ipykernel_12748\3512343986.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date'])


Successfully saved 160 texts by Oleg Navalny from 2015-10-24 to 2018-07-01 into a text file.
Successfully saved 60 texts by Oleg Navalny from 2018-07-01 to 9999-12-31 into a text file.


C:\Users\User\AppData\Local\Temp\ipykernel_12748\3512343986.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date'])


In [428]:
# Remove some weird NaNs from Oleg's texts

def remove_word_from_file(input_txt_path, output_txt_path, word_to_remove):
    with open(input_txt_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    updated_lines = []
    count_removed = 0
    for line in lines:
        count_removed += line.count(word_to_remove)
        # Remove the specific word
        line = line.replace(word_to_remove, '')
        updated_lines.append(line)

    with open(output_txt_path, 'w', encoding='utf-8') as f:
        for line in updated_lines:
            f.write(line)
    
    print(f"Removed {count_removed} instances of the word '{word_to_remove}'.")

# Usage
txt_path_OJ = "../data/OlegJailed_raw.txt"
txt_path_OF = "../data/OlegFree_raw.txt"
word_to_remove = "nan"

remove_word_from_file(txt_path_OJ, txt_path_OJ, word_to_remove)
remove_word_from_file(txt_path_OF, txt_path_OF, word_to_remove)

Removed 21 instances of the word 'nan'.
Removed 1 instances of the word 'nan'.


In [403]:
# Clean imported raw text files for Navalny_jailed and Oleg Navalny (jailed & free)

import re

def clean_text_file(input_txt_path, output_txt_path):
    with open(input_txt_path, 'r', encoding='utf-8') as f:
        texts = f.readlines()
    
    cleaned_texts = []
    for text in texts:
        # Remove URLs
        text = re.sub(r'http[s]?://\S+', '', text)
        
        # Remove other unwanted characters (any character that is not a word character or a whitespace)
        text = re.sub(r'[^\w\s]', '', text)
        
        # Remove extra white spaces
        text = ' '.join(text.split())
        
        # Remove line breaks
        text = text.replace('\n', ' ')

        # Convert to lowercase
        text = text.lower()
        
        cleaned_texts.append(text)
    
    with open(output_txt_path, 'w', encoding='utf-8') as f:
        for text in cleaned_texts:
            f.write(f"{text}\n")

    print(f"Cleaned text saved to {output_txt_path}")

# Usage
# Navalny jailed
input_txt_path_N = '../data/Navalny_jailed_texts_raw.txt'
output_txt_path_N = '../data/Navalny_jailed_texts_clean.txt'
clean_text_file(input_txt_path_N, output_txt_path_N)

# Oleg Navalny jailed
input_txt_path_OJ = '../data/oleg_jailed_texts_raw.txt'
output_txt_path_OJ = '../data/oleg_jailed_texts_clean.txt'
clean_text_file(input_txt_path_OJ, output_txt_path_OJ)

# Oleg Navalny free
input_txt_path_OF = '../data/oleg_free_texts_raw.txt'
output_txt_path_OF = '../data/oleg_free_texts_clean.txt'
clean_text_file(input_txt_path_OF, output_txt_path_OF)


Cleaned text saved to ../data/oleg_jailed_texts_clean.txt
Cleaned text saved to ../data/oleg_free_texts_clean.txt
